## Imports and constants

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as st
import statsmodels.api as sm

import os
import time
import random
import itertools

from copy import copy
from cycler import cycler
from datetime import date
from scipy.optimize import curve_fit
# from statannot import add_stat_annotation

pd.set_option('mode.chained_assignment',None)
pd.set_option('display.max_columns', None)

# Add the folder to path
COMPUTERNAME = os.environ['COMPUTERNAME']
if COMPUTERNAME == 'ORDI-JOSEPH':
    mainDir = "C://Users//JosephVermeil//Desktop//ActinCortexAnalysis"
elif COMPUTERNAME == 'LARISA':
    mainDir = "C://Users//Joseph//Desktop//ActinCortexAnalysis"
elif COMPUTERNAME == '':
    mainDir = "C://Users//josep//Desktop//ActinCortexAnalysis"

experimentalDataDir = os.path.join(mainDir, "Data_Experimental")
dataDir = os.path.join(mainDir, "Data_Analysis")
figDir = os.path.join(dataDir, "Figures")
todayFigDir = os.path.join(figDir, "Historique//" + str(date.today()))
timeSeriesDataDir = os.path.join(dataDir, "TimeSeriesData")

import sys
sys.path.append(mainDir + "//Code_Python")

from getExperimentalConditions import getExperimentalConditions
from PincherAnalysis_JV import *

%load_ext autoreload

In [2]:
# Matplotlib

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

# %matplotlib inline
%matplotlib widget 
matplotlib.rcParams.update({'figure.autolayout': True})

# Fontsizes

SMALLER_SIZE = 10
SMALL_SIZE = 14
MEDIUM_SIZE = 16
BIGGER_SIZE = 20

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=MEDIUM_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALLER_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

# Bokeh

from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, HoverTool, Range1d
from bokeh.transform import factor_cmap
from bokeh.palettes import Category10
from bokeh.layouts import gridplot
output_notebook()

# Markers

my_default_marker_list = ['o', 's', 'D', '>', '^', 'P', 'X', '<', 'v', 'p']

# Colors

# prop_cycle = plt.rcParams['axes.prop_cycle']
# colors = prop_cycle.by_key()['color']
my_default_color_list = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']
my_default_color_cycle = cycler(color=my_default_color_list)
plt.rcParams['axes.prop_cycle'] = my_default_color_cycle

pairedPalette = sns.color_palette("tab20")
pairedPalette = pairedPalette.as_hex()
pairedPalette

# clist = ['#1f77b4', '#aec7e8', '#ff7f0e', '#ffbb78', '#2ca02c', '#98df8a']
# sns.color_palette(clist)

sns.color_palette(my_default_color_list)

Loading BokehJS ...

[(0.12156862745098039, 0.4666666666666667, 0.7058823529411765),
 (1.0, 0.4980392156862745, 0.054901960784313725),
 (0.17254901960784313, 0.6274509803921569, 0.17254901960784313),
 (0.8392156862745098, 0.15294117647058825, 0.1568627450980392),
 (0.5803921568627451, 0.403921568627451, 0.7411764705882353),
 (0.5490196078431373, 0.33725490196078434, 0.29411764705882354),
 (0.8901960784313725, 0.4666666666666667, 0.7607843137254902),
 (0.4980392156862745, 0.4980392156862745, 0.4980392156862745),
 (0.7372549019607844, 0.7411764705882353, 0.13333333333333333),
 (0.09019607843137255, 0.7450980392156863, 0.8117647058823529)]

In [3]:
N = len(my_default_marker_list)
X = np.arange(1, N+1)
Y = np.arange(1, N+1)
fig, ax = plt.subplots(1, 1, figsize = (3, 3))
for i in range(N):
    for j in range(N):
        ax.plot([X[i]], [Y[-1-j]], color = my_default_color_list[i], marker = my_default_marker_list[j], 
                ls = '', markersize = 10, markeredgecolor = 'k')
ax.set_xticks([])
ax.set_yticks([])
ax.set_xticklabels([])
ax.set_yticklabels([])
fig.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [4]:
# Tests with colors

# sns.color_palette(my_default_color_list)
# #
# cTest = my_default_color_list[3]
# sns.color_palette([cTest])
# #
# cTestRGB = matplotlib.colors.to_rgb(cTest)
# print(cTestRGB)
# sns.color_palette([cTestRGB])
# #
# cTestHSV = matplotlib.colors.rgb_to_hsv(cTestRGB)
# print(cTestHSV)
# cTestHSV[2] = 0.8
# cTestrgb = matplotlib.colors.hsv_to_rgb(cTestHSV)
# sns.color_palette([cTestrgb])
# #
# cTestHSV_2 = copy(cTestHSV)
# cTestHSV_2[1] *= (5/8)
# cTestHSV_2[2] *= 1
# print(cTestHSV_2)

# cTestrgb_2 = matplotlib.colors.hsv_to_rgb(cTestHSV_2)
# sns.color_palette([cTestrgb_2])
# #
# cTestHSV_2[1] *= (5/8)
# cTestHSV_2[2] *= 1
# print(cTestHSV_2)

# cTestrgb_2 = matplotlib.colors.hsv_to_rgb(cTestHSV_2)
# sns.color_palette([cTestrgb_2])
# #
# cTestHSV_2 = copy(cTestHSV)
# cTestHSV_2[1] *= 1
# cTestHSV_2[2] *= (5/8)
# print(cTestHSV_2)

# cTestrgb_2 = matplotlib.colors.hsv_to_rgb(cTestHSV_2)
# sns.color_palette([cTestrgb_2])
# #
# cTestHSV_2[1] *= 1
# cTestHSV_2[2] *= (5/8)
# print(cTestHSV_2)

# cTestrgb_2 = matplotlib.colors.hsv_to_rgb(cTestHSV_2)
# sns.color_palette([cTestrgb_2])

### Reminders

#### Stats

(A) How to compute confidence intervals of fitted parameters with (1-alpha) confidence:

    0) from scipy import stats
    1) df = nb_pts - nb_parms ; se = diag(cov)**0.5
    2) Student t coefficient : q = stat.t.ppf(1 - alpha / 2, df)
    3) ConfInt = [params - q*se, params + q*se]

## TimeSeries functions

In [ ]:
allTimeSeriesDataFiles = [f for f in os.listdir(timeSeriesDataDir) if (os.path.isfile(os.path.join(timeSeriesDataDir, f)) and f.endswith(".csv"))]
allTimeSeriesDataFiles

In [ ]:
df = getCellTimeSeriesData('22-02-09_M1_P1_C7')
df

In [ ]:
plotCellTimeSeriesData('21-01-18_M1-2_P1_C5')

In [ ]:
plotCellTimeSeriesData('22-02-09_M1_P1_C1')

In [ ]:
allTimeSeriesDataFiles = [f for f in os.listdir(timeSeriesDataDir) if (os.path.isfile(os.path.join(timeSeriesDataDir, f)) and f.endswith(".csv"))]
for f in allTimeSeriesDataFiles:
    if '22-02-09_M3' in f:
        plotCellTimeSeriesData(f[:-4])

In [ ]:
allTimeSeriesDataFiles = [f for f in os.listdir(timeSeriesDataDir) if (os.path.isfile(os.path.join(timeSeriesDataDir, f)) and f.endswith(".csv"))]
for f in allTimeSeriesDataFiles:
    if '22-02-09_M2' in f:
        plotCellTimeSeriesData(f[:-4])

In [ ]:
plt.close('all')

In [ ]:
%autoreload 2

listeTraj = getCellTrajData('21-12-16_M1_P1_C10', Ntraj = 2)
listeTraj[1]['pos']

def testTraj(D0):
    trajDir = os.path.join(timeSeriesDataDir, 'Trajectories')
    allTimeSeriesDataFiles = [f for f in os.listdir(timeSeriesDataDir) 
                              if (os.path.isfile(os.path.join(timeSeriesDataDir, f)) 
                                  and f.endswith(".csv"))]
    cellIDList = []
    for f in allTimeSeriesDataFiles:
        cellID = findInfosInFileName(f, 'cellID')
        if '21-12-08' in cellID:
            cellIDList.append(cellID)

    print(cellIDList)

    fig, ax = plt.subplots(1,2, figsize = (8,4))
    width = 15
    ax[0].set_title('Inside bead')
    ax[0].axis([-width,width,-width,width])
    ax[1].set_title('Outside bead')
    ax[1].axis([-width,width,-width,width])

    for C in (cellIDList):
        listeTraj = getCellTrajData(C)
        iOut = (listeTraj[1]['pos'] == 'out')
        iIn = 1 - iOut
        dfIn, dfOut = listeTraj[iIn]['df'], listeTraj[iOut]['df']
        Xin = dfIn.X.values - dfIn.X.values[0]
        Yin = dfIn.Y.values - dfIn.Y.values[0]
        Xout = dfOut.X.values - dfOut.X.values[0]
        Yout = dfOut.Y.values - dfOut.Y.values[0]
        npts = len(Xin)
        for iii in range(1, npts):
            D2in = ((Xin[iii]-Xin[iii-1])**2 + (Yin[iii]-Yin[iii-1])**2) ** 0.5
            D2out = ((Xout[iii]-Xout[iii-1])**2 + (Yout[iii]-Yout[iii-1])**2) ** 0.5
            if D2in > D0 and D2out > D0:
                dxCorrIn = Xin[iii-1]-Xin[iii]
                dyCorrIn = Yin[iii-1]-Yin[iii]
                dxCorrOut = Xout[iii-1]-Xout[iii]
                dyCorrOut = Yout[iii-1]-Yout[iii]
                Xin[iii:] += dxCorrIn
                Yin[iii:] += dyCorrIn
                Xout[iii:] += dxCorrOut
                Yout[iii:] += dyCorrOut
        if np.max(np.abs(Xin)) < width and np.max(np.abs(Yin)) < width and \
        np.max(np.abs(Xout)) < width and np.max(np.abs(Yout)) < width:
            ax[0].plot(Xin, Yin)
            ax[1].plot(Xout, Yout)

    fig.show()
    
testTraj(1)
    
testTraj(1.5)

## GlobalTables functions

### Experimental conditions

In [ ]:
%autoreload 2
getExperimentalConditions(experimentalDataDir, save=True , sep = ';')

### Constant Field

In [ ]:
computeGlobalTable_ctField(task='updateExisting', save=False)

In [ ]:
getGlobalTable_ctField().head()

### Mechanics

In [ ]:
%autoreload 2
# computeGlobalTable_meca(task = 'updateExisting', fileName = 'Global_MecaData_Py', save = False, PLOT = False, source = 'Matlab') # task = 'updateExisting'

In [ ]:
%autoreload 2
# computeGlobalTable_meca(task = 'fromScratch', fileName = 'Global_MecaData_Py2', save = True, PLOT = False, source = 'Python') # task = 'updateExisting'

In [5]:
%autoreload 2
computeGlobalTable_meca(task = '22-02-09_M1_P1_C1', fileName = 'Global_MecaData_Py2', save = False, PLOT = True, source = 'Python') # task = 'updateExisting'

Importing Experimental Conditions
Extracted a table with 67 lines and 24 columns
ramp field : converted to list successfully
22.803509950637817


,date,cellName,cellID,manipID,compNum,compDuration,compStartTime,compAbsStartTime,compStartTimeThisDay,initialThickness,minThickness,maxIndent,previousThickness,surroundingThickness,surroundingDx,surroundingDz,validatedThickness,jumpD3,minForce,maxForce,minStress,maxStress,minStrain,maxStrain,ctFieldThickness,ctFieldFluctuAmpli,ctFieldDX,ctFieldDZ,bestH0,H0Chadwick,EChadwick,R2Chadwick,EChadwick_CIWidth,hysteresis,critFit,validatedFit,comments,KChadwick_50<s<200,R2Chadwick_50<s<200,K2Chadwick_50<s<200,H0Chadwick_50<s<200,Npts_50<s<200,validatedFit_50<s<200,KChadwick_200<s<350,R2Chadwick_200<s<350,K2Chadwick_200<s<350,H0Chadwick_200<s<350,Npts_200<s<350,validatedFit_200<s<350,KChadwick_350<s<500,R2Chadwick_350<s<500,K2Chadwick_350<s<500,H0Chadwick_350<s<500,Npts_350<s<500,validatedFit_350<s<500,KChadwick_500<s<700,R2Chadwick_500<s<700,K2Chadwick_500<s<700,H0Chadwick_500<s<700,Npts_500<s<700,validatedFit_500<s<700,KChadwick_700<s<1000,R2Chadwick_700<s<1000,K2Chadwick_700<s<1000,H0Chadwick_700<s<1000,Npts_700<s<1000,validatedFit_700<s<1000,KChadwick_1000<s<1500,R2Chadwick_1000<s<1500,K2Chadwick_1000<s<1500,H0Chadwick_1000<s<1500,Npts_1000<s<1500,validatedFit_1000<s<1500,KChadwick_1500<s<2000,R2Chadwick_1500<s<2000,K2Chadwick_1500<s<2000,H0Chadwick_1500<s<2000,Npts_1500<s<2000,validatedFit_1500<s<2000
0,22-02-09,M1_P1_C10,22-02-09_M1_P1_C10,22-02-09_M1,1,1.5s,6.779,19287.044,19287.044,339.491162,133.688258,205.802904,344.834189,292.930797,272.569620,-340.375,True,-27.761477,16.073546,1247.924470,62.910277,722.661753,0.031849,0.215457,231.133384,66.369403,191.841772,-499.625,378.047374,356.120326,3498.891620,0.980983,346.980496,123.118532,R2 > 0.6,True,ok,2218.927869,0.953283,2179.610309,379.091803,20,True,2394.152672,0.980176,2353.745240,375.129166,18,True,5.829267e+03,8.947118e-01,5.331556e+03,307.630736,16,True,6.383227e+03,9.761381e-01,5.756581e+03,304.117584,24,True,3.066696e+03,5.311760e-01,2.543650e+03,434.057634,6,False,NaN,NaN,NaN,NaN,0,False,NaN,NaN,NaN,NaN,0,False
1,22-02-09,M1_P1_C10,22-02-09_M1_P1_C10,22-02-09_M1,2,1.5s,27.475,19307.740,19307.740,355.330732,122.961183,232.369549,344.310657,319.900884,295.765823,-282.625,True,-0.717884,15.850596,1213.894061,45.806418,612.470727,0.039213,0.233462,231.133384,66.369403,191.841772,-499.625,410.396996,359.153668,2935.272984,0.963197,400.816374,147.558150,R2 > 0.6,True,ok,1444.728798,0.972257,1397.224486,400.231120,26,True,3216.355544,0.960783,2958.197616,338.165676,19,True,5.852903e+03,9.556170e-01,5.310430e+03,292.146043,20,True,1.118654e+04,5.492566e-01,8.956204e+03,245.956539,17,False,NaN,NaN,NaN,NaN,0,False,NaN,NaN,NaN,NaN,0,False,NaN,NaN,NaN,NaN,0,False
2,22-02-09,M1_P1_C10,22-02-09_M1_P1_C10,22-02-09_M1,3,1.5s,48.488,19328.753,19328.753,382.176884,127.885479,254.291405,375.043394,326.996190,278.645570,-586.250,True,18.965489,17.370480,1203.120771,39.709811,536.538268,0.046334,0.237519,231.133384,66.369403,191.841772,-499.625,444.905506,387.560711,3001.547538,0.977276,327.177563,137.489319,R2 > 0.6,True,ok,1694.356189,0.899592,1568.973922,419.091325,26,True,4566.008306,0.908535,4342.195421,348.731739,21,True,4.400728e+03,8.520110e-01,3.781372e+03,364.262952,21,True,1.182091e+03,7.276286e-01,1.472658e+03,569.055144,12,True,NaN,NaN,NaN,NaN,0,False,NaN,NaN,NaN,NaN,0,False,NaN,NaN,NaN,NaN,0,False
3,22-02-09,M1_P1_C10,22-02-09_M1_P1_C10,22-02-09_M1,4,1.5s,68.286,19348.551,19348.551,315.546602,151.349031,164.197571,306.844756,286.518246,222.664557,-645.750,True,26.256326,20.789261,1133.007518,64.495762,779.657979,0.041730,0.194740,231.133384,66.369403,191.841772,-499.625,364.011075,318.640100,4892.512794,0.959167,747.881736,93.129025,R2 > 0.6,False,R2 < 0.6,1754.944355,0.975650,1719.660832,360.747536,21,True,4754.634889,0.773348,4313.400681,311.041838,15,True,6.196793e+03,7.799035e-01,5.445561e+03,305.533492,15,True,7.201085e+03,8.345359e-01,6.251306e+03,297.925105,22,True,7.274988e+10,1.231928e-14,5.699051e+04,196.705151,9,False,NaN,NaN,NaN,NaN,0,False,NaN,NaN,NaN,NaN,0,Fals

In [ ]:
%autoreload 2
# computeGlobalTable_meca(task = '22-02-09', fileName = 'Global_MecaData_Py2', save = False, PLOT = True, source = 'Python') # task = 'updateExisting'

In [ ]:
%autoreload 2
# computeGlobalTable_meca(task = '21-01-18', fileName = 'Global_MecaData_Py2', save = False, PLOT = True, source = 'Python') # task = 'updateExisting'
# computeGlobalTable_meca(task = '21-01-21', fileName = 'Global_MecaData_Py2', save = False, PLOT = True, source = 'Python') # task = 'updateExisting'

In [ ]:
getGlobalTable_meca().tail()

In [ ]:
getGlobalTable_meca('Global_MecaData_Py2').tail()

### Fluorescence

In [ ]:
getFluoData().head()

## Data import & DataFrame formatting

### Data import

In [ ]:
pd.set_option('max_columns', None)
# pd.reset_option('max_columns')
pd.set_option('max_rows', None)
pd.reset_option('max_rows')

In [ ]:
# getExperimentalConditions().head()
# getGlobalTable_ctField().head()
# getGlobalTable_meca().head()
# getFluoData().head()

In [ ]:
# GlobalTable_ctField
%autoreload 2
GlobalTable_ctField = getGlobalTable(kind = 'ctField')
GlobalTable_ctField.head()

In [ ]:
# GlobalTable_meca
%autoreload 2
GlobalTable_meca = getGlobalTable(kind = 'meca_matlab')
GlobalTable_meca.tail()

In [ ]:
# GlobalTable_mecaBis
%autoreload 2
GlobalTable_meca_Py = getGlobalTable(kind = 'meca_py')
GlobalTable_meca_Py.tail()

In [17]:
# GlobalTable_mecaPy
%autoreload 2
GlobalTable_meca_Py2 = getGlobalTable(kind = 'meca_py2')
GlobalTable_meca_Py2.tail()

Extracted a table with 2275 lines and 132 columns.
Importing Experimental Conditions
Extracted a table with 67 lines and 24 columns
ramp field : converted to list successfully
Extracted a table with 209 lines and 2 columns.
Merged table has 2275 lines and 155 columns.


,date,cellName,cellID,manipID,compNum,compDuration,compStartTime,compAbsStartTime,compStartTimeThisDay,initialThickness,minThickness,maxIndent,previousThickness,surroundingThickness,surroundingDx,surroundingDz,validatedThickness,jumpD3,minForce,maxForce,minStress,maxStress,minStrain,maxStrain,ctFieldThickness,ctFieldFluctuAmpli,ctFieldDX,ctFieldDZ,H0Chadwick,EChadwick,R2Chadwick,EChadwick_CIWidth,hysteresis,critFit,validatedFit,comments,H0Chadwick_s<100Pa,EChadwick_s<100Pa,R2Chadwick_s<100Pa,Npts_s<100Pa,validatedFit_s<100Pa,H0Chadwick_s<100Pa_included,EChadwick_s<100Pa_included,R2Chadwick_s<100Pa_included,Npts_s<100Pa_included,validatedFit_s<100Pa_included,H0Chadwick_s<200Pa,EChadwick_s<200Pa,R2Chadwick_s<200Pa,Npts_s<200Pa,validatedFit_s<200Pa,H0Chadwick_s<200Pa_included,EChadwick_s<200Pa_included,R2Chadwick_s<200Pa_included,Npts_s<200Pa_included,validatedFit_s<200Pa_included,H0Chadwick_s<300Pa,EChadwick_s<300Pa,R2Chadwick_s<300Pa,Npts_s<300Pa,validatedFit_s<300Pa,H0Chadwick_s<300Pa_included,EChadwick_s<300Pa_included,R2Chadwick_s<300Pa_included,Npts_s<300Pa_included,validatedFit_s<300Pa_included,H0Chadwick_100<s<200Pa,EChadwick_100<s<200Pa,R2Chadwick_100<s<200Pa,Npts_100<s<200Pa,validatedFit_100<s<200Pa,H0Chadwick_200<s<300Pa,EChadwick_200<s<300Pa,R2Chadwick_200<s<300Pa,Npts_200<s<300Pa,validatedFit_200<s<300Pa,H0Chadwick_50<s<250Pa_150included,EChadwick_50<s<250Pa_150included,R2Chadwick_50<s<250Pa_150included,Npts_50<s<250Pa_150included,validatedFit_50<s<250Pa_150included,H0Chadwick_100<s<300Pa_200included,EChadwick_100<s<300Pa_200included,R2Chadwick_100<s<300Pa_200included,Npts_100<s<300Pa_200included,validatedFit_100<s<300Pa_200included,H0Chadwick_s<500Pa,EChadwick_s<500Pa,R2Chadwick_s<500Pa,Npts_s<500Pa,validatedFit_s<500Pa,H0Chadwick_200<s<500Pa_350included,EChadwick_200<s<500Pa_350included,R2Chadwick_200<s<500Pa_350included,Npts_200<s<500Pa_350included,validatedFit_200<s<500Pa_350included,H0Chadwick_s<500Pa_included,EChadwick_s<500Pa_included,R2Chadwick_s<500Pa_included,Npts_s<500Pa_included,validatedFit_s<500Pa_included,H0Chadwick_250<s<750Pa,EChadwick_250<s<750Pa,R2Chadwick_250<s<750Pa,Npts_250<s<750Pa,validatedFit_250<s<750Pa,H0Chadwick_500<s<1000Pa,EChadwick_500<s<1000Pa,R2Chadwick_500<s<1000Pa,Npts_500<s<1000Pa,validatedFit_500<s<1000Pa,H0Chadwick_s<400Pa,EChadwick_s<400Pa,R2Chadwick_s<400Pa,Npts_s<400Pa,validatedFit_s<400Pa,H0Chadwick_s<400Pa_included,EChadwick_s<400Pa_included,R2Chadwick_s<400Pa_included,Npts_s<400Pa_included,validatedFit_s<400Pa_included,H0Chadwick_300<s<700Pa,EChadwick_300<s<700Pa,R2Chadwick_300<s<700Pa,Npts_300<s<700Pa,validatedFit_300<s<700Pa,H0Chadwick_600<s<1000Pa,EChadwick_600<s<1000Pa,R2Chadwick_600<s<1000Pa,Npts_600<s<1000Pa,validatedFit_600<s<1000Pa,manip,experimentType,drug,substrate,objective magnification,scale pixel per um,objective immersion,optical index correction,magnetic field correction,cell type,cell subtype,bead type,bead diameter,beads bright spot delta,normal field,ramp field,compression duration,with fluo images,normal field multi images,multi image Z step,loop structure,meanFluoPeakAmplitude
2270,22-02-09,M3_P1_C9,22-02-09_M3_P1_C9,22-02-09_M3,2,1.5s,29.564,32220.712,15397.706,437.030050,204.461819,232.568231,424.699263,378.401718,182.727848,-1080.625,True,3.574787,18.341320,1079.125683,-544.102298,3167.445634,-0.003633,0.177480,355.170689,131.624162,201.620253,-872.375,437.296068,2813.793603,0.906468,805.587378,131.765614,R2 > 0.6,False,R2 < 0.6,NaN,NaN,NaN,1.0,False,NaN,NaN,NaN,1.0,False,488.765730,1100.368996,0.977888,17.0,True,488.765730,1100.368996,0.977888,17.0,True,483.558761,1251.881037,0.972435,29.0,True,483.558761,1251.881037,0.972435,29.0,True,477.378554,1249.940516,0.979676,16.0,True,494.888303,1206.800337,0.983213,12.0,True,480.444369,1288.271744,0.970812,28.0,True,488.272244,1141.616356,0.978830,24.0,True,452.737544,2034.287776,0.939765,44.0,True,459.631479,1982.963147,0.930464,27.0,True,452.737544,2034.287776,0.939765,44.0,True,428.165096,3149.725158,0.8

### Experiment counter

In [ ]:
# Experiment counter - Matlab table

# count ct field cells
cellID = 'cellID'
GlobalTable_ctField_CountCell = GlobalTable_ctField.groupby(['cell type', 'cell subtype', 'bead type', 'drug', 'substrate']).count()
GlobalTable_ctField_CountCell = GlobalTable_ctField_CountCell.loc[:, [cellID]].rename(columns={cellID : 'Count cells - ctField'})

# count meca compressions
cellID = 'CellName'
GlobalTable_meca_CountComp = GlobalTable_meca.groupby(['cell type', 'cell subtype', 'bead type', 'drug', 'substrate']).count()
GlobalTable_meca_CountComp = GlobalTable_meca_CountComp.loc[:, [cellID]].rename(columns={cellID : 'Count compressions'})

# count valid meca compressions
cellID = 'CellName'
GlobalTable_meca_CountCompOK = GlobalTable_meca[GlobalTable_meca['Validated'] == True].groupby(['cell type', 'cell subtype', 'bead type', 'drug', 'substrate']).count()
GlobalTable_meca_CountCompOK = GlobalTable_meca_CountCompOK.loc[:, [cellID]].rename(columns={cellID : 'Count OK compressions'})

# count meca cells
cellID = 'CellName'
group = GlobalTable_meca.groupby(cellID)
dictAggMean = getDictAggMean(GlobalTable_meca)
GlobalTable_meca_perCell = group.agg(dictAggMean)
GlobalTable_meca_CountCell = GlobalTable_meca_perCell.groupby(['cell type', 'cell subtype', 'bead type', 'drug', 'substrate']).count()
GlobalTable_meca_CountCell = GlobalTable_meca_CountCell.loc[:, [cellID]].rename(columns={cellID : 'Count cells - meca'})


# Fuse all the previous tables
GlobalTable_CountAll = pd.concat([GlobalTable_ctField_CountCell, GlobalTable_meca_CountCell, GlobalTable_meca_CountComp, GlobalTable_meca_CountCompOK], axis=1)
GlobalTable_CountAll = GlobalTable_CountAll.fillna(0)
GlobalTable_CountAll = GlobalTable_CountAll.loc[:,:].astype(int)
GlobalTable_CountAll

In [ ]:
# Experiment counter - Python table

# count ct field cells
cellID = 'cellID'
GlobalTable_ctField_CountCell = GlobalTable_ctField.groupby(['cell type', 'cell subtype', 'bead type', 'drug', 'substrate']).count()
GlobalTable_ctField_CountCell = GlobalTable_ctField_CountCell.loc[:, [cellID]].rename(columns={cellID : 'Count cells - ctField'})

# count meca compressions
cellID = 'cellID'
GlobalTable_meca_CountComp = GlobalTable_meca_Py.groupby(['cell type', 'cell subtype', 'bead type', 'drug', 'substrate']).count()
GlobalTable_meca_CountComp = GlobalTable_meca_CountComp.loc[:, [cellID]].rename(columns={cellID : 'Count compressions'})

# count valid meca compressions
cellID = 'cellID'
GlobalTable_meca_CountCompOK = GlobalTable_meca_Py[GlobalTable_meca_Py['validatedFit'] == True].groupby(['cell type', 'cell subtype', 'bead type', 'drug', 'substrate']).count()
GlobalTable_meca_CountCompOK = GlobalTable_meca_CountCompOK.loc[:, [cellID]].rename(columns={cellID : 'Count OK compressions'})

# count meca cells
cellID = 'cellID'
group = GlobalTable_meca_Py.groupby(cellID)
dictAggMean = getDictAggMean(GlobalTable_meca_Py)
GlobalTable_meca_perCell = group.agg(dictAggMean)
GlobalTable_meca_CountCell = GlobalTable_meca_perCell.groupby(['cell type', 'cell subtype', 'bead type', 'drug', 'substrate']).count()
GlobalTable_meca_CountCell = GlobalTable_meca_CountCell.loc[:, [cellID]].rename(columns={cellID : 'Count cells - meca'})


# Fuse all the previous tables
GlobalTable_CountAll = pd.concat([GlobalTable_ctField_CountCell, GlobalTable_meca_CountCell, GlobalTable_meca_CountComp, GlobalTable_meca_CountCompOK], axis=1)
GlobalTable_CountAll = GlobalTable_CountAll.fillna(0)
GlobalTable_CountAll = GlobalTable_CountAll.loc[:,:].astype(int)
GlobalTable_CountAll

In [ ]:
# Experiment counter - Python table 2

# count ct field cells
cellID = 'cellID'
GlobalTable_ctField_CountCell = GlobalTable_ctField.groupby(['cell type', 'cell subtype', 'bead type', 'drug', 'substrate']).count()
GlobalTable_ctField_CountCell = GlobalTable_ctField_CountCell.loc[:, [cellID]].rename(columns={cellID : 'Count cells - ctField'})

# count meca compressions
cellID = 'cellID'
GlobalTable_meca_CountComp = GlobalTable_meca_Py2.groupby(['cell type', 'cell subtype', 'bead type', 'drug', 'substrate']).count()
GlobalTable_meca_CountComp = GlobalTable_meca_CountComp.loc[:, [cellID]].rename(columns={cellID : 'Count compressions'})

# count valid meca compressions
cellID = 'cellID'
GlobalTable_meca_CountCompOK = GlobalTable_meca_Py2[GlobalTable_meca_Py2['validatedFit'] == True].groupby(['cell type', 'cell subtype', 'bead type', 'drug', 'substrate']).count()
GlobalTable_meca_CountCompOK = GlobalTable_meca_CountCompOK.loc[:, [cellID]].rename(columns={cellID : 'Count OK compressions'})

# count meca cells
cellID = 'cellID'
group = GlobalTable_meca_Py2.groupby(cellID)
dictAggMean = getDictAggMean(GlobalTable_meca_Py2)
GlobalTable_meca_perCell = group.agg(dictAggMean)
GlobalTable_meca_CountCell = GlobalTable_meca_perCell.groupby(['cell type', 'cell subtype', 'bead type', 'drug', 'substrate']).count()
GlobalTable_meca_CountCell = GlobalTable_meca_CountCell.loc[:, [cellID]].rename(columns={cellID : 'Count cells - meca'})


# Fuse all the previous tables
GlobalTable_CountAll = pd.concat([GlobalTable_ctField_CountCell, GlobalTable_meca_CountCell, GlobalTable_meca_CountComp, GlobalTable_meca_CountCompOK], axis=1)
GlobalTable_CountAll = GlobalTable_CountAll.fillna(0)
GlobalTable_CountAll = GlobalTable_CountAll.loc[:,:].astype(int)
GlobalTable_CountAll

In [ ]:
# Filters = [(GlobalTable_meca_Py['validatedFit'] == True), (GlobalTable_meca_Py['validatedThickness'] == True), (GlobalTable_meca_Py['cell subtype'] == 'aSFL-A8')]
# GlobalTable_meca_PyF = GlobalTable_meca_Py
# for fltr in Filters:
#         GlobalTable_meca_PyF = GlobalTable_meca_PyF.loc[fltr]
# # GlobalTable_mecaBis
# cellID = 'cellID'
# group = GlobalTable_meca_PyF.groupby(cellID)
# dictAggMean = getDictAggMean(GlobalTable_meca_PyF)
# GlobalTable_meca_PyF_perCell = group.agg(dictAggMean)
# GlobalTable_meca_PyF_perCell

### Custom data export

In [ ]:
# 21-06-28 Export of E - h data for Julien
# GlobalTable_meca_Py_expJ = GlobalTable_meca_Py.loc[GlobalTable_meca_Py['validatedFit'] & GlobalTable_meca_Py['validatedThickness']]\
#                                                [['cell type', 'cell subtype', 'bead type', 'drug', 'substrate', 'compNum', \
#                                                  'EChadwick', 'H0Chadwick', 'surroundingThickness', 'ctFieldThickness']]
# GlobalTable_meca_Py_expJ = GlobalTable_meca_Py_expJ.reset_index()
# GlobalTable_meca_Py_expJ = GlobalTable_meca_Py_expJ.drop('index', axis=1)
# savePath = os.path.join(dataDir, 'mecanicsData_3T3.csv')
# GlobalTable_meca_Py_expJ.to_csv(savePath, sep=';')

## Plots

### Functions

In [6]:
Styles = {''} # Project of automatic formatting according to the type of data

renameDict1 = {'SurroundingThickness':'Thickness (nm) [b&a]',\
               'surroundingThickness':'Thickness (nm) [b&a]',\
               'ctFieldThickness':'Thickness at low force (nm)',\
               'EChadwick': 'E Chadwick (Pa)',\
               'medianThickness': 'Median Thickness (nm)',\
               'fluctuAmpli': 'Fluctuations Amplitude (nm)',\
               'meanFluoPeakAmplitude' : 'Fluo Intensity (a.u.)',\
               'none':'control',\
               'doxycyclin':'expressing iMC linker',\
               'none & BSA coated glass':'control & non adherent',\
               'doxycyclin & BSA coated glass':'iMC & non adherent',\
               'none & 20um fibronectin discs':'control & adherent on fibro',\
               'doxycyclin & 20um fibronectin discs':'iMC & adherent on fibro',\
               'BSA coated glass & none':'control & non adherent',\
               'BSA coated glass & doxycyclin':'iMC & non adherent',\
               '20um fibronectin discs & none':'control & adherent on fibro',\
               '20um fibronectin discs & doxycyclin':'iMC & adherent on fibro',\
               'aSFL & none':'aSFL control',\
               'aSFL & doxycyclin':'aSFL iMC',\
               'aSFL-6FP & none':'aSFL-6FP control',\
               'aSFL-6FP & doxycyclin':'aSFL-6FP long-iMC',\
               'aSFL-6FP-2 & none':'aSFL-6FP-2 control',\
               'aSFL-6FP-2 & doxycyclin':'aSFL-6FP-2 long-iMC',\
               'aSFL-A8 & none':'aSFL-A8 control',\
               'aSFL-A8 & doxycyclin':'aSFL-A8 iMC',\
               'aSFL-A8-2 & none':'aSFL-A8-2 control',\
               'aSFL-A8-2 & doxycyclin':'aSFL-A8-2 iMC',\
               'dmso' : 'DMSO, no linker', 
               'smifh2' : 'SMIFH2, no linker', 
               'dmso, doxycyclin' : 'DMSO, iMC linker', 
               'smifh2, doxycyclin' : 'SMIFH2, iMC linker'}

styleDict1 =  {'none & BSA coated glass':{'color':'#ff9896','marker':'^'},\
               'doxycyclin & BSA coated glass':{'color':'#d62728','marker':'^'},\
               'none & 20um fibronectin discs':{'color':'#aec7e8','marker':'o'},\
               'doxycyclin & 20um fibronectin discs':{'color':'#1f77b4','marker':'o'},\
               'none':{'color':'#aec7e8','marker':'o'},\
               'doxycyclin':{'color':'#1f77b4','marker':'o'},\
               'BSA coated glass & none':{'color':'#ff9896','marker':'^'},\
               'BSA coated glass & doxycyclin':{'color':'#d62728','marker':'^'},\
               '20um fibronectin discs & none':{'color':'#aec7e8','marker':'o'},\
               '20um fibronectin discs & doxycyclin':{'color':'#1f77b4','marker':'o'},\
               'aSFL':{'color':'#1f77b4','marker':'o'},\
               'aSFL-6FP':{'color':'#2ca02c','marker':'o'},\
               'aSFL-A8':{'color':'#ff7f0e','marker':'o'}, \
               'aSFL & none':{'color':'#aec7e8','marker':'o'},\
               'aSFL & doxycyclin':{'color':'#1f77b4','marker':'o'},\
               'aSFL-6FP & none':{'color':'#98df8a','marker':'o'},\
               'aSFL-6FP & doxycyclin':{'color':'#2ca02c','marker':'o'},\
               'aSFL-A8 & none':{'color':'#ffbb78','marker':'o'},\
               'aSFL-A8 & doxycyclin':{'color':'#ff7f0e','marker':'o'}}


These functions use matplotlib.pyplot and seaborn libraries to display 1D categorical or 2D plots

In [7]:
def D1Plot(data, CondCol=[], Parameters=[], Filters=[], Boxplot=True, AvgPerCell=False, cellID='cellID', co_order=[],\
           stats=True, statMethod='Mann-Whitney', box_pairs=[], figSizeFactor = 1, markerSizeFactor=1, orientation = 'h',
           useHue = False):
    data_filtered = data
    for fltr in Filters:
        data_filtered = data_filtered.loc[fltr]    
    NCond = len(CondCol)
    
    if NCond == 1:
        CondCol = CondCol[0]
        
    elif NCond > 1:
        newColName = ''
        for i in range(NCond):
            newColName += CondCol[i]
            newColName += ' & '
        newColName = newColName[:-3]
        data_filtered[newColName] = ''
        for i in range(NCond):
            data_filtered[newColName] += data_filtered[CondCol[i]].astype(str)
            data_filtered[newColName] = data_filtered[newColName].apply(lambda x : x + ' & ')
        data_filtered[newColName] = data_filtered[newColName].apply(lambda x : x[:-3])
        CondCol = newColName
    
    if AvgPerCell:
        group = data_filtered.groupby(cellID)
        dictAggMean = getDictAggMean(data_filtered)
#         dictAggMean['EChadwick'] = 'median'
        data_filtered = group.agg(dictAggMean)
        
    data_filtered.sort_values(CondCol, axis=0, ascending=True, inplace=True)
    
    if len(co_order) > 0:
        p = getStyleLists(co_order, styleDict1)
    else:
        p = sns.color_palette()
    
    NPlots = len(Parameters)
    Conditions = list(data_filtered[CondCol].unique())
    if len(co_order) == 0:
        co_order = Conditions
    
    if orientation == 'h':
        fig, ax = plt.subplots(1, NPlots, figsize = (5*NPlots*NCond*figSizeFactor, 5))
    elif orientation == 'v':
        fig, ax = plt.subplots(NPlots, 1, figsize = (5*NCond*figSizeFactor, 5*NPlots))
    
    markerSize = 5*markerSizeFactor
    
    if NPlots == 1:
        ax = np.array([ax])
    
    for k in range(NPlots):

        if 'EChadwick' in Parameters[k]:
            ax[k].set_yscale('log')


        if Boxplot:
            sns.boxplot(x=CondCol, y=Parameters[k], data=data_filtered, ax=ax[k], 
                        color='w', linewidth = 2, width = 0.5, showfliers = False, order= co_order)
            # data_filtered.boxplot(column=Parameters[k], by = CondCol, ax=ax[k],showfliers = False) # linewidth = 2, width = 0.5

        if stats:
            if len(box_pairs) == 0:
                box_pairs = makeBoxPairs(co_order)
            addStat_df(ax[k], data_filtered, box_pairs, Parameters[k], CondCol, test = statMethod)
            # add_stat_annotation(ax[k], x=CondCol, y=Parameters[k], data=data_filtered,box_pairs = box_pairs,test=statMethod, text_format='star',loc='inside', verbose=2)
        
        if not useHue:
            sns.swarmplot(x=CondCol, y=Parameters[k], data=data_filtered, ax=ax[k], order = co_order,
                          size=markerSize, edgecolor='k', linewidth = 1*markerSizeFactor, palette = p)
        else:
            sns.swarmplot(x=CondCol, y=Parameters[k], data=data_filtered, ax=ax[k], order = co_order,
                          size=markerSize, edgecolor='k', linewidth = 1*markerSizeFactor, palette = p, hue = 'manipID')
            legend = ax[k].legend()
            legend.remove()

        ax[k].set_xlabel('')
        ax[k].set_ylabel(Parameters[k])
        ax[k].tick_params(axis='x', labelrotation = 10)
        ax[k].yaxis.grid(True)           
            
#     else:
#         if Parameters[0] == 'EChadwick':
#             ax.set_yscale('log')
                
#         if Boxplot:
#             sns.boxplot(x=CondCol, y=Parameters[0], data=data_filtered, ax=ax, 
#                         color='w', linewidth = 2, width = 0.5, showfliers = False, order= co_order)
#             # data_filtered.boxplot(column=Parameters[k], by = CondCol, ax=ax[k],showfliers = False) # linewidth = 2, width = 0.5

#         if stats:
#             if len(box_pairs) == 0:
#                 box_pairs = makeBoxPairs(co_order)
#             addStat(ax, data_filtered, box_pairs, Parameters[0], CondCol, test = statMethod)
#             # add_stat_annotation(ax, x=CondCol, y=Parameters[0], data=data_filtered,box_pairs = box_pairs,test=statMethod, text_format='star',loc='inside', verbose=2)

#         sns.swarmplot(x=CondCol, y=Parameters[0], data=data_filtered, ax=ax, order= co_order, 
#                       size=markerSize, edgecolor='k',linewidth = 1*markerSizeFactor, palette = p)
    
#         ax.set_xlabel('')
#         ax.set_ylabel(Parameters[0])
#         ax.tick_params(axis='x', labelrotation = 10)
#         ax.yaxis.grid(True)
    
    plt.rcParams['axes.prop_cycle'] = my_default_color_cycle
    return(fig, ax)

In [8]:
def D1PlotDetailed(data, CondCol=[], Parameters=[], Filters=[], Boxplot=True, cellID='cellID', 
                   co_order=[], stats=True, statMethod='Mann-Whitney', box_pairs=[],
                   figSizeFactor = 1, markersizeFactor=1, orientation = 'h', showManips = True):
    
    data_f = data
    for fltr in Filters:
        data_f = data_f.loc[fltr]    
    NCond = len(CondCol)
    
    if NCond == 1:
        CondCol = CondCol[0]
        
    elif NCond > 1:
        newColName = ''
        for i in range(NCond):
            newColName += CondCol[i]
            newColName += ' & '
        newColName = newColName[:-3]
        data_f[newColName] = ''
        for i in range(NCond):
            data_f[newColName] += data_f[CondCol[i]].astype(str)
            data_f[newColName] = data_f[newColName].apply(lambda x : x + ' & ')
        data_f[newColName] = data_f[newColName].apply(lambda x : x[:-3])
        CondCol = newColName
        
    data_f_agg = getAggDf(data_f, cellID, CondCol, Parameters)
    
    NPlots = len(Parameters)
    Conditions = list(data_f[CondCol].unique())
        
    if orientation == 'h':
        fig, ax = plt.subplots(1, NPlots, figsize = (5*NPlots*NCond*figSizeFactor, 5))
    elif orientation == 'v':
        fig, ax = plt.subplots(NPlots, 1, figsize = (5*NCond*figSizeFactor, 5*NPlots))
    
    markersize = 5*markersizeFactor
    
    if NPlots == 1:
        ax = np.array([ax])
        
    if len(co_order) == 0:
        co_order = Conditions
        
        
    for k in range(NPlots):
        
        Parm = Parameters[k]
        if 'EChadwick' in Parm:
            ax[k].set_yscale('log')

        for i in range(len(Conditions)):
            c = Conditions[i]
            sub_data_f_agg = data_f_agg.loc[data_f_agg[CondCol] == c]
            Ncells = sub_data_f_agg.shape[0]
            
            color = my_default_color_list[i]
            
            if showManips:
                allManipID = list(sub_data_f_agg['manipID'].unique())
                alreadyLabeledManip = []
                dictMarker = {}
                for mi in range(len(allManipID)):
                    dictMarker[allManipID[mi]] = my_default_marker_list[mi]

                
            midPos = i
            startPos = i-0.4
            stopPos = i+0.4
            
            values = sub_data_f_agg[Parm + '_mean'].values
            errors = sub_data_f_agg[Parm + '_std'].values
            
            if Boxplot:
                ax[k].boxplot(values, positions=[midPos], widths=0.4, patch_artist=True,
                            showmeans=False, showfliers=False,
                            medianprops={"color": 'k', "linewidth": 1.5, 'alpha' : 0.5},
                            boxprops={"facecolor": color, "edgecolor": 'k',
                                      "linewidth": 1, 'alpha' : 0.5},
    #                         boxprops={"color": color, "linewidth": 0.5},
                            whiskerprops={"color": 'k', "linewidth": 1.5, 'alpha' : 0.5},
                            capprops={"color": 'k', "linewidth": 1.5, 'alpha' : 0.5},
                             zorder = 1)
                

            step = 0.8/(Ncells-1)
            
            for m in range(Ncells):
                
                thisCellID = sub_data_f_agg.index.values[m]
                thisCell_data = data_f.loc[data_f[cellID] == thisCellID]
                thisCell_allValues = thisCell_data[Parm].values
                nval = len(thisCell_allValues)
                
                if showManips:
                    thisManipID = sub_data_f_agg['manipID'].values[m]
                    marker = dictMarker[thisManipID]
                else:
                    marker = 'o'
                
                cellpos = startPos + m*step
                ax[k].errorbar([cellpos], [values[m]], color = color, marker = marker, markeredgecolor = 'k', 
                               yerr=errors[m], capsize = 2, ecolor = 'k', elinewidth=0.8, barsabove = False)
                
                if showManips and thisManipID not in alreadyLabeledManip:
                    alreadyLabeledManip.append(thisManipID)
                    textLabel = thisManipID
                    ax[k].plot([cellpos], [values[m]], 
                               color = color, 
                               marker = marker, markeredgecolor = 'k', markersize = markersize,
                               label = textLabel)
                    
                ax[k].plot(cellpos*np.ones(nval), thisCell_allValues, 
                           color = color, marker = '_', ls = '', markersize = markersize)
                
        

        ax[k].set_xlabel('')
        ax[k].set_xticklabels(labels = Conditions)
        ax[k].tick_params(axis='x', labelrotation = 10)
        
        ax[k].set_ylabel(Parameters[k])
        ax[k].yaxis.grid(True)
        
        ax[k].legend(fontsize = 6)
        
        if stats:
            renameDict = {Parameters[k] + '_mean' : Parameters[k]}
            if len(box_pairs) == 0:
                box_pairs = makeBoxPairs(co_order)
            addStat_df(ax[k], data_f_agg.rename(columns = renameDict), 
                    box_pairs, Parameters[k], CondCol, test = statMethod)
        

    plt.rcParams['axes.prop_cycle'] = my_default_color_cycle
    return(fig, ax)

In [9]:
def D1PlotPaired(data, Parameters=[], Filters=[], Boxplot=True, cellID='cellID', 
                   co_order=[], stats=True, statMethod='Wilcox_greater', box_pairs=[],
                   figSizeFactor = 1, markersizeFactor=1, orientation = 'h', labels = []):
    
    data_f = data
    for fltr in Filters:
        data_f = data_f.loc[fltr]
    
    if orientation == 'h':
        fig, ax = plt.subplots(1, 1, figsize = (5*figSizeFactor, 5))
    elif orientation == 'v':
        fig, ax = plt.subplots(1, 1, figsize = (5*figSizeFactor, 5))
        
    markersize = 5*markersizeFactor
        
    if 'EChadwick' in Parameters[0]:
        ax.set_yscale('log')
    
    pos = 1
    start = pos - 0.4
    stop = pos + 0.4
    NParms = len(Parameters)
    width = (stop-start)/NParms
    
    if len(labels) == len(Parameters):
        tickLabels = labels
    else:
        tickLabels = Parameters
    
    posParms = [start + 0.5*width + i*width for i in range(NParms)]
    parmsValues = np.array([data_f[P].values for P in Parameters])
#     for P in Parameters:
#         print(len(data_f[P].values))
#         print(data_f[P].values)
#     print(parmsValues)
    NValues = parmsValues.shape[1]
    print('Number of values : {:.0f}'.format(NValues))
    
    for i in range(NParms):
        color = my_default_color_list[i]
        
        ax.plot([posParms[i] for k in range(NValues)], parmsValues[i,:], 
                color = color, marker = 'o', markeredgecolor = 'k', markersize = markersize,
                ls = '', zorder = 3)
        
        if Boxplot:
            ax.boxplot([parmsValues[i]], positions=[posParms[i]], widths=width*0.8, patch_artist=True,
                        showmeans=False, showfliers=False,
                        medianprops={"color": 'k', "linewidth": 2, 'alpha' : 0.75},
                        boxprops={"facecolor": color, "edgecolor": 'k',
                                  "linewidth": 1, 'alpha' : 0.5},
#                         boxprops={"color": color, "linewidth": 0.5},
                        whiskerprops={"color": 'k', "linewidth": 2, 'alpha' : 0.5},
                        capprops={"color": 'k', "linewidth": 2, 'alpha' : 0.5},
                         zorder = 1)
    
    for k in range(NValues):
        ax.plot(posParms, parmsValues[:,k], 'k', ls = '-', linewidth = 0.5, marker = '', zorder = 2)
        
    if stats:
        X = np.array(posParms)
        Y = parmsValues
        box_pairs = makeBoxPairs([i for i in range(NParms)])
        addStat_arrays(ax, X, Y, box_pairs, test = statMethod, percentHeight = 99)
        
    ax.set_xlabel('')
    ax.set_xlim([pos-0.5, pos+0.5])
    ax.set_xticks(posParms)
    ax.set_xticklabels(labels = tickLabels, fontsize = 10)
    ax.tick_params(axis='x', labelrotation = 10)
        
    return(fig, ax)

In [10]:
def D2Plot(data, XCol='', YCol='', CondCol='', Filters=[], cellID='cellID', AvgPerCell=False,\
           xscale = 'lin', yscale = 'lin', figSizeFactor = 1, markers = []):
    data_filtered = data
    for fltr in Filters:
        data_filtered = data_filtered.loc[fltr]
    
    NCond = len(CondCol)    
    if NCond == 1:
        CondCol = CondCol[0]
    elif NCond > 1:
        newColName = ''
        for i in range(NCond):
            newColName += CondCol[i]
            newColName += ' & '
        newColName = newColName[:-3]
        data_filtered[newColName] = ''
        for i in range(NCond):
            data_filtered[newColName] += data_filtered[CondCol[i]].astype(str)
            data_filtered[newColName] = data_filtered[newColName].apply(lambda x : x + ' & ')
        data_filtered[newColName] = data_filtered[newColName].apply(lambda x : x[:-3])
        CondCol = newColName
    
    if AvgPerCell:
        group = data_filtered.groupby(cellID)
        dictAggMean = getDictAggMean(data_filtered)
        data_filtered = group.agg(dictAggMean.pop(cellID)) #.reset_index(level=0, inplace=True)
        data_filtered.reset_index(level=0, inplace=True)
        
    Conditions = list(data_filtered[CondCol].unique())
    fig, ax = plt.subplots(1, 1, figsize = (8*figSizeFactor,5))
    markerSize = 8
    
    if xscale == 'log':
        ax.set_xscale('log')
    if yscale == 'log':
        ax.set_yscale('log')
    
    current_color_list = getStyleLists(Conditions, styleDict1).as_hex()
    cc = cycler(color=current_color_list)
    ax.set_prop_cycle(cc)
    
    im = 0
    for c in Conditions:
        Xraw = data_filtered[data_filtered[CondCol] == c][XCol].values
        Yraw = data_filtered[data_filtered[CondCol] == c][YCol].values
        XYraw = np.array([Xraw,Yraw]).T
        XY = XYraw[~np.isnan(XYraw).any(axis=1), :]
        X, Y = XY[:,0], XY[:,1]
        if len(markers) == 0:
            m = 'o'
        else:
            m = markers[im]
            im += 1
        
        if len(X) == 0:
            ax.plot([], [])
            if modelFit:
                ax.plot([], [])
                
        elif len(X) > 0:                    
            ax.plot(X, Y, m, markersize = markerSize, markeredgecolor='k', markeredgewidth = 1, label=c)
            
    ax.set_xlabel(XCol)
    ax.set_xlim([min(0,1.1*np.min(data_filtered[XCol])), 1.1*np.max(data_filtered[XCol])])
    ax.set_ylabel(YCol)
    if not yscale == 'log':
        ax.set_ylim([min(0,1.1*np.min(data_filtered[YCol])), 1.1*np.max(data_filtered[YCol])])
    ax.legend(loc='upper left')
    return(fig, ax)

In [11]:
def D2Plot_wFit(data, XCol='', YCol='', CondCol='', Filters=[], cellID='cellID', AvgPerCell=False, showManips = True,
                modelFit=False, modelType='y=ax+b',\
                xscale = 'lin', yscale = 'lin', 
                figSizeFactor = 1):
    
    data_filtered = data
    for fltr in Filters:
        data_filtered = data_filtered.loc[fltr]
    
    NCond = len(CondCol)    
    if NCond == 1:
        CondCol = CondCol[0]
    elif NCond > 1:
        newColName = ''
        for i in range(NCond):
            newColName += CondCol[i]
            newColName += ' & '
        newColName = newColName[:-3]
        data_filtered[newColName] = ''
        for i in range(NCond):
            data_filtered[newColName] += data_filtered[CondCol[i]].astype(str)
            data_filtered[newColName] = data_filtered[newColName].apply(lambda x : x + ' & ')
        data_filtered[newColName] = data_filtered[newColName].apply(lambda x : x[:-3])
        CondCol = newColName
    
    if AvgPerCell:
        group = data_filtered.groupby(cellID)
        dictAggMean = getDictAggMean(data_filtered)
        data_filtered = group.agg(dictAggMean.pop(cellID)) #.reset_index(level=0, inplace=True)
        data_filtered.reset_index(level=0, inplace=True)
        
    Conditions = list(data_filtered[CondCol].unique())
    fig, ax = plt.subplots(1, 1, figsize = (8*figSizeFactor,5))
    markerSize = 5
    
    if xscale == 'log':
        ax.set_xscale('log')
    if yscale == 'log':
        ax.set_yscale('log')
    
#     current_color_list = getStyleLists(Conditions, styleDict1).as_hex()
#     cc = cycler(color=current_color_list)
#     ax.set_prop_cycle(cc)
    
#     if modelFit:
#         # Tweak the style cycle to plot for each condition: the points ('o') and then the fit ('-') with the same color.
# #         current_prop_cycle = plt.rcParams['axes.prop_cycle']
# #         current_color_list = prop_cycle.by_key()['color']
#         ncustom_color_list = list(np.array([current_color_list, current_color_list]).flatten(order='F'))
#         # if new_color_list was ['red', 'green', blue'], custom_color_list is now ['red', 'red', 'green', 'green', blue', blue']
#         cc = cycler(color=ncustom_color_list)
#         ax.set_prop_cycle(cc)
    
    for i in range(len(Conditions)):
        c = Conditions[i]
        color = my_default_color_list[i]
#         marker = my_default_marker_list[i]
        Xraw = data_filtered[data_filtered[CondCol] == c][XCol].values
        Yraw = data_filtered[data_filtered[CondCol] == c][YCol].values
        Mraw = data_filtered[data_filtered[CondCol] == c]['manipID'].values
        XYraw = np.array([Xraw,Yraw]).T
        XY = XYraw[~np.isnan(XYraw).any(axis=1), :]
        X, Y = XY[:,0], XY[:,1]
        M = Mraw[~np.isnan(XYraw).any(axis=1)]
        if len(X) == 0:
            ax.plot([], [])
            if modelFit:
                ax.plot([], [])
                
        elif len(X) > 0:
            eqnText = ''

            if modelFit:
                print('Fitting condition ' + c + ' with model ' + modelType)
                if modelType == 'y=ax+b':
                    params, results = fitLine(X, Y) # Y=a*X+b ; params[0] = b,  params[1] = a
                    pval = results.pvalues[1] # pvalue on the param 'a'
                    eqnText += " ; Y = {:.1f} X + {:.1f}".format(params[1], params[0])
                    eqnText += " ; p-val = {:.3f}".format(pval)
                    print("Y = {:.5} X + {:.5}".format(params[1], params[0]))
                    print("p-value on the 'a' coefficient: {:.4e}".format(pval))
                    fitY = params[1]*X + params[0]
                    imin = np.argmin(X)
                    imax = np.argmax(X)
                    ax.plot([X[imin],X[imax]], [fitY[imin],fitY[imax]], '--', lw = '1')

                elif modelType == 'y=A*exp(kx)':
                    params, results = fitLine(X, np.log(Y)) # Y=a*X+b ; params[0] = b,  params[1] = a
                    pval = results.pvalues[1] # pvalue on the param 'k'
                    eqnText += " ; Y = {:.1f}*exp({:.1f}*X)".format(params[0], params[1])
                    eqnText += " ; p-val = {:.3f}".format(pval)
                    print("Y = {:.5}*exp({:.5}*X)".format(np.exp(params[0]), params[1]))
                    print("p-value on the 'k' coefficient: {:.4e}".format(pval))
                    fitY = np.exp(params[0])*np.exp(params[1]*X)
                    imin = np.argmin(X)
                    imax = np.argmax(X)
                    ax.plot([X[imin],X[imax]], [fitY[imin],fitY[imax]], '--', lw = '1')
                    
                elif modelType == 'y=k*x^a':
                    posValues = ((X > 0) & (Y > 0))
                    X, Y = X[posValues], Y[posValues]
                    params, results = fitLine(np.log(X), np.log(Y)) # Y=a*X+b ; params[0] = b,  params[1] = a
                    k = np.exp(params[0])
                    a = params[1]
                    pval = results.pvalues[1] # pvalue on the param 'a'
                    eqnText += " ; Y = {:.1e} * X^{:.1f}".format(k, a)
                    eqnText += " ; p-val = {:.3f}".format(pval)
                    print("Y = {:.4e} * X^{:.4f}".format(k, a))
                    print("p-value on the 'a' coefficient: {:.4e}".format(pval))
                    fitY = k * X**a
                    imin = np.argmin(X)
                    imax = np.argmax(X)
                    ax.plot([X[imin],X[imax]], [fitY[imin],fitY[imax]], '--', lw = '1', color = color)
                
                print('Number of values : {:.0f}'.format(len(Y)))
                print('\n')
            
#                 if showManips:
#                     allManipID = list(data_filtered[data_filtered[CondCol] == c]['manipID'].unique())
#                     dictMarker = {}
#                     markers = []
#                     for mi in range(len(allManipID)):
#                         dictMarker[allManipID[mi]] = my_default_marker_list[mi]
#                     for k in range(len(M)):
#                         thisManipID = M[k]
#                         markers = dictMarker[thisManipID]
#                 else:
#                     markers = ['o' for i in range(len(M))]
#                 markers = np.array(markers)
                
            ax.plot(X, Y, 
                    color = color, ls = '',
                    marker = 'o', markersize = markerSize, markeredgecolor='k', markeredgewidth = 1, 
                    label = c + eqnText)
            
            
    ax.set_xlabel(XCol)
    ax.set_xlim([0.9*np.min(data_filtered[XCol]), 1.1*np.max(data_filtered[XCol])])
    ax.set_ylabel(YCol)
    if not yscale == 'log':
        ax.set_ylim([0.9*np.min(data_filtered[YCol]), 1.1*np.max(data_filtered[YCol])])
    ax.legend(loc='upper left')
    
    
    return(fig, ax)

These functions use the Bokeh library to display 1D categorical or 2D plots with interactive plots. They are less flexible but can be nice to explore the data set since you can display the cellID which is the source of each point by passing your pointer over it.

In [12]:
def D1PlotInteractive(data, CondCol='',Parameters=[],Filters=[],AvgPerCell=False,cellID='cellID'):
    data_filtered = data
    for fltr in Filters:
        data_filtered = data_filtered.loc[fltr]
        
#     print(data_filtered[cellID])
    if AvgPerCell:
        group = data_filtered.groupby(cellID)
        dictAggMean = getDictAggMean(data_filtered)
        data_filtered = group.agg(dictAggMean.pop(cellID)) #.reset_index(level=0, inplace=True)
        data_filtered.reset_index(level=0, inplace=True)
    
#     return(data_filtered)
    
    NPlots = len(Parameters)
    Conditions = list(data_filtered[CondCol].unique())
    if NPlots > 1:
        plots = []
        NCond = len(Conditions)
        data_filtered['X'] = 0
        data_filtered['X_jitter'] = 0.
        dictTicks = {}
        for i in range(NCond):
            mask = data_filtered[CondCol] == Conditions[i]
            data_filtered.loc[mask, 'X'] = i+1
            dictTicks[i+1] = Conditions[i]
        for i in data_filtered.index:
            data_filtered.loc[i, 'X_jitter'] = data_filtered.loc[i, 'X'] + 0.4*(np.random.rand(1)[0]-0.5)
        source = ColumnDataSource(
            data=data_filtered[[cellID]+[CondCol]+Parameters+['X','X_jitter']]
        )        
        
        for k in range(NPlots):
            hover = HoverTool(
                tooltips=[
                    ('Cell ID', "@"+cellID),
                    (Parameters[k], "@"+Parameters[k]),
                ]
            )
            index_cmap = factor_cmap(CondCol, palette=Category10[10], factors=sorted(data_filtered[CondCol].unique()), end=1)
            p = figure(plot_width=450, plot_height=500, tools=[hover], title="InteractivePlot") # 
            p.circle('X_jitter', Parameters[k], size=8, alpha = 0.6, source=source,fill_color=index_cmap,line_color='black')
            # Format
            p.x_range = Range1d(0, NCond+1)
            p.y_range = Range1d(min(0,1.1*np.min(data_filtered[Parameters[0]])), 1.1*np.max(data_filtered[Parameters[k]]))
            p.xaxis.ticker = [i for i in range(1,NCond+1)]
            p.xaxis.major_label_overrides = dictTicks
            p.xaxis.axis_label = CondCol
            p.xaxis.axis_label_text_font_size = '18pt'
            p.xaxis.major_label_text_font_size = '16pt'
            p.yaxis.axis_label = Parameters[k]
            p.yaxis.axis_label_text_font_size = '18pt'
            p.yaxis.major_label_text_font_size = '16pt'
            
            plots.append(p)
            
        p = gridplot(plots, ncols=2, toolbar_location=None)
        
        
    else:
        hover = HoverTool(
            tooltips=[
                ('Cell ID', "@"+cellID),
                (Parameters[0], "@"+Parameters[0]),
            ]
        )
        
        NCond = len(Conditions)
        data_filtered['X'] = 0
        data_filtered['X_jitter'] = 0.
        dictTicks = {}
        for i in range(NCond):
            mask = data_filtered[CondCol] == Conditions[i]
            data_filtered.loc[mask, 'X'] = i+1
            dictTicks[i+1] = Conditions[i]
        for i in data_filtered.index:
            data_filtered.loc[i, 'X_jitter'] = data_filtered.loc[i, 'X'] + 0.4*(np.random.rand(1)[0]-0.5)
        source = ColumnDataSource(
            data=data_filtered[[cellID]+[CondCol]+Parameters+['X','X_jitter']]
        )
        index_cmap = factor_cmap(CondCol, palette=Category10[10], factors=sorted(data_filtered[CondCol].unique()), end=1)
        TOOLS = "hover,pan,box_zoom,wheel_zoom,reset,save,help"
        p = figure(plot_width=500, plot_height=500, tools=TOOLS, title="InteractivePlot") # 
        p.circle('X_jitter', Parameters[0], size=8, alpha = 0.6, source=source,fill_color=index_cmap,line_color='black')
        # Format
        p.x_range = Range1d(0, NCond+1)
        p.y_range = Range1d(min(0,1.1*np.min(data_filtered[Parameters[0]])), 1.1*np.max(data_filtered[Parameters[0]]))
        p.xaxis.ticker = [i for i in range(1,NCond+1)]
        p.xaxis.major_label_overrides = dictTicks
        p.xaxis.axis_label = CondCol
        p.xaxis.axis_label_text_font_size = '18pt'
        p.xaxis.major_label_text_font_size = '16pt'
        p.yaxis.axis_label = Parameters[0]
        p.yaxis.axis_label_text_font_size = '18pt'
        p.yaxis.major_label_text_font_size = '16pt'
    return(p)

In [13]:
def D2PlotInteractive(data, XCol='',YCol='',CondCol='',Filters=[], cellID='cellID',AvgPerCell=False):
    
    data_filtered = data
    for fltr in Filters:
        data_filtered = data_filtered.loc[fltr]
        
    if AvgPerCell:
        group = data_filtered.groupby(cellID)
        dictAggMean = getDictAggMean(data_filtered)
        data_filtered = group.agg(dictAggMean.pop(cellID)) #.reset_index(level=0, inplace=True)
        data_filtered.reset_index(level=0, inplace=True)
    
    Conditions = list(data_filtered[CondCol].unique())

    NCond = len(Conditions)
    dictTicks = {}
    for i in range(NCond):
        dictTicks[i+1] = Conditions[i]
    
    source = ColumnDataSource(
        data=data_filtered[[cellID,CondCol,XCol,YCol]]
    )
    
    hover = HoverTool(
        tooltips=[
            ('Cell ID', "@"+cellID),
            (XCol, "@"+XCol),
            (YCol, "@"+YCol),
            (CondCol, "@"+CondCol),
        ]
    )
    
    index_cmap = factor_cmap(CondCol, palette=Category10[10], factors=sorted(data_filtered[CondCol].unique()), end=1)
    TOOLS = "pan,box_zoom,wheel_zoom,reset,save,help"
    p = figure(plot_width=900, plot_height=500, tools=TOOLS, title="InteractivePlot",toolbar_location="below") # 
    p.circle(XCol, YCol, size=8, alpha = 0.6, source=source,fill_color=index_cmap,line_color='black')
    p.add_tools(hover)
    # Format
    p.x_range = Range1d(0, 1.1*np.max(data_filtered[XCol]))
    p.y_range = Range1d(0, 1.1*np.max(data_filtered[YCol]))
    p.xaxis.axis_label = XCol
    p.xaxis.axis_label_text_font_size = '18pt'
    p.xaxis.major_label_text_font_size = '16pt'
    p.yaxis.axis_label = YCol
    p.yaxis.axis_label_text_font_size = '18pt'
    p.yaxis.major_label_text_font_size = '16pt'
    return(p)

Other subfunctions useful to plot

In [14]:
def getDictAggMean(df):
    dictAggMean = {}
    for c in df.columns:
    #         t = df[c].dtype
    #         print(c, t)
            try :
                if np.array_equal(df[c], df[c].astype(bool)):
                    dictAggMean[c] = 'min'
                else:
                    try:
                        if not c.isnull().all():
                            np.mean(df[c])
                            dictAggMean[c] = 'mean'
                    except:
                        dictAggMean[c] = 'first'
            except:
                    dictAggMean[c] = 'first'
    return(dictAggMean)


def getAggDf(df, cellID, CondCol, Variables):
    
    allCols = df.columns.values
    group = df.groupby(cellID)
    
    dictAggCategories = {}
    Categories = ['date', 'cellName', 'manipID', 'experimentType', 'drug', 
                  'substrate', 'cell type', 'cell subtype', 'bead type', 'bead diameter']
    if CondCol not in Categories:
        Categories += [CondCol]
    for c in Categories:
        if c in allCols:
            dictAggCategories[c] = 'first'
    dfCats = group.agg(dictAggCategories)
    
    dictAggCount = {'compNum' : 'count'}
    renameDict = {'compNum' : 'Ncomp'}
    dfCount = group.agg(dictAggCount).rename(columns=renameDict)
    
    dictAggMeans = {}
    renameDict = {}
    for v in Variables:
        dictAggMeans[v] = 'mean'
        renameDict[v] = v + '_mean'
    dfMeans = group.agg(dictAggMeans).rename(columns=renameDict)
        
    dictAggStds = {}
    renameDict = {}
    for v in Variables:
        dictAggStds[v] = 'std'
        renameDict[v] = v + '_std'
    dfStds = group.agg(dictAggStds).rename(columns=renameDict)
    
    if 'ctFieldThickness' in Variables:
        dfStds['ctFieldThickness_std'] = group.agg({'ctFieldFluctuAmpli' : 'median'}).values
        #ctFieldThickness ; ctFieldFluctuAmpli
    
#     for v in Variables:
        
    
    dfAgg = pd.merge(dfCats, dfCount, how="inner", on='cellID',
        #     left_on=None,right_on=None,left_index=False,right_index=False,sort=True,
        #     suffixes=("_x", "_y"),copy=True,indicator=False,validate=None,
        )
    
    dfAgg = pd.merge(dfAgg, dfMeans, how="inner", on='cellID',
        #     left_on=None,right_on=None,left_index=False,right_index=False,sort=True,
        #     suffixes=("_x", "_y"),copy=True,indicator=False,validate=None,
        )
    
    dfAgg = pd.merge(dfAgg, dfStds, how="inner", on='cellID',
        #     left_on=None,right_on=None,left_index=False,right_index=False,sort=True,
        #     suffixes=("_x", "_y"),copy=True,indicator=False,validate=None,
        )
    
    return(dfAgg)


def makeOrder(*args):
    order = []
    listeTuple = list(itertools.product(*args, repeat=1))
    for tup in listeTuple:
        tmpText = ''
        for word in tup:
            tmpText += word
            tmpText += ' & '
        tmpText = tmpText[:-3]
        order.append(tmpText)
    return(order)


def makeBoxPairs(O):
    return(list(itertools.combinations(O, 2)))


def renameAxes(axes, rD):
    try:
        N = len(axes)
    except:
        axes = [axes]
        N = 1
    for i in range(N):
        # set xticks
        xticksTextObject = axes[i].get_xticklabels()
        xticksList = [xticksTextObject[j].get_text() for j in range(len(xticksTextObject))]
        test_hasXLabels = (len(''.join(xticksList)) > 0)
        if test_hasXLabels:
            newXticksList = [rD.get(k, k) for k in xticksList]
            axes[i].set_xticklabels(newXticksList)
        
        # set xlabel
        xlabel = axes[i].get_xlabel()
        newXlabel = rD.get(xlabel, xlabel)
        axes[i].set_xlabel(newXlabel)
        # set ylabel
        ylabel = axes[i].get_ylabel()
        newYlabel = rD.get(ylabel, ylabel)
        axes[i].set_ylabel(newYlabel)
        

def addStat_df(ax, data, box_pairs, param, cond, test = 'Mann-Whitney', percentHeight = 98):
    refHeight = np.percentile(data[param].values, percentHeight)
    currentHeight = refHeight
    scale = ax.get_yscale()
    xTicks = ax.get_xticklabels()
    dictXTicks = {xTicks[i].get_text() : xTicks[i].get_position()[0] for i in range(len(xTicks))}
    for bp in box_pairs:
        c1 = data[data[cond] == bp[0]][param].values
        c2 = data[data[cond] == bp[1]][param].values
        if test=='Mann-Whitney':
            statistic, pval = st.mannwhitneyu(c1,c2)
        elif test=='Wilcox_2s':
            statistic, pval = st.wilcoxon(c1,c2, alternative = 'two-sided')
        elif test=='Wilcox_greater':
            statistic, pval = st.wilcoxon(c1,c2, alternative = 'greater')
        elif test=='Wilcox_less':
            statistic, pval = st.wilcoxon(c1,c2, alternative = 'less')
        elif test=='t-test':
            statistic, pval = st.ttest_ind(c1,c2)
        text = 'ns'
        if pval < 0.05 and pval > 0.01:
            text = '*'
        elif pval < 0.01 and pval > 0.001:
            text = '**'
        elif pval < 0.001 and pval < 0.001:
            text = '***'
        elif pval < 0.0001:
            text = '****'
        ax.plot([bp[0], bp[1]], [currentHeight, currentHeight], 'k-', lw = 1)
        XposText = (dictXTicks[bp[0]]+dictXTicks[bp[1]])/2
        if scale == 'log':
            power = 0.01* (text=='ns') + 0.000 * (text!='ns')
            YposText = currentHeight*(refHeight**power)
        else:
            factor = 0.03 * (text=='ns') + 0.000 * (text!='ns')
            YposText = currentHeight + factor*refHeight
        ax.text(XposText, YposText, text, ha = 'center', color = 'k')
#         if text=='ns':
#             ax.text(posText, currentHeight + 0.025*refHeight, text, ha = 'center')
#         else:
#             ax.text(posText, currentHeight, text, ha = 'center')
        if scale == 'log':
            currentHeight = currentHeight*(refHeight**0.05)
        else:
            currentHeight =  currentHeight + 0.15*refHeight
    ax.set_ylim([ax.get_ylim()[0], currentHeight])
    

def addStat_arrays(ax, X, Y, box_pairs, test = 'Mann-Whitney', percentHeight = 98):
    refHeight = np.percentile(Y, percentHeight)
    currentHeight = refHeight
    scale = ax.get_yscale()
    
    for bp in box_pairs:
        y1 = Y[bp[0], :]
        y2 = Y[bp[1], :]
        
        if test=='Mann-Whitney':
            statistic, pval = st.mannwhitneyu(y1, y2)
        elif test=='Wilcox_2s':
            statistic, pval = st.wilcoxon(y1, y2, alternative = 'two-sided')
        elif test=='Wilcox_greater':
            statistic, pval = st.wilcoxon(y1, y2, alternative = 'greater')
        elif test=='Wilcox_less':
            statistic, pval = st.wilcoxon(y1, y2, alternative = 'less')
        elif test=='t-test':
            statistic, pval = st.ttest_ind(y1, y2)
            
        text = 'ns'
        if pval < 0.05 and pval > 0.01:
            text = '*'
        elif pval < 0.01 and pval > 0.001:
            text = '**'
        elif pval < 0.001 and pval < 0.001:
            text = '***'
        elif pval < 0.0001:
            text = '****'
            
        ax.plot([X[bp[0]], X[bp[1]]], [currentHeight, currentHeight], 'k-', lw = 1)
        XposText = (X[bp[0]]+X[bp[1]])/2
        if scale == 'log':
            power = 0.01* (text=='ns') + 0.000 * (text!='ns')
            YposText = currentHeight*(refHeight**power)
        else:
            factor = 0.03 * (text=='ns') + 0.000 * (text!='ns')
            YposText = currentHeight + factor*refHeight
        ax.text(XposText, YposText, text, ha = 'center', color = 'k')
#         if text=='ns':
#             ax.text(posText, currentHeight + 0.025*refHeight, text, ha = 'center')
#         else:
#             ax.text(posText, currentHeight, text, ha = 'center')
        if scale == 'log':
            currentHeight = currentHeight*(refHeight**0.05)
        else:
            currentHeight =  currentHeight + 0.15*refHeight
    ax.set_ylim([ax.get_ylim()[0], currentHeight])


def getStyleCycle(co_order, styleDict):
    colors = []
    markers = []
    linestyles = []
    linewidth = []

    for co in co_order:
        coStyle = styleDict[co]
        if 'color' in coStyle.keys():
            colors.append(coStyle['color'])
        else:
            colors.append('')
        if 'marker' in coStyle.keys():
            markers.append(coStyle['marker'])
        else:
            markers.append('')
        if 'linestyle' in coStyle.keys():
            linestyles.append(coStyle['marker'])
        else:
            linestyles.append('')
        if 'linewidth' in coStyle.keys():
            linewidth.append(coStyle['linewidth'])
        else:
            linewidth.append(1)
            
    cc = (cycler(color=colors) +
          cycler(linestyle=linestyles) +
          cycler(marker=markers) +
          cycler(linewidth=linewidth))
            
    return(cc)


def getStyleLists(co_order, styleDict):
    colors = []
    markers = []
    try:
        for co in co_order:
            coStyle = styleDict[co]
            if 'color' in coStyle.keys():
                colors.append(coStyle['color'])
            else:
                colors.append('')
            if 'marker' in coStyle.keys():
                markers.append(coStyle['marker'])
            else:
                markers.append('')
        palette = sns.color_palette(colors)
    except:
        palette = sns.color_palette()
    return(palette)


def buildStyleDictMCA():
    # TBC
    styleDict = {}
    return(styleDict)

In [15]:
# Test getAggDf(df, cellID, CondCol, Variables)

# data = GlobalTable_meca_Py2

# Filters = [(data['validatedFit'] == True), (data['validatedThickness'] == True)]

# data_f = data
# for fltr in Filters:
#     data_f = data_f.loc[fltr]

# # dfA = getAggDf(data_f, 'cellID', 'bead type', ['surroundingThickness', 'EChadwick'])
# dfA = getAggDf(data_f, 'cellID', 'bead type', ['ctFieldThickness', 'EChadwick'])
# dfA

In [16]:
# Test the D2Plot_wFit
# data = GlobalTable_meca_Py2

# Filters = [(data['validatedFit'] == True), 
#            (data['validatedThickness'] == True), 
#            (data['substrate'] == '20um fibronectin discs'),
#            (data['date'].apply(lambda x : x in ['21-12-16','21-12-08']))]

# fig, ax = D2Plot_wFit(data, XCol='ctFieldThickness', YCol='EChadwick', CondCol = ['bead type'],
#            Filters=Filters, cellID = 'cellID', AvgPerCell=True, xscale = 'log', yscale = 'log', 
#            modelFit=True, modelType = 'y=k*x^a')

# ax.set_ylabel('EChadwick (Pa)')
# ax.set_xlabel('Thickness at low force (nm)')
# fig.suptitle('3T3aSFL: E(h)')
# ax.legend(loc = 'upper right')

# # archiveFig(fig, ax, name='E(h)_3T3aSFL_Dec21_M450-5-13_vs_M270-14-54', figDir = todayFigDir, figSubDir=figSubDir)
# fig.show()

In [ ]:
# Test of the averaging per cell routine

# data = GlobalTable_meca
# CondCol='drug'
# Parameters=['SurroundingThickness','EChadwick']
# Filters = [(GlobalTable_meca['Validated'] == 1)]
# AvgPerCell=True
# cellID='CellName'

# data_filtered = data
# for fltr in Filters:
#     data_filtered = data_filtered.loc[fltr]

# group = data_filtered.groupby(cellID)
# dictAggMean = getDictAggMean(data_filtered)
# data_filtered = group.agg(dictAggMean.pop(cellID)) #.reset_index(level=0, inplace=True)
# data_filtered.reset_index(level=0, inplace=True)
# data_filtered=data_filtered[[cellID]+[CondCol]+Parameters]
# print(data_filtered)

In [ ]:
# Test of a routine to remove points of a list of XY positions where at least 1 of the coordinates is 'nan'

# XYraw = np.array([[np.nan, 2, 3, np.nan, 5], [10,20,30,40,50]])
# XYraw = XYraw.T
# XY = XYraw[~np.isnan(XYraw).any(axis=1), :]
# X, Y = XY[:,0], XY[:,1]
# X, Y

In [ ]:
# Test of a routine to double each element in a list ; example [1, 2, 3] -> [1, 1, 2, 2, 3, 3]

# newnew_color_list = np.array([new_color_list, new_color_list])
# custom_color_list = list(np.array([new_color_list, new_color_list]).flatten(order='F'))
# custom_color_list

In [ ]:
# Test of makeOrder function

# print(makeOrder(['none','doxycyclin'],['BSA coated glass','20um fibronectin discs']))
# print(makeOrder(['A','B']))
# print(makeOrder(['A','B'], ['C','D']))
# print(makeOrder(['A','B'], ['C','D'], ['E','F']))

In [ ]:
# Test of makeBoxPairs function

# O = makeOrder(['none','doxycyclin'],['BSA coated glass','20um fibronectin discs'])
# makeBoxPairs(O)

In [ ]:
# Test of custom cycles

cc = (cycler(color=list('rgb')) +
      cycler(linestyle=['', '', '']) +
      cycler(marker=['o','*','<']) +
     cycler(linewidth=[1,1,1]))
cc

# plt.rcParams['axes.prop_cycle'] = cc

# fig, ax = plt.subplots()
# for i in range(10):
#     ax.plot([0,1], [0,i])
# fig.show()

### Exploratory Graphs

#### Classic

In [ ]:
rawMecaTable = getGlobalTable_meca()
# rawMecaTable.head()

In [ ]:
# rawMecaTable.loc[]
# Agréger tous les M450 WT sous un même nom

In [ ]:
Filters = [(rawMecaTable['Validated'] == 1), ((rawMecaTable['ExpType'] == 'DictyDB_M450'))] #  | (rawMecaTable['ExpType'] == 'DictyDB_M450-Multi')
co_order = makeOrder(['02s', '05s', '1s', '2s', '4s', '7s', '12s']) # co_order = co_order,
fig, ax00 = D1Plot(rawMecaTable, CondCol=['TpsComp'],Parameters=['SurroundingThickness','EChadwick'], co_order = co_order,\
                   Filters=Filters,AvgPerCell=False,cellID='CellName',stats=False,figSizeFactor = 1.8,markerSizeFactor=0.5)
fig.suptitle('M450, various rates')
renameAxes(ax00,renameDict1)
fig.show()

In [ ]:
co_order

In [ ]:
Filters = [(rawMecaTable['Validated'] == 1), ((rawMecaTable['ExpType'] == 'DictyDB_M270'))]
fig, ax01 = D1Plot(rawMecaTable, CondCol=['TpsComp'],Parameters=['SurroundingThickness','EChadwick'],Filters=Filters,\
                   AvgPerCell=False,cellID='CellName',stats=False,figSizeFactor = 1.8,markerSizeFactor=0.5)
fig.suptitle('M270, various rates')
fig.show()

In [ ]:
Filters = [(GlobalTable_ctField['validated'] == True), (GlobalTable_ctField['medianThickness'] <= 1000)]
fig, ax = D1Plot(GlobalTable_ctField, CondCol=['drug','substrate'],Parameters=['medianThickness','fluctuAmpli'],Filters=Filters)
fig.suptitle('3T3aSFL on patterns: Ct Field')
fig.show()

In [ ]:
Filters = [(GlobalTable_meca['Validated'] == 1), (GlobalTable_meca['cell subtype'] == 'aSFL')]
co_order = makeOrder(['none','doxycyclin'],['BSA coated glass','20um fibronectin discs'])
fig, ax = D1Plot(GlobalTable_meca, CondCol=['drug','substrate'],Parameters=['SurroundingThickness','EChadwick'],Filters=Filters,AvgPerCell=True,cellID='CellName', co_order=co_order)
fig.suptitle('3T3aSFL on diverse substrates: Compressions')
fig.show()

In [ ]:
Filters = [(GlobalTable_meca['Validated'] == 1), (GlobalTable_meca['cell subtype'] == 'aSFL-6FP')]
fig, ax = D1Plot(GlobalTable_meca, CondCol=['drug','substrate'],Parameters=['SurroundingThickness','EChadwick'],Filters=Filters,AvgPerCell=True,cellID='CellName')
fig.suptitle('3T3aSFL on patterns: Compressions')
fig.show()

In [ ]:
Filters = [(GlobalTable_meca['Validated'] == 1), (GlobalTable_meca['substrate'] == '20um fibronectin discs')]
co_order = makeOrder([['aSFL','aSFL-6FP'],['none','doxycyclin']])
fig, ax = D1Plot(GlobalTable_meca, CondCol=['cell subtype','drug'],Parameters=['SurroundingThickness','EChadwick'],Filters=Filters,AvgPerCell=True,cellID='CellName',co_order=co_order)
fig.suptitle('3T3aSFL on diverse substrates: Compressions')
fig.show()

In [ ]:
Filters = [(GlobalTable_meca['Validated'] == 1), (GlobalTable_meca['substrate'] == '20um fibronectin discs')]
co_order = makeOrder([['aSFL','aSFL-6FP'],['none','doxycyclin']])
fig, ax = D1Plot(GlobalTable_meca, CondCol=['cell subtype','drug'],Parameters=['SurroundingThickness','EChadwick'],Filters=Filters,AvgPerCell=True,cellID='CellName',co_order=co_order)
fig.suptitle('3T3aSFL SHORT vs LONG linker: Compressions')
fig.show()

In [ ]:
Filters = [(GlobalTable_ctField['validated'] == True), (GlobalTable_ctField['cell subtype'] == 'aSFL')]
fig, ax = D2Plot(GlobalTable_ctField, XCol='medianThickness',YCol='fluctuAmpli',CondCol = ['drug'], Filters=Filters, modelFit=True)
fig.suptitle('3T3aSFL - Dh = f(H)')
archiveFig(fig, ax, name='aSFL_Dh(h)_drug', figDir = todayFigDir, figSubDir='ThicknessPlots')
fig.show()

In [ ]:
# Same as above without the 2 thickest cells
Filters = [(GlobalTable_ctField['validated'] == True), (GlobalTable_ctField['cell subtype'] == 'aSFL'), (GlobalTable_ctField['medianThickness'] <= 700)]
fig, ax = D2Plot(GlobalTable_ctField, XCol='medianThickness',YCol='fluctuAmpli',CondCol = ['drug'], Filters=Filters, modelFit=True)
fig.suptitle('3T3aSFL - Dh = f(H)')
archiveFig(fig, ax, name='aSFL_Dh(h)_drug_wo2LastPoints', figDir = todayFigDir, figSubDir='ThicknessPlots')
fig.show()

In [ ]:
Filters = [(GlobalTable_meca['Validated'] == True), (GlobalTable_meca['cell subtype'] == 'aSFL')]
fig, ax = D2Plot(GlobalTable_meca, XCol='SurroundingThickness',YCol='EChadwick',CondCol = ['substrate','drug'],\
           Filters=Filters, cellID = 'CellName', AvgPerCell=True, modelFit=False, modelType='y=A*exp(kx)', yscale = 'log')
fig.suptitle('3T3aSFL: E(h)')
renameAxes(ax,renameDict1)
ax.legend(loc='upper right')
archiveFig(fig, ax, name='aSFL_E(h)_drug&substrate_01', figDir = todayFigDir, figSubDir='')
fig.show()

In [ ]:
Filters = [(GlobalTable_meca['Validated'] == True), (GlobalTable_meca['cell subtype'] == 'aSFL'), (GlobalTable_meca['substrate'] == '20um fibronectin discs')]
fig, ax = D2Plot(GlobalTable_meca, XCol='SurroundingThickness',YCol='EChadwick',CondCol = ['substrate','drug'],\
           Filters=Filters, cellID = 'CellName', AvgPerCell=False, modelFit=False, modelType='y=A*exp(kx)', xscale = 'log', \
                 yscale = 'log')
fig.suptitle('3T3aSFL: E(h)')
renameAxes(ax,renameDict1)
ax.set_xlim([90, 1500])
ax.legend(loc='upper right')
archiveFig(fig, ax, name='aSFL_E(h)_drug&substrate_00_allComp', figDir = todayFigDir, figSubDir='')
fig.show()

In [ ]:
Filters = [(GlobalTable_meca['Validated'] == True), (GlobalTable_meca['substrate'] == '20um fibronectin discs')]
fig, ax = D2Plot(GlobalTable_meca, XCol='SurroundingThickness',YCol='EChadwick',CondCol = ['cell subtype','drug'],\
           Filters=Filters, cellID = 'CellName', AvgPerCell=True, modelFit=False, modelType='y=A*exp(kx)', yscale = 'log')
fig.suptitle('3T3aSFL: E(h)')
renameAxes(ax,renameDict1)
ax.legend(loc='upper right')
archiveFig(fig, ax, name='aSFL_E(h)_drug&substrate_02', figDir = todayFigDir, figSubDir='')
fig.show()

In [ ]:
Filters = [(GlobalTable_meca['Validated'] == True), (GlobalTable_meca['substrate'] == '20um fibronectin discs')]
fig, ax = D2Plot(GlobalTable_meca, XCol='SurroundingThickness',YCol='EChadwick',CondCol = ['cell subtype','drug'],\
           Filters=Filters, cellID = 'CellName', AvgPerCell=False, modelFit=False, modelType='y=A*exp(kx)', yscale = 'log')
fig.suptitle('3T3aSFL: E(h)')
renameAxes(ax,renameDict1)
ax.legend(loc='upper right')
archiveFig(fig, ax, name='aSFL_E(h)_drug&substrate_02_allComp', figDir = todayFigDir, figSubDir='')
fig.show()

In [ ]:
co_o = ['BSA coated glass & none', 'BSA coated glass & doxycyclin', '20um fibronectin discs & doxycyclin', '20um fibronectin discs & none']
# getStyleLists(co_o, styleDict1)
sns.color_palette(['#ff9896', '#d62728', '#1f77b4', '#aec7e8'])

In [ ]:
Filters = [(GlobalTable_meca['Validated'] == True), (GlobalTable_meca['cell subtype'] == 'aSFL')]
fig, ax = D2Plot(GlobalTable_meca, XCol='SurroundingThickness',YCol='EChadwick',CondCol = ['substrate','drug'],\
           Filters=Filters, cellID = 'CellName', AvgPerCell=True, modelFit=False, modelType='y=A*exp(kx)', yscale = 'log')
fig.suptitle('3T3aSFL: E(h)')
ax.legend(loc='upper right')
archiveFig(fig, ax, name='aSFL_E(h)_drug&substrate', figDir = todayFigDir, figSubDir='')
fig.show()

In [ ]:
# xticksTextObject = ax.get_xticklabels()
# xticksList = [xticksTextObject[j].get_text() for j in range(len(xticksTextObject))]
# newXticksList = [renameDict1.get(k, k) for k in xticksList]
# newXticksList

In [ ]:
Filters = [(GlobalTable_meca_Py['validatedFit'] == True), (GlobalTable_meca_Py['validatedThickness'] == True), (GlobalTable_meca_Py['substrate'] == '20um fibronectin discs')]
co_order = makeOrder(['aSFL','aSFL-A8','aSFL-6FP'], ['none','doxycyclin'])
box_pairs=[('aSFL & none', 'aSFL & doxycyclin'),
 ('aSFL-A8 & none', 'aSFL-A8 & doxycyclin'),
 ('aSFL-6FP & none', 'aSFL-6FP & doxycyclin'),
 ('aSFL & none', 'aSFL-A8 & none'),
 ('aSFL & none', 'aSFL-6FP & none')]
fig, ax = D1Plot(GlobalTable_meca_Py, CondCol=['cell subtype','drug'],Parameters=['surroundingThickness','EChadwick'],\
                 Filters=Filters,AvgPerCell=False,cellID='cellID',co_order=co_order,box_pairs=box_pairs,stats=True)
renameAxes(ax,renameDict1)
fig.suptitle('3T3aSFL - All linker types: All Compressions')
fig.show()

#### Big vs Small Beads, new code

##### Oct-2021

In [ ]:
figSubDir = 'BigVsSmallPlots'

In [ ]:
GlobalTable_meca_Py2.tail()

In [ ]:
Filters = [(GlobalTable_meca_Py2['validatedFit'] == True), 
           (GlobalTable_meca_Py2['validatedThickness'] == True), 
           (GlobalTable_meca_Py2['substrate'] == '20um fibronectin discs'),
           (GlobalTable_meca_Py2['date_x'].apply(lambda x : x in ['21-10-18', '21-10-25']))]
co_order = makeOrder(['M270','M450'])

fig, ax = D1Plot(GlobalTable_meca_Py2, CondCol=['bead type'],Parameters=['ctFieldThickness','EChadwick_f<150pN'],\
                 Filters=Filters,AvgPerCell=True,cellID='cellID',co_order=co_order,stats=True, useHue = True)
renameAxes(ax,renameDict1)
fig.suptitle('3T3aSFL - Oct 21 experiment, bead types')
# archiveFig(fig, ax, name='3T3aSFL_Oct21_M450-1-10_vs_M270-5-40', figSubDir = figSubDir)
fig.show()

In [ ]:
Filters = [(GlobalTable_meca_Py2['validatedFit'] == True), 
           (GlobalTable_meca_Py2['validatedThickness'] == True), 
           (GlobalTable_meca_Py2['substrate'] == '20um fibronectin discs'),
           (GlobalTable_meca_Py2['date_x'].apply(lambda x : x in ['21-10-18', '21-10-25'])),
           (GlobalTable_meca_Py2['ctFieldThickness'] < 400),
          (GlobalTable_meca_Py2['ctFieldThickness'] > 200)]
co_order = makeOrder(['M270','M450'])

fig, ax = D1Plot(GlobalTable_meca_Py2, CondCol=['bead type'],Parameters=['ctFieldThickness','EChadwick_f<150pN'],\
                 Filters=Filters,AvgPerCell=True,cellID='cellID',co_order=co_order,stats=True, useHue = True)
renameAxes(ax,renameDict1)
fig.suptitle('3T3aSFL - Oct 21 experiment, bead types, 200-400nm')
# archiveFig(fig, ax, name='3T3aSFL_Oct21_M450-1-10_vs_M270-5-40_h=200-400nm', figSubDir = figSubDir)
fig.show()

In [ ]:
Filters = [(GlobalTable_meca_Py2['validatedFit'] == True), 
           (GlobalTable_meca_Py2['validatedThickness'] == True), 
           (GlobalTable_meca_Py2['substrate'] == '20um fibronectin discs'),
           (GlobalTable_meca_Py2['date_x'].apply(lambda x : x in ['21-10-18', '21-10-25'])),
           (GlobalTable_meca_Py2['ctFieldThickness'] < 600),
          (GlobalTable_meca_Py2['ctFieldThickness'] > 400)]
co_order = makeOrder(['M270','M450'])

fig, ax = D1Plot(GlobalTable_meca_Py2, CondCol=['bead type'],Parameters=['ctFieldThickness','EChadwick_f<150pN'],\
                 Filters=Filters,AvgPerCell=True,cellID='cellID',co_order=co_order,stats=True, useHue = True)
renameAxes(ax,renameDict1)
fig.suptitle('3T3aSFL - Oct 21 experiment, bead types, 400-600nm')
archiveFig(fig, ax, name='3T3aSFL_Oct21_M450-1-10_vs_M270-5-40_h=400-600nm', figSubDir = figSubDir)
fig.show()

In [ ]:
Filters = [(GlobalTable_meca_Py2['validatedFit'] == True), 
           (GlobalTable_meca_Py2['validatedThickness'] == True), 
           (GlobalTable_meca_Py2['substrate'] == '20um fibronectin discs'),
           (GlobalTable_meca_Py2['date_x'].apply(lambda x : x in ['21-10-18', '21-10-25'])),
           (GlobalTable_meca_Py2['ctFieldThickness'] < 1000),
          (GlobalTable_meca_Py2['ctFieldThickness'] > 600)]
co_order = makeOrder(['M270','M450'])

fig, ax = D1Plot(GlobalTable_meca_Py2, CondCol=['bead type'],Parameters=['ctFieldThickness','EChadwick_f<150pN'],\
                 Filters=Filters,AvgPerCell=True,cellID='cellID',co_order=co_order,stats=True, useHue = True)
renameAxes(ax,renameDict1)
fig.suptitle('3T3aSFL - Oct 21 experiment, bead types, 600-1000nm')
# archiveFig(fig, ax, name='3T3aSFL_Oct21_M450-1-10_vs_M270-5-40_h=600-1000nm', figSubDir = figSubDir)
fig.show()

In [ ]:
# GlobalTable_meca_Py2

In [ ]:
Filters = [(GlobalTable_meca_Py2['validatedFit'] == True), 
           (GlobalTable_meca_Py2['validatedThickness'] == True), 
           (GlobalTable_meca_Py2['substrate'] == '20um fibronectin discs'),
           (GlobalTable_meca_Py2['date_x'].apply(lambda x : x in ['21-10-18', '21-10-25']))]
fig, ax = D2Plot(GlobalTable_meca_Py2, XCol='ctFieldThickness',YCol='EChadwick_f<150pN',CondCol = ['bead type'],\
           Filters=Filters, cellID = 'cellID', AvgPerCell=True, modelFit=False, modelType='y=A*exp(kx)',xscale = 'log', yscale = 'log')
fig.suptitle('3T3aSFL: E(h)')
# archiveFig(fig, ax, name='E(h)_3T3aSFL_Oct21_M450-1-10_vs_M270-5-40', figDir = todayFigDir, figSubDir=figSubDir)
fig.show()

In [ ]:
Filters = [(GlobalTable_meca_Py2['validatedFit'] == True), 
           (GlobalTable_meca_Py2['validatedThickness'] == True), 
           (GlobalTable_meca_Py2['substrate'] == '20um fibronectin discs'),
           (GlobalTable_meca_Py2['date_x'].apply(lambda x : x in ['21-10-18', '21-10-25']))]
fig, ax = D2Plot(GlobalTable_meca_Py2, XCol='surroundingDz',YCol='surroundingDx',CondCol = ['bead type'],\
           Filters=Filters, cellID = 'cellID', AvgPerCell=False, modelFit=False, xscale = 'lin', yscale = 'lin')
fig.suptitle('3T3aSFL: dz(dx) - all compressions')
# archiveFig(fig, ax, name='dz(dx)_3T3aSFL_Oct21_M450-1-10_vs_M270-5-40', figDir = todayFigDir, figSubDir=figSubDir)
fig.show()

In [ ]:
Filters = [(GlobalTable_meca_Py2['validatedFit'] == True), 
           (GlobalTable_meca_Py2['validatedThickness'] == True), 
           (GlobalTable_meca_Py2['substrate'] == '20um fibronectin discs'),
           (GlobalTable_meca_Py2['date_x'].apply(lambda x : x in ['21-10-18', '21-10-25']))]
fig, ax = D2Plot(GlobalTable_meca_Py2, XCol='ctFieldDZ',YCol='ctFieldDX',CondCol = ['bead type'],\
           Filters=Filters, cellID = 'cellID', AvgPerCell=True, modelFit=False, xscale = 'lin', yscale = 'lin')
fig.suptitle('3T3aSFL: dz(dx) - median per cell')
# archiveFig(fig, ax, name='DZ(DX)_3T3aSFL_Oct21_M450-1-10_vs_M270-5-40', figDir = todayFigDir, figSubDir=figSubDir)
fig.show()

##### Compute nice field range matching stress-wise

In [ ]:
def computeMag_M270(B):
    M = 0.74257*1.05*1600 * (0.001991*B**3 + 17.54*B**2 + 153.4*B) / (B**2 + 35.53*B + 158.1)
    return(M)

def computeMag_M450(B):
    M = 1.05*1600 * (0.001991*B**3 + 17.54*B**2 + 153.4*B) / (B**2 + 35.53*B + 158.1)
    return(M)

h0 = 500
B_M270 = 50
B_M450 = 8.5
anglefactor = 2
V_M270 = (4/3)*np.pi*(2690/2)**3 # volume [nm^3]
V_M450 = (4/3)*np.pi*(4503/2)**3 # volume [nm^3]
m_M270 = computeMag_M270(B_M270) * 1e-9 * V_M270
m_M450 = computeMag_M450(B_M450) * 1e-9 * V_M450
D3nm_270 = np.arange(1, h0, 1, dtype = float) + 2690
D3nm_450 = np.arange(1, h0, 1, dtype = float) + 4503
F_270 = 3e5*anglefactor*m_M270**2/D3nm_270**4
F_450 = 3e5*anglefactor*m_M450**2/D3nm_450**4

s_270 = F_270[:-20]/((2690/2e6)*(h0+2690-D3nm_270[:-20]))
s_450 = F_450[:-20]/((4503/2e6)*(h0+4503-D3nm_450[:-20]))

fig, ax = plt.subplots(2,1, figsize = (8, 8))
ax[0].plot(D3nm_270-2690, F_270, 'c', label = 'M270 - B='+str(B_M270)+'mT')
ax[0].plot(D3nm_450-4503, F_450, 'r', label = 'M450 - B='+str(B_M450)+'mT')
ax[0].legend()
ax[0].set_ylabel('F (pN)')
ax[1].plot(D3nm_270[:-20]-2690, s_270, 'c', label = 'M270 - B='+str(B_M270)+'mT')
ax[1].plot(D3nm_450[:-20]-4503, s_450, 'r', label = 'M450 - B='+str(B_M450)+'mT')
# ax[1].plot(ax[1].get_xlim(), [60, 60], 'k--')
ax[1].legend()
ax[1].set_ylabel('sigma for h0=' + str(h0) + 'nm (Pa)')
fig.suptitle('Understanding the stress range mismatch\nTest', fontsize = 16)
#archiveFig(fig, ax, name='Mismatch_3T3aSFL_Test2', figDir = todayFigDir, figSubDir=figSubDir)
fig.show()

In [ ]:
def computeMag_M270(B):
    M = 0.74257*1.05*1600 * (0.001991*B**3 + 17.54*B**2 + 153.4*B) / (B**2 + 35.53*B + 158.1)
    return(M)

def computeMag_M450(B):
    M = 1.05*1600 * (0.001991*B**3 + 17.54*B**2 + 153.4*B) / (B**2 + 35.53*B + 158.1)
    return(M)

h0 = 500
B_M270 = 5
B_M450 = 5
anglefactor = 2
V_M270 = (4/3)*np.pi*(2690/2)**3 # volume [nm^3]
V_M450 = (4/3)*np.pi*(4503/2)**3 # volume [nm^3]
m_M270 = computeMag_M270(B_M270) * 1e-9 * V_M270
m_M450 = computeMag_M450(B_M450) * 1e-9 * V_M450
D3nm_270 = np.arange(1, h0, 1, dtype = float) + 2690
D3nm_450 = np.arange(1, h0, 1, dtype = float) + 4503
F_270 = 3e5*anglefactor*m_M270**2/D3nm_270**4
F_450 = 3e5*anglefactor*m_M450**2/D3nm_450**4

s_270 = F_270[:-20]/((2690/2e6)*(h0+2690-D3nm_270[:-20]))
s_450 = F_450[:-20]/((4503/2e6)*(h0+4503-D3nm_450[:-20]))

fig, ax = plt.subplots(2,1, figsize = (8, 8))
ax[0].plot(D3nm_270-2690, F_270, 'c', label = 'M270 - B='+str(B_M270)+'mT')
ax[0].plot(D3nm_450-4503, F_450, 'r', label = 'M450 - B='+str(B_M450)+'mT')
ax[0].legend()
ax[0].set_ylabel('F (pN)')
ax[1].plot(D3nm_270[:-20]-2690, s_270, 'c', label = 'M270 - B='+str(B_M270)+'mT')
ax[1].plot(D3nm_450[:-20]-4503, s_450, 'r', label = 'M450 - B='+str(B_M450)+'mT')
ax[1].legend()
ax[1].set_ylabel('sigma for h0=' + str(h0) + 'nm (Pa)')
fig.suptitle('Understanding the stress range mismatch\nValue used for standard experiments : 5mT', fontsize = 16)
#archiveFig(fig, ax, name='Mismatch_3T3aSFL_Oct21_M450_5mT-vs-M270_5mT', figDir = todayFigDir, figSubDir=figSubDir)
fig.show()

In [ ]:
def computeMag_M270(B):
    M = 0.74257*1.05*1600 * (0.001991*B**3 + 17.54*B**2 + 153.4*B) / (B**2 + 35.53*B + 158.1)
    return(M)

def computeMag_M450(B):
    M = 1.05*1600 * (0.001991*B**3 + 17.54*B**2 + 153.4*B) / (B**2 + 35.53*B + 158.1)
    return(M)

h0 = 500
B_M270 = 5
B_M450 = 1
anglefactor = 2
V_M270 = (4/3)*np.pi*(2690/2)**3 # volume [nm^3]
V_M450 = (4/3)*np.pi*(4503/2)**3 # volume [nm^3]
m_M270 = computeMag_M270(B_M270) * 1e-9 * V_M270
m_M450 = computeMag_M450(B_M450) * 1e-9 * V_M450
D3nm_270 = np.arange(1, h0, 1, dtype = float) + 2690
D3nm_450 = np.arange(1, h0, 1, dtype = float) + 4503
F_270 = 3e5*anglefactor*m_M270**2/D3nm_270**4
F_450 = 3e5*anglefactor*m_M450**2/D3nm_450**4

s_270 = F_270[:-20]/((2690/2e6)*(h0+2690-D3nm_270[:-20]))
s_450 = F_450[:-20]/((4503/2e6)*(h0+4503-D3nm_450[:-20]))

fig, ax = plt.subplots(2,1, figsize = (8, 8))
ax[0].plot(D3nm_270-2690, F_270, 'c', label = 'M270 - B='+str(B_M270)+'mT')
ax[0].plot(D3nm_450-4503, F_450, 'r', label = 'M450 - B='+str(B_M450)+'mT')
ax[0].legend()
ax[0].set_ylabel('F (pN)')
ax[1].plot(D3nm_270[:-20]-2690, s_270, 'c', label = 'M270 - B='+str(B_M270)+'mT')
ax[1].plot(D3nm_450[:-20]-4503, s_450, 'r', label = 'M450 - B='+str(B_M450)+'mT')
# ax[1].plot(ax[1].get_xlim(), [60, 60], 'k--')
ax[1].legend()
ax[1].set_ylabel('sigma for h0=' + str(h0) + 'nm (Pa)')
fig.suptitle('Understanding the stress range mismatch\nValues used for october 2021 experiment', fontsize = 16)
#archiveFig(fig, ax, name='Mismatch_3T3aSFL_Oct21_M450_1mT-vs-M270_5mT', figDir = todayFigDir, figSubDir=figSubDir)
fig.show()

In [ ]:
def computeMag_M270(B):
    M = 0.74257*1.05*1600 * (0.001991*B**3 + 17.54*B**2 + 153.4*B) / (B**2 + 35.53*B + 158.1)
    return(M)

def computeMag_M450(B):
    M = 1.05*1600 * (0.001991*B**3 + 17.54*B**2 + 153.4*B) / (B**2 + 35.53*B + 158.1)
    return(M)

h0 = 500
B_M270 = 14
B_M450 = 5
anglefactor = 2
V_M270 = (4/3)*np.pi*(2690/2)**3 # volume [nm^3]
V_M450 = (4/3)*np.pi*(4503/2)**3 # volume [nm^3]
m_M270 = computeMag_M270(B_M270) * 1e-9 * V_M270
m_M450 = computeMag_M450(B_M450) * 1e-9 * V_M450
D3nm_270 = np.arange(1, h0, 1, dtype = float) + 2690
D3nm_450 = np.arange(1, h0, 1, dtype = float) + 4503
F_270 = 3e5*anglefactor*m_M270**2/D3nm_270**4
F_450 = 3e5*anglefactor*m_M450**2/D3nm_450**4

s_270 = F_270[:-20]/((2690/2e6)*(h0+2690-D3nm_270[:-20]))
s_450 = F_450[:-20]/((4503/2e6)*(h0+4503-D3nm_450[:-20]))

fig, ax = plt.subplots(2,1, figsize = (8, 8))
ax[0].plot(D3nm_270-2690, F_270, 'c', label = 'M270 - B='+str(B_M270)+'mT')
ax[0].plot(D3nm_450-4503, F_450, 'r', label = 'M450 - B='+str(B_M450)+'mT')
ax[0].legend()
ax[0].set_ylabel('F (pN)')
ax[1].plot(D3nm_270[:-20]-2690, s_270, 'c', label = 'M270 - B='+str(B_M270)+'mT')
ax[1].plot(D3nm_450[:-20]-4503, s_450, 'r', label = 'M450 - B='+str(B_M450)+'mT')
ax[1].legend()
ax[1].set_ylabel('sigma for h0=' + str(h0) + 'nm (Pa)')
fig.suptitle('Understanding the stress range mismatch\nProposition of values for a future experiment', fontsize = 16)
# archiveFig(fig, ax, name='Mismatch_3T3aSFL_Oct21_M450_5mT-vs-M270_14mT', figDir = todayFigDir, figSubDir=figSubDir)
fig.show()

In [ ]:
def computeMag_M270(B):
    M = 0.74257*1.05*1600 * (0.001991*B**3 + 17.54*B**2 + 153.4*B) / (B**2 + 35.53*B + 158.1)
    return(M)

def computeMag_M450(B):
    M = 1.05*1600 * (0.001991*B**3 + 17.54*B**2 + 153.4*B) / (B**2 + 35.53*B + 158.1)
    return(M)

h0 = 500
B_M270 = 54
B_M450 = 13
anglefactor = 2
V_M270 = (4/3)*np.pi*(2690/2)**3 # volume [nm^3]
V_M450 = (4/3)*np.pi*(4503/2)**3 # volume [nm^3]
m_M270 = computeMag_M270(B_M270) * 1e-9 * V_M270
m_M450 = computeMag_M450(B_M450) * 1e-9 * V_M450
D3nm_270 = np.arange(1, h0, 1, dtype = float) + 2690
D3nm_450 = np.arange(1, h0, 1, dtype = float) + 4503
F_270 = 3e5*anglefactor*m_M270**2/D3nm_270**4
F_450 = 3e5*anglefactor*m_M450**2/D3nm_450**4

s_270 = F_270[:-20]/((2690/2e6)*(h0+2690-D3nm_270[:-20]))
s_450 = F_450[:-20]/((4503/2e6)*(h0+4503-D3nm_450[:-20]))

fig, ax = plt.subplots(2,1, figsize = (8, 8))
ax[0].plot(D3nm_270-2690, F_270, 'c', label = 'M270 - B='+str(B_M270)+'mT')
ax[0].plot(D3nm_450-4503, F_450, 'r', label = 'M450 - B='+str(B_M450)+'mT')
ax[0].legend()
ax[0].set_ylabel('F (pN)')
ax[1].plot(D3nm_270[:-20]-2690, s_270, 'c', label = 'M270 - B='+str(B_M270)+'mT')
ax[1].plot(D3nm_450[:-20]-4503, s_450, 'r', label = 'M450 - B='+str(B_M450)+'mT')
ax[1].legend()
ax[1].set_ylabel('sigma for h0=' + str(h0) + 'nm (Pa)')
fig.suptitle('Understanding the stress range mismatch\nProposition of values for a future experiment', fontsize = 16)
# archiveFig(fig, ax, name='Mismatch_3T3aSFL_Oct21_M450_13mT-vs-M270_54mT', figDir = todayFigDir, figSubDir=figSubDir)
fig.show()

In [ ]:
def computeMag_M270(B):
    M = 0.74257*1.05*1600 * (0.001991*B**3 + 17.54*B**2 + 153.4*B) / (B**2 + 35.53*B + 158.1)
    return(M)

def computeMag_M450(B):
    M = 1.05*1600 * (0.001991*B**3 + 17.54*B**2 + 153.4*B) / (B**2 + 35.53*B + 158.1)
    return(M)

h0 = 500
B_M270 = 2.1
B_M450 = 1
anglefactor = 2
V_M270 = (4/3)*np.pi*(2690/2)**3 # volume [nm^3]
V_M450 = (4/3)*np.pi*(4503/2)**3 # volume [nm^3]
m_M270 = computeMag_M270(B_M270) * 1e-9 * V_M270
m_M450 = computeMag_M450(B_M450) * 1e-9 * V_M450
D3nm_270 = np.arange(1, h0, 1, dtype = float) + 2690
D3nm_450 = np.arange(1, h0, 1, dtype = float) + 4503
F_270 = 3e5*anglefactor*m_M270**2/D3nm_270**4
F_450 = 3e5*anglefactor*m_M450**2/D3nm_450**4

s_270 = F_270[:-20]/((2690/2e6)*(h0+2690-D3nm_270[:-20]))
s_450 = F_450[:-20]/((4503/2e6)*(h0+4503-D3nm_450[:-20]))

fig, ax = plt.subplots(2,1, figsize = (8, 8))
ax[0].plot(D3nm_270-2690, F_270, 'c', label = 'M270 - B='+str(B_M270)+'mT')
ax[0].plot(D3nm_450-4503, F_450, 'r', label = 'M450 - B='+str(B_M450)+'mT')
ax[0].legend()
ax[0].set_ylabel('F (pN)')
ax[1].plot(D3nm_270[:-20]-2690, s_270, 'c', label = 'M270 - B='+str(B_M270)+'mT')
ax[1].plot(D3nm_450[:-20]-4503, s_450, 'r', label = 'M450 - B='+str(B_M450)+'mT')
ax[1].legend()
ax[1].set_ylabel('sigma for h0=' + str(h0) + 'nm (Pa)')
fig.suptitle('Understanding the stress range mismatch\nProposition of values for a future experiment', fontsize = 16)
# archiveFig(fig, ax, name='Mismatch_3T3aSFL_Oct21_M450_13mT-vs-M270_54mT', figDir = todayFigDir, figSubDir=figSubDir)
fig.show()

In [ ]:
Filters270 = [(GlobalTable_meca_Py2['validatedFit'] == True), 
           (GlobalTable_meca_Py2['validatedThickness'] == True),\
           (GlobalTable_meca_Py2['cell subtype'] == 'aSFL'), 
           (GlobalTable_meca_Py2['bead type'] == 'M270'), 
           (GlobalTable_meca_Py2['substrate'] == '20um fibronectin discs'),
           (GlobalTable_meca_Py2['date_x'].apply(lambda x : x in ['21-10-18', '21-10-25']))]
Filter270 = Filters270[0]
for i in range(1, len(Filters270)):
    Filter270 = Filter270 & Filters270[i]
    
Filters450 = [(GlobalTable_meca_Py2['validatedFit'] == True), 
           (GlobalTable_meca_Py2['validatedThickness'] == True),\
           (GlobalTable_meca_Py2['cell subtype'] == 'aSFL'), 
           (GlobalTable_meca_Py2['bead type'] == 'M450'), 
           (GlobalTable_meca_Py2['substrate'] == '20um fibronectin discs'),
           (GlobalTable_meca_Py2['date_x'].apply(lambda x : x in ['21-10-18', '21-10-25']))]   
Filter450 = Filters450[0]
for i in range(1, len(Filters450)):
    Filter450 = Filter450 & Filters450[i]
    

fig, ax = plt.subplots(1,2, figsize = (8, 6))
GlobalTable_meca_Py2[Filter270].plot(kind = 'scatter', ax = ax[0], x = 'surroundingThickness', y = 'minS', marker = 'o', color = 'c', alpha = 0.3, label = 'Min stress')
GlobalTable_meca_Py2[Filter270].plot(kind = 'scatter', ax = ax[0], x = 'surroundingThickness', y = 'maxS', marker = 'o', color = 'r', alpha = 0.3, label = 'Max stress')
GlobalTable_meca_Py2[Filter450].plot(kind = 'scatter', ax = ax[1], x = 'surroundingThickness', y = 'minS', marker = 'o', color = 'c', alpha = 0.3, label = 'Min stress')
GlobalTable_meca_Py2[Filter450].plot(kind = 'scatter', ax = ax[1], x = 'surroundingThickness', y = 'maxS', marker = 'o', color = 'r', alpha = 0.3, label = 'Max stress')
ax[0].set_xlabel('Cortical thickness (nm)')
ax[0].set_ylabel('Extremal stress values (Pa)')
ax[0].set_ylim([0, 800])
ax[0].set_xlim([0, 1200])
ax[0].set_title('M270')
ax[1].set_xlabel('Cortical thickness (nm)')
ax[1].set_ylabel('Extremal stress values (Pa)')
ax[1].set_ylim([0, 800])
ax[1].set_xlim([0, 1200])
ax[1].set_title('M450')
fig.show()

In [ ]:
fig, ax = plt.subplots(1,1, figsize = (8, 6))
GlobalTable_meca_Py2[Filter270].plot(kind = 'scatter', ax = ax, x = 'surroundingThickness', y = 'minS', marker = 'o', color = 'c', alpha = 0.3, label = 'M270 - Min stress')
GlobalTable_meca_Py2[Filter270].plot(kind = 'scatter', ax = ax, x = 'surroundingThickness', y = 'maxS', marker = 'o', color = 'orange', alpha = 0.3, label = 'M270 - Max stress')
GlobalTable_meca_Py2[Filter450].plot(kind = 'scatter', ax = ax, x = 'surroundingThickness', y = 'minS', marker = 'o', color = 'b', alpha = 0.3, label = 'M450 - Min stress')
GlobalTable_meca_Py2[Filter450].plot(kind = 'scatter', ax = ax, x = 'surroundingThickness', y = 'maxS', marker = 'o', color = 'r', alpha = 0.3, label = 'M450 - Max stress')
ax.set_xlabel('Cortical thickness (nm)')
ax.set_ylabel('Extremal stress values (Pa)')
ax.set_ylim([0, 1000])
ax.set_xlim([0, 1200])
fig.show()

##### Dec-2021

In [ ]:
figSubDir = 'BigVsSmallPlots'

In [ ]:
GlobalTable_meca_Py2.head()

In [ ]:
Filters = [(GlobalTable_meca_Py2['validatedFit'] == True), 
           (GlobalTable_meca_Py2['validatedThickness'] == True), 
           (GlobalTable_meca_Py2['substrate'] == '20um fibronectin discs'),
           (GlobalTable_meca_Py2['date'].apply(lambda x : x in ['21-12-08']))]
co_order = makeOrder(['M270','M450'])

fig, ax = D1Plot(GlobalTable_meca_Py2, CondCol=['bead type'],Parameters=['ctFieldThickness','EChadwick_f<150pN'],\
                 Filters=Filters,AvgPerCell=True,cellID='cellID',co_order=co_order,stats=True, useHue = True)
renameAxes(ax,renameDict1)
fig.suptitle('3T3aSFL - Dec 21 experiment, bead types')
# archiveFig(fig, ax, name='3T3aSFL_Oct21_M450-1-10_vs_M270-5-40', figSubDir = figSubDir)
fig.show()

In [ ]:
Filters = [(GlobalTable_meca_Py2['validatedFit'] == True), 
           (GlobalTable_meca_Py2['validatedThickness'] == True), 
           (GlobalTable_meca_Py2['substrate'] == '20um fibronectin discs'),
           (GlobalTable_meca_Py2['date'].apply(lambda x : x in ['21-12-16']))]
co_order = makeOrder(['M270','M450'])

fig, ax = D1Plot(GlobalTable_meca_Py2, CondCol=['bead type'],Parameters=['ctFieldThickness','EChadwick'],\
                 Filters=Filters,AvgPerCell=True,cellID='cellID',co_order=co_order,stats=True, useHue = True)
renameAxes(ax,renameDict1)
fig.suptitle('3T3aSFL - Dec 21 experiment, bead types')
# archiveFig(fig, ax, name='3T3aSFL_Oct21_M450-1-10_vs_M270-5-40', figSubDir = figSubDir)
fig.show()

In [ ]:
Filters = [(GlobalTable_meca_Py2['validatedFit'] == True), 
           (GlobalTable_meca_Py2['validatedThickness'] == True), 
           (GlobalTable_meca_Py2['substrate'] == '20um fibronectin discs'),
           (GlobalTable_meca_Py2['date'].apply(lambda x : x in ['21-12-08']))]

co_order = makeOrder(['M270','M450'])

fig, ax = D1Plot(GlobalTable_meca_Py2, CondCol=['bead type'],Parameters=['ctFieldThickness','EChadwick'],\
                 Filters=Filters,AvgPerCell=True,cellID='cellID',co_order=co_order,stats=True, useHue = True)
renameAxes(ax,renameDict1)
fig.suptitle('3T3aSFL, Dec 21 - Matching ranges of stress for 2 bead types\nB{M270}=14>54mT ; B{M450}=5>13mT')
# archiveFig(fig, ax, name='3T3aSFL_Oct21_M450-1-10_vs_M270-5-40', figSubDir = figSubDir)
fig.show()

In [ ]:
Filters = [(GlobalTable_meca_Py2['validatedFit'] == True), 
           (GlobalTable_meca_Py2['validatedThickness'] == True), 
           (GlobalTable_meca_Py2['substrate'] == '20um fibronectin discs'),
           (GlobalTable_meca_Py2['date'].apply(lambda x : x in ['21-12-08'])),
           (GlobalTable_meca_Py2['ctFieldThickness'] < 400),
           (GlobalTable_meca_Py2['ctFieldThickness'] > 150)]

co_order = makeOrder(['M270','M450'])

fig, ax = D1Plot(GlobalTable_meca_Py2, CondCol=['bead type'],Parameters=['ctFieldThickness','EChadwick'],\
                 Filters=Filters,AvgPerCell=True,cellID='cellID',co_order=co_order,stats=True, useHue = True)
renameAxes(ax,renameDict1)
fig.suptitle('3T3aSFL, Dec 21 - Matching ranges of stress for 2 bead types\nB{M270}=14>54mT ; B{M450}=5>13mT')
# archiveFig(fig, ax, name='3T3aSFL_Oct21_M450-1-10_vs_M270-5-40', figSubDir = figSubDir)
fig.show()

In [ ]:
Filters = [(GlobalTable_meca_Py2['validatedFit'] == True), 
           (GlobalTable_meca_Py2['validatedThickness'] == True), 
           (GlobalTable_meca_Py2['substrate'] == '20um fibronectin discs'),
           (GlobalTable_meca_Py2['date'].apply(lambda x : x in ['21-12-08'])),
           (GlobalTable_meca_Py2['ctFieldThickness'] < 600),
           (GlobalTable_meca_Py2['ctFieldThickness'] > 400)]

co_order = makeOrder(['M270','M450'])

fig, ax = D1Plot(GlobalTable_meca_Py2, CondCol=['bead type'],Parameters=['ctFieldThickness','EChadwick'],\
                 Filters=Filters,AvgPerCell=True,cellID='cellID',co_order=co_order,stats=True, useHue = True)
renameAxes(ax,renameDict1)
fig.suptitle('3T3aSFL, Dec 21 - Matching ranges of stress for 2 bead types\nB{M270}=14>54mT ; B{M450}=5>13mT')
# archiveFig(fig, ax, name='3T3aSFL_Oct21_M450-1-10_vs_M270-5-40', figSubDir = figSubDir)
fig.show()

In [ ]:
Filters = [(GlobalTable_meca_Py2['validatedFit'] == True), 
           (GlobalTable_meca_Py2['validatedThickness'] == True), 
           (GlobalTable_meca_Py2['substrate'] == '20um fibronectin discs'),
           (GlobalTable_meca_Py2['date'].apply(lambda x : x in ['21-12-08']))]
fig, ax = D2Plot(GlobalTable_meca_Py2, XCol='ctFieldThickness',YCol='EChadwick',CondCol = ['bead type'],\
           Filters=Filters, cellID = 'cellID', AvgPerCell=True, xscale = 'log', yscale = 'log')
ax.set_ylabel('EChadwick (Pa)')
ax.set_xlabel('Thickness at low force (nm)')
fig.suptitle('3T3aSFL: E(h)')
# archiveFig(fig, ax, name='E(h)_3T3aSFL_Dec21_M450-5-13_vs_M270-14-54', figDir = todayFigDir, figSubDir=figSubDir)
fig.show()

In [ ]:
Filters = [(GlobalTable_meca_Py2['validatedFit'] == True), 
           (GlobalTable_meca_Py2['validatedThickness'] == True), 
           (GlobalTable_meca_Py2['substrate'] == '20um fibronectin discs'),
           (GlobalTable_meca_Py2['date'].apply(lambda x : x in ['21-12-16']))]
fig, ax = D2Plot(GlobalTable_meca_Py2, XCol='ctFieldThickness',YCol='EChadwick',CondCol = ['bead type'],\
           Filters=Filters, cellID = 'cellID', AvgPerCell=True,xscale = 'log', yscale = 'log')
ax.set_ylabel('EChadwick (Pa)')
ax.set_xlabel('Thickness at low force (nm)')
fig.suptitle('3T3aSFL: E(h)')
# archiveFig(fig, ax, name='E(h)_3T3aSFL_Dec21_M450-5-13_vs_M270-14-54', figDir = todayFigDir, figSubDir=figSubDir)
# fig.show()

In [ ]:
Filters = [(GlobalTable_meca_Py2['validatedFit'] == True), 
           (GlobalTable_meca_Py2['validatedThickness'] == True), 
           (GlobalTable_meca_Py2['substrate'] == '20um fibronectin discs'),
           (GlobalTable_meca_Py2['date'].apply(lambda x : x in ['21-12-08','21-12-16']))]

co_order = makeOrder(['M270','M450'], ['21-12-08','21-12-16'])

fig, ax = D1Plot(GlobalTable_meca_Py2, CondCol=['bead type', 'date'],Parameters=['ctFieldThickness','EChadwick'],\
                 Filters=Filters,AvgPerCell=True,cellID='cellID',co_order=co_order,stats=True, useHue = True, orientation = 'v',
                 figSizeFactor = 0.9)
renameAxes(ax,renameDict1)
fig.suptitle('3T3aSFL, Dec 21 - Matching ranges of stress for 2 bead types\nB{M270}=14>54mT ; B{M450}=5>13mT')
# archiveFig(fig, ax, name='3T3aSFL_Oct21_M450-1-10_vs_M270-5-40', figSubDir = figSubDir)
# fig.show()

In [19]:
Filters = [(GlobalTable_meca_Py2['validatedFit'] == True), 
           (GlobalTable_meca_Py2['validatedThickness'] == True), 
           (GlobalTable_meca_Py2['substrate'] == '20um fibronectin discs'),
           (GlobalTable_meca_Py2['date'].apply(lambda x : x in ['21-12-16','21-12-08']))]
fig, ax = D2Plot(GlobalTable_meca_Py2, XCol='ctFieldThickness',YCol='EChadwick',CondCol = ['bead type', 'date'],\
           Filters=Filters, cellID = 'cellID', AvgPerCell=True, xscale = 'log', yscale = 'log', markers = ['o', 'o', '>', '>'])
ax.set_ylabel('EChadwick (Pa)')
ax.set_xlabel('Thickness at low force (nm)')
fig.suptitle('3T3aSFL: E(h)')
ax.legend(loc = 'upper right')
# archiveFig(fig, ax, name='E(h)_3T3aSFL_Dec21_M450-5-13_vs_M270-14-54', figDir = todayFigDir, figSubDir=figSubDir)
# fig.show()

C:\Users\JosephVermeil\anaconda3\lib\site-packages\ipykernel_launcher.py:64: UserWarning: Attempted to set non-positive left xlim on a log-scaled axis.
Invalid limit will be ignored.


In [20]:
Filters = [(GlobalTable_meca_Py2['validatedFit'] == True), 
           (GlobalTable_meca_Py2['validatedThickness'] == True), 
           (GlobalTable_meca_Py2['substrate'] == '20um fibronectin discs'),
           (GlobalTable_meca_Py2['date'].apply(lambda x : x in ['21-12-16','21-12-08']))]
fig, ax = D2Plot_wFit(GlobalTable_meca_Py2, XCol='ctFieldThickness',YCol='EChadwick',CondCol = ['bead type'],
           Filters=Filters, cellID = 'cellID', AvgPerCell=True, xscale = 'log', yscale = 'log', 
           modelFit=True, modelType = 'y=k*x^a')
ax.set_ylabel('EChadwick (Pa)')
ax.set_xlabel('Thickness at low force (nm)')
fig.suptitle('3T3aSFL: E(h)')
ax.legend(loc = 'upper right')
# archiveFig(fig, ax, name='E(h)_3T3aSFL_Dec21_M450-5-13_vs_M270-14-54', figDir = todayFigDir, figSubDir=figSubDir)
fig.show()

TypeError: unhashable type: 'list'

In [ ]:
Filters270 = [(GlobalTable_meca_Py2['validatedFit'] == True), 
           (GlobalTable_meca_Py2['validatedThickness'] == True),\
           (GlobalTable_meca_Py2['cell subtype'] == 'aSFL'), 
           (GlobalTable_meca_Py2['bead type'] == 'M270'), 
           (GlobalTable_meca_Py2['substrate'] == '20um fibronectin discs'),
           (GlobalTable_meca_Py2['date'].apply(lambda x : x in ['21-12-08']))]
Filter270 = Filters270[0]
for i in range(1, len(Filters270)):
    Filter270 = Filter270 & Filters270[i]
    
Filters450 = [(GlobalTable_meca_Py2['validatedFit'] == True), 
           (GlobalTable_meca_Py2['validatedThickness'] == True),\
           (GlobalTable_meca_Py2['cell subtype'] == 'aSFL'), 
           (GlobalTable_meca_Py2['bead type'] == 'M450'), 
           (GlobalTable_meca_Py2['substrate'] == '20um fibronectin discs'),
           (GlobalTable_meca_Py2['date'].apply(lambda x : x in ['21-12-08']))]   
Filter450 = Filters450[0]
for i in range(1, len(Filters450)):
    Filter450 = Filter450 & Filters450[i]
    

fig, ax = plt.subplots(1,2, figsize = (9, 5))
GlobalTable_meca_Py2[Filter270].plot(kind = 'scatter', ax = ax[0], x = 'surroundingThickness', y = 'minStress', marker = 'o', color = 'c', alpha = 0.3, label = 'Min stress')
GlobalTable_meca_Py2[Filter270].plot(kind = 'scatter', ax = ax[0], x = 'surroundingThickness', y = 'maxStress', marker = 'o', color = 'r', alpha = 0.3, label = 'Max stress')
GlobalTable_meca_Py2[Filter450].plot(kind = 'scatter', ax = ax[1], x = 'surroundingThickness', y = 'minStress', marker = 'o', color = 'c', alpha = 0.3, label = 'Min stress')
GlobalTable_meca_Py2[Filter450].plot(kind = 'scatter', ax = ax[1], x = 'surroundingThickness', y = 'maxStress', marker = 'o', color = 'r', alpha = 0.3, label = 'Max stress')
ax[0].set_xlabel('Cortical thickness (nm)')
ax[0].set_ylabel('Extremal stress values (Pa)')
ax[0].set_ylim([0, 600])
ax[0].set_xlim([0, 1200])
ax[0].set_title('M270')
ax[1].set_xlabel('Cortical thickness (nm)')
ax[1].set_ylabel('Extremal stress values (Pa)')
ax[1].set_ylim([0, 600])
ax[1].set_xlim([0, 1200])
ax[1].set_title('M450')
# fig.show()

In [ ]:
fig, ax = plt.subplots(1,1, figsize = (9, 5))
GlobalTable_meca_Py2[Filter270].plot(kind = 'scatter', ax = ax, x = 'surroundingThickness', y = 'minStress', marker = 'o', color = 'c', alpha = 0.3, label = 'M270 - Min stress')
GlobalTable_meca_Py2[Filter270].plot(kind = 'scatter', ax = ax, x = 'surroundingThickness', y = 'maxStress', marker = 'o', color = 'orange', alpha = 0.3, label = 'M270 - Max stress')
GlobalTable_meca_Py2[Filter450].plot(kind = 'scatter', ax = ax, x = 'surroundingThickness', y = 'minStress', marker = 'o', color = 'b', alpha = 0.3, label = 'M450 - Min stress')
GlobalTable_meca_Py2[Filter450].plot(kind = 'scatter', ax = ax, x = 'surroundingThickness', y = 'maxStress', marker = 'o', color = 'r', alpha = 0.3, label = 'M450 - Max stress')
ax.set_xlabel('Cortical thickness (nm)')
ax.set_ylabel('Extremal stress values (Pa)')
ax.set_title('Dec 21 - Extremal stress values vs. thickness,\nfor each compression')
# fig.show()

In [ ]:
Filters270 = [(GlobalTable_meca_Py2['validatedFit'] == True), 
           (GlobalTable_meca_Py2['validatedThickness'] == True),\
           (GlobalTable_meca_Py2['cell subtype'] == 'aSFL'), 
           (GlobalTable_meca_Py2['bead type'] == 'M270'), 
           (GlobalTable_meca_Py2['substrate'] == '20um fibronectin discs'),
           (GlobalTable_meca_Py2['date'].apply(lambda x : x in ['21-12-16']))]
Filter270 = Filters270[0]
for i in range(1, len(Filters270)):
    Filter270 = Filter270 & Filters270[i]
    
Filters450 = [(GlobalTable_meca_Py2['validatedFit'] == True), 
           (GlobalTable_meca_Py2['validatedThickness'] == True),\
           (GlobalTable_meca_Py2['cell subtype'] == 'aSFL'), 
           (GlobalTable_meca_Py2['bead type'] == 'M450'), 
           (GlobalTable_meca_Py2['substrate'] == '20um fibronectin discs'),
           (GlobalTable_meca_Py2['date'].apply(lambda x : x in ['21-12-16']))]   
Filter450 = Filters450[0]
for i in range(1, len(Filters450)):
    Filter450 = Filter450 & Filters450[i]
    

fig, ax = plt.subplots(1,2, figsize = (9, 5))
GlobalTable_meca_Py2[Filter270].plot(kind = 'scatter', ax = ax[0], x = 'surroundingThickness', y = 'minStress', marker = 'o', color = 'c', alpha = 0.3, label = 'Min stress')
GlobalTable_meca_Py2[Filter270].plot(kind = 'scatter', ax = ax[0], x = 'surroundingThickness', y = 'maxStress', marker = 'o', color = 'r', alpha = 0.3, label = 'Max stress')
GlobalTable_meca_Py2[Filter450].plot(kind = 'scatter', ax = ax[1], x = 'surroundingThickness', y = 'minStress', marker = 'o', color = 'c', alpha = 0.3, label = 'Min stress')
GlobalTable_meca_Py2[Filter450].plot(kind = 'scatter', ax = ax[1], x = 'surroundingThickness', y = 'maxStress', marker = 'o', color = 'r', alpha = 0.3, label = 'Max stress')
ax[0].set_xlabel('Cortical thickness (nm)')
ax[0].set_ylabel('Extremal stress values (Pa)')
ax[0].set_ylim([0, 2000])
ax[0].set_xlim([0, 1200])
ax[0].set_title('M270')
ax[1].set_xlabel('Cortical thickness (nm)')
ax[1].set_ylabel('Extremal stress values (Pa)')
ax[1].set_ylim([0, 2000])
ax[1].set_xlim([0, 1200])
ax[1].set_title('M450')
# fig.show()

In [ ]:
fig, ax = plt.subplots(1,1, figsize = (9, 5))
GlobalTable_meca_Py2[Filter270].plot(kind = 'scatter', ax = ax, x = 'surroundingThickness', y = 'minStress', marker = 'o', color = 'c', alpha = 0.3, label = 'M270 - Min stress')
GlobalTable_meca_Py2[Filter270].plot(kind = 'scatter', ax = ax, x = 'surroundingThickness', y = 'maxStress', marker = 'o', color = 'orange', alpha = 0.3, label = 'M270 - Max stress')
GlobalTable_meca_Py2[Filter450].plot(kind = 'scatter', ax = ax, x = 'surroundingThickness', y = 'minStress', marker = 'o', color = 'b', alpha = 0.3, label = 'M450 - Min stress')
GlobalTable_meca_Py2[Filter450].plot(kind = 'scatter', ax = ax, x = 'surroundingThickness', y = 'maxStress', marker = 'o', color = 'r', alpha = 0.3, label = 'M450 - Max stress')
ax.set_xlabel('Cortical thickness (nm)')
ax.set_ylabel('Extremal stress values (Pa)')
ax.set_title('Dec 21 - Extremal stress values vs. thickness,\nfor each compression')
# fig.show()

##### Jan-2022

In [ ]:
figSubDir = 'BigVsSmallPlots'

In [ ]:
GlobalTable_meca_Py2.head()

In [ ]:
Filters = [(GlobalTable_meca_Py2['validatedFit'] == True), 
           (GlobalTable_meca_Py2['validatedThickness'] == True), 
           (GlobalTable_meca_Py2['substrate'] == '20um fibronectin discs'),
           (GlobalTable_meca_Py2['date'].apply(lambda x : x in ['21-12-08', '22-01-12']))]
co_order = makeOrder(['M270','M450'])

fig, ax = D1Plot(GlobalTable_meca_Py2, CondCol=['bead type'],Parameters=['ctFieldThickness','EChadwick'],\
                 Filters=Filters,AvgPerCell=True,cellID='cellID',co_order=co_order,stats=True, useHue = True)
renameAxes(ax,renameDict1)
fig.suptitle('3T3aSFL - Different bead diameters')
# archiveFig(fig, ax, name='3T3aSFL_Jan22_M450vsM270_CompressionsLowStart_Simple1DPlot', figSubDir = figSubDir)
fig.show()

In [ ]:
Filters = [(GlobalTable_meca_Py2['validatedFit'] == True), 
           (GlobalTable_meca_Py2['validatedThickness'] == True), 
           (GlobalTable_meca_Py2['substrate'] == '20um fibronectin discs'),
           (GlobalTable_meca_Py2['date'].apply(lambda x : x in ['22-01-12']))]
fig, ax = D2Plot(GlobalTable_meca_Py2, XCol='ctFieldThickness',YCol='EChadwick',CondCol = ['bead type', 'date'],\
           Filters=Filters, cellID = 'cellID', AvgPerCell=True, xscale = 'log', yscale = 'log', markers = ['o', 'o'])
ax.set_ylabel('EChadwick (Pa)')
ax.set_xlabel('Thickness at low force (nm)')
fig.suptitle('3T3aSFL: E(h)')
ax.legend(loc = 'upper right')
# archiveFig(fig, ax, name='3T3aSFL_Jan22_M450vsM270_CompressionsLowStart_E(h)', figDir = todayFigDir, figSubDir=figSubDir)
fig.show()

In [18]:
Filters = [(GlobalTable_meca_Py2['validatedFit'] == True), 
           (GlobalTable_meca_Py2['validatedThickness'] == True), 
           (GlobalTable_meca_Py2['substrate'] == '20um fibronectin discs'),
           (GlobalTable_meca_Py2['date'].apply(lambda x : x in ['21-12-08', '21-12-16', '22-01-12']))]
fig, ax = D2Plot(GlobalTable_meca_Py2, XCol='ctFieldThickness',YCol='EChadwick',CondCol = ['bead type', 'date'],\
           Filters=Filters, cellID = 'cellID', AvgPerCell=True, xscale = 'log', yscale = 'log', markers = ['o', 'o','v', 'v', 's', 's'])
ax.set_ylabel('EChadwick (Pa)')
ax.set_xlabel('Thickness at low force (nm)')
fig.suptitle('3T3aSFL: E(h)')
ax.legend(loc = 'upper right')
# archiveFig(fig, ax, name='3T3aSFL_Jan22_M450vsM270_CompressionsLowStart_E(h)_all3exp', figDir = todayFigDir, figSubDir=figSubDir)
fig.show()

C:\Users\JosephVermeil\anaconda3\lib\site-packages\ipykernel_launcher.py:64: UserWarning: Attempted to set non-positive left xlim on a log-scaled axis.
Invalid limit will be ignored.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
# Test getAggDf(df, cellID, CondCol, Variables)
data = GlobalTable_meca_Py2

Filters = [(data['validatedFit'] == True), (data['validatedThickness'] == True)]

data_f = data
for fltr in Filters:
    data_f = data_f.loc[fltr]

dfA = getAggDf(data_f, 'cellID', 'bead type', ['surroundingThickness', 'EChadwick'])
dfA.head()

In [ ]:
plt.close('all')
data = GlobalTable_meca_Py2

Filters = [(data['validatedFit'] == True), (data['validatedThickness'] == True),
          (data['date'].apply(lambda x : x in ['21-12-08', '22-01-12']))] #, '21-12-16'

fig, ax = D1PlotDetailed(data, CondCol=['bead type'], Parameters=['surroundingThickness', 'EChadwick'], Filters=Filters, 
                Boxplot=True, cellID='cellID', co_order=[], stats=True, statMethod='Mann-Whitney', 
               box_pairs=[], figSizeFactor = 1.8, markersizeFactor=1, orientation = 'v', showManips = True)
# archiveFig(fig, ax, name='3T3aSFL_Jan22_M450vsM270_CompressionsLowStart_Detailed1DPlot', figSubDir = figSubDir)
fig.show()

In [ ]:
plt.close('all')
data = GlobalTable_meca_Py2

Filters = [(data['validatedFit'] == True), (data['validatedThickness'] == True),
          (data['date'].apply(lambda x : x in ['21-12-08', '21-12-16', '22-01-12']))] #, '21-12-16'

box_pairs=[('21-12-08 & M270', '21-12-08 & M450'),
             ('21-12-16 & M450', '21-12-16 & M270'),
             ('22-01-12 & M270', '22-01-12 & M450')]

fig, ax = D1PlotDetailed(data, CondCol=['date', 'bead type'], Parameters=['surroundingThickness', 'EChadwick'], Filters=Filters, 
                Boxplot=True, cellID='cellID', co_order=[], stats=True, statMethod='Mann-Whitney', 
               box_pairs=box_pairs, figSizeFactor = 0.9, markersizeFactor=1, orientation = 'v', showManips = True)
# archiveFig(fig, ax, name='3T3aSFL_Jan22_M450vsM270_CompressionsLowStart_Detailed1DPlot_all3exp', figSubDir = figSubDir)
fig.show()

In [ ]:
data = GlobalTable_meca_Py2

Filters = [(data['validatedFit'] == True), (data['validatedThickness'] == True),
          (data['date'].apply(lambda x : x in ['21-12-08', '22-01-12']))] #, '21-12-16'

data_f = data
for fltr in Filters:
    data_f = data_f.loc[fltr]

rangeStress = np.arange(0, 2001)
countStress = [np.zeros(2001), np.zeros(2001)]

data_M270 = data_f.loc[data_f['bead type'] == 'M270']
data_M450 = data_f.loc[data_f['bead type'] == 'M450']

allData = [data_M270, data_M450]
text = ['M270', 'M450']

fig, ax = plt.subplots(1,1, figsize = (9, 4))


for i in range(2):
    df = allData[i]

    minS = df.minStress.values
    maxS = df.maxStress.values
    for k in range(len(minS)):
        countStress[i] += ((rangeStress > minS[k]) & (rangeStress < maxS[k]))
    
    ax.plot(rangeStress, countStress[i], label = text[i] + ' - total Ncomp = {:.0f}'.format(df.shape[0]))

ax.legend()
ax.set_xlim([0, 1000])
ax.set_xlabel('stress value (Pa)')
ax.set_ylabel('# compressions')
fig.suptitle('Number of compression which stress interval\ncontains a given stress value')
fig.show()

In [ ]:
data = GlobalTable_meca_Py2

Filters = [(data['validatedFit'] == True), (data['validatedThickness'] == True),
          (data['date'].apply(lambda x : x in ['21-12-08', '22-01-12']))] #, '21-12-16'

data_f = data
for fltr in Filters:
    data_f = data_f.loc[fltr]

binSize = 50
rangeStressLower = np.arange(0, 2000-binSize, binSize)
rangeStressUpper = np.arange(binSize, 2000, binSize)
countStress = [np.zeros(2000//binSize - 1), np.zeros(2000//binSize - 1)]

data_M270 = data_f.loc[data_f['bead type'] == 'M270']
data_M450 = data_f.loc[data_f['bead type'] == 'M450']

allData = [data_M270, data_M450]
text = ['M270', 'M450']

fig, ax = plt.subplots(1,1, figsize = (9, 6))


for i in range(2):
    df = allData[i]

    minS = df.minStress.values
    maxS = df.maxStress.values
    for k in range(1, len(minS)):
        countStress[i] += ((rangeStressLower > minS[k]) & (rangeStressUpper < maxS[k]))
    
    binPos = (rangeStressLower + rangeStressUpper)/2
    ax.plot(binPos, countStress[i], label = text[i] + ' - total Ncomp = {:.0f}'.format(df.shape[0]))

ax.legend()
ax.set_xlim([0, 800])
locator = matplotlib.ticker.MultipleLocator(binSize)
ax.xaxis.set_major_locator(locator)
ax.set_xlabel('stress value (Pa)')
ax.set_ylabel('# compressions')
locator = matplotlib.ticker.MultipleLocator(5)
ax.yaxis.set_major_locator(locator)
ax.yaxis.grid(True)
fig.suptitle('Number of compression which stress interval contains\n a given stress bin of width = {:.0f} Pa'.format(binSize))
fig.show()

In [ ]:
# ['21-12-08', '21-12-16', '22-01-12']
data = GlobalTable_meca_Py2
beadTypes = ['M270', 'M450']
dates = ['22-01-12']

fig = plt.figure(figsize = (9, 10), tight_layout=True)
gs = GridSpec(2, 2, figure=fig)
ax1 = fig.add_subplot(gs[0, 0])
ax2 = fig.add_subplot(gs[0, 1])
ax3 = fig.add_subplot(gs[1, :])
ax = [ax1, ax2, ax3]
colors = ['c', 'orange', 'b', 'r']

for i in range(len(beadTypes)):
    bT = beadTypes[i]
    filterList = [(GlobalTable_meca_Py2['validatedFit'] == True), 
           (GlobalTable_meca_Py2['validatedThickness'] == True),\
           (GlobalTable_meca_Py2['cell subtype'] == 'aSFL'), 
           (GlobalTable_meca_Py2['bead type'] == bT), 
           (GlobalTable_meca_Py2['substrate'] == '20um fibronectin discs'),
           (GlobalTable_meca_Py2['date'].apply(lambda x : x in dates))]
    globalFilter = filterList[0]
    
    for ii in range(1, len(filterList)):
        globalFilter = globalFilter & filterList[ii]
        
    data[globalFilter].plot(kind = 'scatter', ax = ax[i], x = 'surroundingThickness', y = 'minStress', 
                            marker = 'o', color = 'c', alpha = 0.3, label = 'Min stress')
    data[globalFilter].plot(kind = 'scatter', ax = ax[i], x = 'surroundingThickness', y = 'maxStress', 
                            marker = 'o', color = 'r', alpha = 0.3, label = 'Max stress')
    ax[i].set_xlabel('Cortical thickness (nm)')
    ax[i].set_ylabel('Extremal stress values (Pa)')
    ax[i].set_ylim([0, 2000])
    ax[i].set_xlim([0, 1200])
    ax[i].set_title(bT)
    data[globalFilter].plot(kind = 'scatter', ax = ax[2], x = 'surroundingThickness', y = 'minStress', 
                            marker = 'o', color = colors[2*i], alpha = 0.3, label = bT + ' - Min stress')
    data[globalFilter].plot(kind = 'scatter', ax = ax[2], x = 'surroundingThickness', y = 'maxStress', 
                            marker = 'o', color = colors[2*i+1], alpha = 0.3, label = bT + ' - Max stress')
    
ax[2].set_xlabel('Cortical thickness (nm)')
ax[2].set_xlim([0, 1200])

ax[2].set_ylabel('Extremal stress values (Pa)')
ax[2].set_ylim([0, 1000])
locator = matplotlib.ticker.MultipleLocator(100)
ax[2].yaxis.set_major_locator(locator)
ax[2].yaxis.grid(True)

ax[2].set_title('Jan 22 - Extremal stress values vs. thickness, for each compression')
# archiveFig(fig, ax, name='3T3aSFL_Jan22_M450vsM270_CompressionsLowStart_StressRanges', figSubDir = figSubDir)
fig.show()

In [ ]:
# ['21-12-08', '21-12-16', '22-01-12']
data = GlobalTable_meca_Py2
beadTypes = ['M270', 'M450']
dates = ['21-12-08', '22-01-12']

fig = plt.figure(figsize = (9, 10), tight_layout=True)
gs = GridSpec(2, 2, figure=fig)
ax1 = fig.add_subplot(gs[0, 0])
ax2 = fig.add_subplot(gs[0, 1])
ax3 = fig.add_subplot(gs[1, :])
ax = [ax1, ax2, ax3]
colors = ['c', 'orange', 'b', 'r']

for i in range(len(beadTypes)):
    bT = beadTypes[i]
    filterList = [(GlobalTable_meca_Py2['validatedFit'] == True), 
           (GlobalTable_meca_Py2['validatedThickness'] == True),\
           (GlobalTable_meca_Py2['cell subtype'] == 'aSFL'), 
           (GlobalTable_meca_Py2['bead type'] == bT), 
           (GlobalTable_meca_Py2['substrate'] == '20um fibronectin discs'),
           (GlobalTable_meca_Py2['date'].apply(lambda x : x in dates))]
    globalFilter = filterList[0]
    
    for ii in range(1, len(filterList)):
        globalFilter = globalFilter & filterList[ii]
        
    data[globalFilter].plot(kind = 'scatter', ax = ax[i], x = 'surroundingThickness', y = 'minStress', 
                            marker = 'o', color = 'c', alpha = 0.3, label = 'Min stress')
    data[globalFilter].plot(kind = 'scatter', ax = ax[i], x = 'surroundingThickness', y = 'maxStress', 
                            marker = 'o', color = 'r', alpha = 0.3, label = 'Max stress')
    ax[i].set_xlabel('Cortical thickness (nm)')
    ax[i].set_ylabel('Extremal stress values (Pa)')
    ax[i].set_ylim([0, 2000])
    ax[i].set_xlim([0, 1200])
    ax[i].set_title(bT)
    data[globalFilter].plot(kind = 'scatter', ax = ax[2], x = 'surroundingThickness', y = 'minStress', 
                            marker = 'o', color = colors[2*i], alpha = 0.3, label = bT + ' - Min stress')
    data[globalFilter].plot(kind = 'scatter', ax = ax[2], x = 'surroundingThickness', y = 'maxStress', 
                            marker = 'o', color = colors[2*i+1], alpha = 0.3, label = bT + ' - Max stress')
    
ax[2].set_xlabel('Cortical thickness (nm)')
ax[2].set_xlim([0, 1200])

ax[2].set_ylabel('Extremal stress values (Pa)')
ax[2].set_ylim([0, 1000])
locator = matplotlib.ticker.MultipleLocator(100)
ax[2].yaxis.set_major_locator(locator)
ax[2].yaxis.grid(True)

ax[2].set_title('Dec 21 - Extremal stress values vs. thickness, for each compression')

fig.show()

#### Plots that explore the impact of the delay before starting experiment

In [ ]:
figSubDir = 'ExploratoryPlots'
GlobalTable_meca_Py['compStartTimeThisDay_min'] = GlobalTable_meca_Py['compStartTimeThisDay']/60
GlobalTable_meca_Py

In [ ]:
figSubDir = 'ExploratoryPlots'
Filters = [(GlobalTable_meca_Py['validatedFit'] == True), (GlobalTable_meca_Py['validatedThickness'] == True),\
           (GlobalTable_meca_Py['cell subtype'] == 'aSFL'), (GlobalTable_meca_Py['substrate'] == '20um fibronectin discs')]
fig, ax = D1Plot(GlobalTable_meca_Py, CondCol=['manipID','drug'],Parameters=['surroundingThickness','EChadwick'],\
                 Filters=Filters,AvgPerCell=True,cellID='cellID',stats=False)
renameAxes(ax,renameDict1)
fig.suptitle('3T3aSFL - Compressions - Differences with manip')
for i in range(len(ax)):
    xTicks = ax[i].get_xticklabels()
    for j in range(len(xTicks)):
        xTicks[j].set_fontsize(9)
archiveFig(fig, ax, name='3T3aSFL-Compressions-DifferencesWithManip_PYTHONTABLE', figSubDir = figSubDir)
fig.show()

In [ ]:
Filters = [(GlobalTable_meca_Py['validatedFit'] == True), (GlobalTable_meca_Py['validatedThickness'] == True),\
           (GlobalTable_meca_Py['cell subtype'] == 'aSFL'), (GlobalTable_meca_Py['substrate'] == '20um fibronectin discs')]
fig, ax = D2Plot(GlobalTable_meca_Py, XCol='compStartTimeThisDay_min',YCol='EChadwick', CondCol = ['date_x','drug'],\
           Filters=Filters, cellID = 'cellID', AvgPerCell=False, modelFit=False)
renameAxes(ax,renameDict1)
ax.legend(loc='upper right')
ax.set_xlabel('Time since start of experiment (min)')
fig.suptitle('3T3aSFL - Compressions - Evolution with time')
archiveFig(fig, ax, name='3T3aSFL-Compressions-EvolWithTime_PYTHONTABLE', figSubDir = figSubDir)
fig.show()

In [ ]:
Filters = [(GlobalTable_meca_Py['validatedFit'] == True), (GlobalTable_meca_Py['validatedThickness'] == True),\
           (GlobalTable_meca_Py['cell subtype'] == 'aSFL-A8'), (GlobalTable_meca_Py['substrate'] == '20um fibronectin discs')]
fig, ax = D1Plot(GlobalTable_meca_Py, CondCol=['manipID','drug'],Parameters=['surroundingThickness','EChadwick'],\
                 Filters=Filters,AvgPerCell=True,cellID='cellID',stats=True)
renameAxes(ax,renameDict1)
fig.suptitle('3T3aSFL-A8 - Compressions - Differences with manip')
for i in range(len(ax)):
    xTicks = ax[i].get_xticklabels()
    for j in range(len(xTicks)):
        xTicks[j].set_fontsize(9)
archiveFig(fig, ax, name='3T3aSFL-A8-Compressions-DifferencesWithManip_PYTHONTABLE', figSubDir = figSubDir)
fig.show()

In [ ]:
Filters = [(GlobalTable_meca_Py['validatedFit'] == True), (GlobalTable_meca_Py['validatedThickness'] == True),\
           (GlobalTable_meca_Py['cell subtype'] == 'aSFL-A8'), (GlobalTable_meca_Py['substrate'] == '20um fibronectin discs')]
fig, ax = D2Plot(GlobalTable_meca_Py, XCol='compStartTimeThisDay_min',YCol='EChadwick', CondCol = ['date_x','drug'],\
           Filters=Filters, cellID = 'cellID', AvgPerCell=False, modelFit=False)
renameAxes(ax,renameDict1)
ax.legend(loc='upper right')
ax.set_xlabel('Time since start of experiment (min)')
fig.suptitle('3T3aSFL-A8 - Compressions - Evolution with time')
archiveFig(fig, ax, name='3T3aSFL-A8-Compressions-EvolWithTime_PYTHONTABLE', figSubDir = figSubDir)
fig.show()

In [ ]:
Filters = [(GlobalTable_meca_Py['validatedFit'] == True), (GlobalTable_meca_Py['validatedThickness'] == True),\
           (GlobalTable_meca_Py['cell subtype'] == 'aSFL-6FP'), (GlobalTable_meca_Py['substrate'] == '20um fibronectin discs')]
fig, ax = D1Plot(GlobalTable_meca_Py, CondCol=['manipID','drug'],Parameters=['surroundingThickness','EChadwick'],\
                 Filters=Filters,AvgPerCell=True,cellID='cellID',stats=True)
renameAxes(ax,renameDict1)
fig.suptitle('3T3aSFL-6FP - Compressions - Differences with manip')
for i in range(len(ax)):
    xTicks = ax[i].get_xticklabels()
    for j in range(len(xTicks)):
        xTicks[j].set_fontsize(9)
archiveFig(fig, ax, name='3T3aSFL-6FP-Compressions-DifferencesWithManip_PYTHONTABLE', figSubDir = figSubDir)
fig.show()

In [ ]:
Filters = [(GlobalTable_meca_Py['validatedFit'] == True), (GlobalTable_meca_Py['validatedThickness'] == True),\
           (GlobalTable_meca_Py['cell subtype'] == 'aSFL-6FP'), (GlobalTable_meca_Py['substrate'] == '20um fibronectin discs')]
fig, ax = D2Plot(GlobalTable_meca_Py, XCol='compStartTimeThisDay_min',YCol='EChadwick', CondCol = ['date_x','drug'],\
           Filters=Filters, cellID = 'cellID', AvgPerCell=False, modelFit=False)
renameAxes(ax,renameDict1)
ax.legend(loc='upper right')
ax.set_xlabel('Time since start of experiment (min)')
fig.suptitle('3T3aSFL-6FP - Compressions - Evolution with time')
archiveFig(fig, ax, name='3T3aSFL-6FP-Compressions-EvolWithTime_PYTHONTABLE', figSubDir = figSubDir)
fig.show()

#### Compare different data from python and matlab code

In [ ]:
Filters = [(GlobalTable_meca_Py['validatedFit'] == True), (GlobalTable_meca_Py['validatedThickness'] == True), 
           (GlobalTable_meca_Py['cell subtype'] == 'aSFL'), GlobalTable_meca_Py['date_x'].apply(lambda x : x in ['21-01-18', '21-01-21'])]
co_order = makeOrder(['20um fibronectin discs'],['none','doxycyclin'])
fig, ax = D1Plot(GlobalTable_meca_Py, CondCol=['substrate','drug'],\
                 Parameters=['surroundingThickness','EChadwick'],Filters=Filters,\
                 AvgPerCell = True, cellID='cellID', co_order=co_order, figSizeFactor = 0.8,
                 useHue = False, orientation = 'h')
renameAxes(ax,renameDict1)
fig.suptitle('3T3aSFL on diverse substrates: Compressions')
# archiveFig(fig, ax, name='3T3aSFL_substrate&drug_SurroundingThickness&EChadwick_NEWTABLE', figSubDir = figSubDir)
fig.show()

In [ ]:
Filters = [(GlobalTable_meca_Py2['validatedFit'] == True), 
           (GlobalTable_meca_Py2['validatedThickness'] == True), 
           (GlobalTable_meca_Py2['cell subtype'] == 'aSFL'), 
           (GlobalTable_meca_Py2['date_x'].apply(lambda x : x in ['21-01-18', '21-01-21']))]
co_order = makeOrder(['20um fibronectin discs'],['none','doxycyclin'])
fig, ax = D1Plot(GlobalTable_meca_Py2, CondCol=['substrate','drug'],\
                 Parameters=['surroundingThickness','EChadwick'],Filters=Filters,\
                 AvgPerCell = True, cellID='cellID', co_order=co_order, figSizeFactor = 0.8,
                 useHue = False, orientation = 'h')
renameAxes(ax,renameDict1)
fig.suptitle('3T3aSFL on diverse substrates: Compressions')
# archiveFig(fig, ax, name='3T3aSFL_substrate&drug_SurroundingThickness&EChadwick_NEWTABLE', figSubDir = figSubDir)
fig.show()

In [ ]:
TableV2

In [ ]:
FiltersPy = [(GlobalTable_meca_Py['validatedFit'] == True), 
           (GlobalTable_meca_Py['validatedThickness'] == True), 
           (GlobalTable_meca_Py['cell subtype'] == 'aSFL'), 
           (GlobalTable_meca_Py['date_x'].apply(lambda x : x in ['21-01-18', '21-01-21']))]
TableV1 = GlobalTable_meca_Py
for fltr in FiltersPy:
    TableV1 = TableV1.loc[fltr]
    
group = TableV1.groupby('cellID')
dictAggMean = getDictAggMean(TableV1)
TableV1 = group.agg(dictAggMean)

FiltersPy2 = [(GlobalTable_meca_Py2['validatedFit'] == True), 
           (GlobalTable_meca_Py2['validatedThickness'] == True), 
           (GlobalTable_meca_Py2['cell subtype'] == 'aSFL'), 
           (GlobalTable_meca_Py2['date_x'].apply(lambda x : x in ['21-01-18', '21-01-21']))]
TableV2 = GlobalTable_meca_Py2
for fltr in FiltersPy2:
    TableV2 = TableV2.loc[fltr]
    
group = TableV2.groupby('cellID')
dictAggMean = getDictAggMean(TableV2)
TableV2 = group.agg(dictAggMean)
    
cellID = TableV2['cellID'].values

plotDict = {'cellID':[],'manipID':[],'drug':[],'E1':[],'h1':[],'E2':[],'h2':[]}
strE = 'EChadwick'
strH = 'H0Chadwick' #'surroundingThickness'
for iD in cellID[:]:
    try:
        plotDict['cellID'].append(iD)
        plotDict['manipID'] = plotDict['manipID'] + [TableV1.loc[TableV1['cellID'] == iD, 'manipID'].values[0]]
        plotDict['drug'] = plotDict['drug'] + [TableV1.loc[TableV1['cellID'] == iD, 'drug'].values[0]]
        plotDict['E1'] = plotDict['E1'] + [TableV1.loc[TableV1['cellID'] == iD, strE].values[0]]
        plotDict['h1'] = plotDict['h1'] + [TableV1.loc[TableV1['cellID'] == iD, strH].values[0]]
        plotDict['E2'] = plotDict['E2'] + [TableV2.loc[TableV2['cellID'] == iD, strE].values[0]]
        plotDict['h2'] = plotDict['h2'] + [TableV2.loc[TableV2['cellID'] == iD, strH].values[0]]
    except:
        print(iD + ' ignored')

plotDf = pd.DataFrame(plotDict)

plotDf['h2/h1'] = plotDf['h2'] / plotDf['h1']
plotDf['E2/E1'] = plotDf['E2'] / plotDf['E1']

fig, ax = plt.subplots(2,2, figsize = (9,9))
dictColor = {'doxycyclin' : 'red', 'none' : 'cyan'}

cyan_line = matplotlib.lines.Line2D([], [], color='cyan', label='Control')
red_line = matplotlib.lines.Line2D([], [], color='red', label='Doxycyclin')

for i in range(plotDf.shape[0]):
    ax[0,0].plot([1, 2], [plotDf['h1'].values[i], plotDf['h2'].values[i]], 
               marker = 'o', color = dictColor[plotDf['drug'].values[i]],
                markeredgecolor='k', linewidth = 1)
    ax[0,0].set_ylabel('h (nm)')
    
    ax[0,1].plot([1, 2], [plotDf['E1'].values[i], plotDf['E2'].values[i]], 
               marker = 'o', color = dictColor[plotDf['drug'].values[i]],
                markeredgecolor='k', linewidth = 1)
    ax[0,1].set_ylabel('E (Pa)')


sns.swarmplot(y='h2/h1', data=plotDf, ax=ax[1,0], color = 'orange', edgecolor='k', linewidth = 1)
sns.swarmplot(y='E2/E1', data=plotDf, ax=ax[1,1], color = 'green', edgecolor='k', linewidth = 1)
ax[1,0].plot(ax[1,0].get_xlim(), [1,1], 'k--')
ax[1,1].plot(ax[1,1].get_xlim(), [1,1], 'k--')
    
for k in range(2):
    ax[0,k].set_xlim([0.6,3.4])
    ax[0,k].set_xticks([1, 2])
    ax[0,k].set_xticklabels(['Matlab', 'Python'])
    ax[0,k].legend(handles=[cyan_line, red_line])
    
#     x = [1, 2]
#     labels = ['Matlab', 'Python']

    
fig.suptitle('2021-01 experiments analysed with the two algos')

plotDf


#### Interactive

In [ ]:
Filters = [(GlobalTable_ctField['validated'] == True), (GlobalTable_meca['cell subtype'] == 'aSFL')]

p = D1PlotInteractive(GlobalTable_ctField, CondCol='drug',Parameters=['medianThickness','fluctuAmpli'],Filters=Filters)
p.children[0][0].title.text = '3T3aSFL on patterns: Ct Field'
p.children[0][0].title.text_font_size = '16pt'
p.children[1][0].title.text = ''
show(p)

In [ ]:
Filters = [(GlobalTable_meca['Validated'] == 1), (GlobalTable_meca['cell subtype'] == 'aSFL')]
p = D1PlotInteractive(GlobalTable_meca, CondCol='drug',Parameters=['SurroundingThickness','EChadwick'],Filters=Filters,AvgPerCell=True,cellID='CellName')
p.children[0][0].title.text = '3T3aSFL on patterns: Compressions'
p.children[0][0].title.text_font_size = '14pt'
p.children[1][0].title.text = ''
show(p)

In [ ]:
Filters = [(GlobalTable_ctField['validated'] == True), (GlobalTable_meca['cell subtype'] == 'aSFL')]
p = D2PlotInteractive(GlobalTable_ctField, XCol='medianThickness',YCol='fluctuAmpli',CondCol = 'drug', Filters=Filters)
show(p)

In [ ]:
Filters = [(GlobalTable_meca['Validated'] == True), (GlobalTable_meca['cell subtype'] == 'aSFL')]
p = D2PlotInteractive(GlobalTable_meca, XCol='SurroundingThickness',YCol='EChadwick',CondCol = 'drug', Filters=Filters, cellID = 'CellName')
show(p)

In [ ]:
Filters = [(GlobalTable_ctField['validated'] == True)]
p = D2PlotInteractive(GlobalTable_ctField, XCol='meanFluoPeakAmplitude',YCol='medianThickness',CondCol = 'drug', Filters=Filters, cellID = 'cellID',AvgPerCell=True)
show(p)

In [ ]:
Filters = [(GlobalTable_meca['Validated'] == True)]
p = D2PlotInteractive(GlobalTable_meca, XCol='meanFluoPeakAmplitude',YCol='SurroundingThickness',CondCol = 'drug', Filters=Filters, cellID = 'CellName',AvgPerCell=True)
p.title.text = '3T3aSFL expressing linker: H(fluo)'
p.title.text_font_size = '16pt'
show(p)

In [ ]:
Filters = [(GlobalTable_meca['Validated'] == True), (GlobalTable_meca['cell subtype'] == 'aSFL')]
p = D2PlotInteractive(GlobalTable_meca, XCol='meanFluoPeakAmplitude', YCol='EChadwick', CondCol = 'drug', Filters=Filters, cellID = 'CellName',AvgPerCell=True)
p.title.text = 'aSFL expressing linker: E(fluo)'
p.title.text_font_size = '16pt'
show(p)

In [ ]:
Filters = [(GlobalTable_meca['Validated'] == True), (GlobalTable_meca['cell subtype'] == 'aSFL')]
p = D2PlotInteractive(GlobalTable_meca, XCol='meanFluoPeakAmplitude',YCol='SurroundingThickness',CondCol = 'drug', Filters=Filters, cellID = 'CellName',AvgPerCell=True)
p.title.text = 'aSFL expressing linker: H(fluo)'
p.title.text_font_size = '16pt'
show(p)

In [ ]:
Outliers = []
Filters = [(GlobalTable_meca_Py['validatedFit'] == True), (GlobalTable_meca_Py['validatedThickness'] == True),
           (GlobalTable_meca_Py['drug'].apply(lambda x : x in ['none', 'doxycyclin'])),
           (GlobalTable_meca_Py['substrate'] == '20um fibronectin discs'), 
           (GlobalTable_meca_Py['cell subtype'].apply(lambda x : x in ['aSFL-A8-2'])),
           (GlobalTable_meca_Py['cellID'].apply(lambda x : x not in Outliers))]
p = D2PlotInteractive(GlobalTable_meca_Py, XCol='meanFluoPeakAmplitude', YCol='EChadwick', CondCol = 'drug', 
                      Filters=Filters, cellID = 'cellID', AvgPerCell=True)
p.title.text = 'aSFL-6FP expressing long linker: E(fluo)'
p.title.text_font_size = '16pt'
show(p)

In [ ]:
Filters = [(GlobalTable_meca['Validated'] == True), (GlobalTable_meca['cell subtype'] == 'aSFL-6FP')]
p = D2PlotInteractive(GlobalTable_meca, XCol='meanFluoPeakAmplitude', YCol='SurroundingThickness', CondCol = 'drug', Filters=Filters, cellID = 'CellName',AvgPerCell=True)
p.title.text = 'aSFL-6FP expressing long linker: H(fluo)'
p.title.text_font_size = '16pt'
show(p)

#### Region Fits

In [ ]:
figSubDir = 'RegionFits'

In [ ]:
Outliers = ['21-07-08_M3_P1_C10', '21-07-08_M3_P1_C5', '21-07-08_M3_P1_C6']
Filters = [(GlobalTable_meca_Py['validatedFit_f<100pN'] == True), (GlobalTable_meca_Py['validatedThickness'] == True), 
           (GlobalTable_meca_Py['drug'] == 'none'), (GlobalTable_meca_Py['substrate'] == '20um fibronectin discs'),
           (GlobalTable_meca_Py['cell subtype'] == 'aSFL'), (GlobalTable_meca_Py['cellID'].apply(lambda x : x not in Outliers)),
           (GlobalTable_meca_Py['bead type'].apply(lambda x : x in ['M270', 'M270_M450', 'M450_M270', 'M450']))]
co_order = makeOrder(['M270', 'M270_M450', 'M450_M270', 'M450'])
fig, ax = D1Plot(GlobalTable_meca_Py, CondCol=['bead type'],\
                 Parameters=['surroundingThickness','EChadwick_f<100pN'],Filters=Filters,\
                 AvgPerCell=True, cellID='cellID', co_order=co_order, figSizeFactor = 2, orientation = 'v', useHue = False)
renameAxes(ax,renameDict1)
fig.suptitle('3T3aSFL asym bead pairs - all comps - f<100pN')
# archiveFig(fig, ax, name='3T3aSFL_asymBeadsAllComp_SurroundingThickness&EChadwick', figSubDir = figSubDir)
fig.show()

In [ ]:
Outliers = ['21-07-08_M3_P1_C10', '21-07-08_M3_P1_C5', '21-07-08_M3_P1_C6']
Filters = [(GlobalTable_meca_Py['validatedFit'] == True), (GlobalTable_meca_Py['validatedThickness'] == True), 
           (GlobalTable_meca_Py['drug'] == 'none'), (GlobalTable_meca_Py['substrate'] == '20um fibronectin discs'),
           (GlobalTable_meca_Py['cell subtype'] == 'aSFL'), (GlobalTable_meca_Py['cellID'].apply(lambda x : x not in Outliers)),
           (GlobalTable_meca_Py['bead type'].apply(lambda x : x in ['M270', 'M270_M450', 'M450_M270', 'M450']))]
co_order = makeOrder(['M270', 'M270_M450', 'M450_M270', 'M450'])
fig, ax = D1Plot(GlobalTable_meca_Py, CondCol=['bead type'],\
                 Parameters=['surroundingThickness','EChadwick'],Filters=Filters,\
                 AvgPerCell=True, cellID='cellID', co_order=co_order, figSizeFactor = 2, orientation = 'v', useHue = False)
renameAxes(ax,renameDict1)
fig.suptitle('3T3aSFL asym bead pairs')
archiveFig(fig, ax, name='3T3aSFL_asymBeads_SurroundingThickness&EChadwick_allDates', figSubDir = figSubDir)
fig.show()

In [ ]:
Outliers = ['21-07-08_M3_P1_C10', '21-07-08_M3_P1_C5', '21-07-08_M3_P1_C6']
Filters = [(GlobalTable_meca_Py['validatedFit'] == True), (GlobalTable_meca_Py['validatedThickness'] == True), 
           (GlobalTable_meca_Py['date_x'] == '21-07-08'),
           (GlobalTable_meca_Py['drug'] == 'none'), (GlobalTable_meca_Py['substrate'] == '20um fibronectin discs'),
           (GlobalTable_meca_Py['cell subtype'] == 'aSFL'), (GlobalTable_meca_Py['cellID'].apply(lambda x : x not in Outliers)),
           (GlobalTable_meca_Py['bead type'].apply(lambda x : x in ['M270', 'M270_M450', 'M450_M270', 'M450']))]
co_order = makeOrder(['M270', 'M270_M450', 'M450_M270', 'M450'])
fig, ax = D1Plot(GlobalTable_meca_Py, CondCol=['bead type'],\
                 Parameters=['surroundingThickness','EChadwick'],Filters=Filters,\
                 AvgPerCell=True, cellID='cellID', co_order=co_order, figSizeFactor = 2, orientation = 'v', useHue = False)
renameAxes(ax,renameDict1)
fig.suptitle('3T3aSFL asym bead pairs')
archiveFig(fig, ax, name='3T3aSFL_asymBeads_SurroundingThickness&EChadwick_21-07-08', figSubDir = )
fig.show()

In [ ]:
Outliers = ['21-07-08_M3_P1_C10', '21-07-08_M3_P1_C5', '21-07-08_M3_P1_C6']
Filters = [(GlobalTable_meca_Py['validatedFit_f<100pN'] == True), (GlobalTable_meca_Py['validatedThickness'] == True), 
           (GlobalTable_meca_Py['drug'] == 'none'), (GlobalTable_meca_Py['substrate'] == '20um fibronectin discs'),
           (GlobalTable_meca_Py['cell subtype'] == 'aSFL'), (GlobalTable_meca_Py['cellID'].apply(lambda x : x not in Outliers)),
           (GlobalTable_meca_Py['bead type'].apply(lambda x : x in ['M270', 'M450']))]
fig, ax = D2Plot(GlobalTable_meca_Py, XCol='surroundingThickness',YCol='EChadwick_f<100pN', CondCol = ['bead type'],\
           Filters=Filters, cellID = 'cellID', AvgPerCell=True, modelFit=False, modelType='y=A*exp(kx)')
fig.suptitle('3T3aSFL: E(h)')
# archiveFig(fig, ax, name='aSFL_E(h)_drug&substrate', figDir = todayFigDir, figSubDir='')
fig.show()

In [ ]:
Outliers = ['21-07-08_M3_P1_C10', '21-07-08_M3_P1_C5', '21-07-08_M3_P1_C6']
Filters = [(GlobalTable_meca_Py['validatedFit_f<150pN'] == True), (GlobalTable_meca_Py['validatedThickness'] == True),
           (GlobalTable_meca_Py['date_x'].apply(lambda x : x in ['21-07-08'])), 
           (GlobalTable_meca_Py['drug'] == 'none'), (GlobalTable_meca_Py['substrate'] == '20um fibronectin discs'),
           (GlobalTable_meca_Py['cell subtype'] == 'aSFL'), (GlobalTable_meca_Py['cellID'].apply(lambda x : x not in Outliers)),
           (GlobalTable_meca_Py['bead type'].apply(lambda x : x in ['M270', 'M450_M270', 'M270_M450', 'M450']))]
fig, ax = D2Plot(GlobalTable_meca_Py, XCol='surroundingThickness',YCol='EChadwick_f<150pN', CondCol = ['bead type'],\
           Filters=Filters, cellID = 'cellID', AvgPerCell=False, modelFit=False, modelType='y=A*exp(kx)')
fig.suptitle('3T3aSFL: E(h) _f<150pN_21-07-08')
archiveFig(fig, ax, name='aSFL_beadTypes01_E(h)_21-07-08', figDir = todayFigDir, figSubDir=figSubDir)
fig.show()

In [ ]:
Outliers = ['21-07-08_M3_P1_C10', '21-07-08_M3_P1_C5', '21-07-08_M3_P1_C6']
Filters = [(GlobalTable_meca_Py['validatedFit_f<150pN'] == True), (GlobalTable_meca_Py['validatedThickness'] == True),
           (GlobalTable_meca_Py['date_x'].apply(lambda x : x in ['21-07-08'])), 
           (GlobalTable_meca_Py['drug'] == 'none'), (GlobalTable_meca_Py['substrate'] == '20um fibronectin discs'),
           (GlobalTable_meca_Py['cell subtype'] == 'aSFL'), (GlobalTable_meca_Py['cellID'].apply(lambda x : x not in Outliers)),
           (GlobalTable_meca_Py['bead type'].apply(lambda x : x in ['M270', 'M450']))]
fig, ax = D2Plot(GlobalTable_meca_Py, XCol='surroundingThickness',YCol='EChadwick_f<150pN', CondCol = ['bead type'],\
           Filters=Filters, cellID = 'cellID', AvgPerCell=False, modelFit=False, modelType='y=A*exp(kx)')
fig.suptitle('3T3aSFL: E(h) _f<150pN_21-07-08')
archiveFig(fig, ax, name='aSFL_beadTypes02_E(h)_21-07-08', figDir = todayFigDir, figSubDir=figSubDir)
fig.show()

#### Region fits, non linearity

In [ ]:
figSubDir = 'RegionFitsPlots'

##### On Jan 2021 data

In [ ]:
data = GlobalTable_meca_Py2

Filters = [(data['validatedFit'] == True), 
           (data['validatedThickness'] == True),
           (data['drug'] == 'none'),
           (data['substrate'] == '20um fibronectin discs'),
           (data['date'].apply(lambda x : x in ['21-01-18', '21-01-21']))]

data_f = data
for fltr in Filters:
    data_f = data_f.loc[fltr]
    
print(data_f[data_f['validatedFit_s<500Pa']].shape)
print(data_f[data_f['validatedFit_500<s<1000Pa']].shape)
print(data_f[data_f['validatedFit_s<500Pa'] & data_f['validatedFit_500<s<1000Pa']].shape)

In [ ]:
data = GlobalTable_meca_Py2
dates = ['21-01-18', '21-01-21']
fits = ['s<500Pa', '250<s<750Pa', '500<s<1000Pa']

Filters = [(data['validatedFit'] == True),
           (data['validatedThickness'] == True),
           (data['drug'] == 'none'),
           (data['substrate'] == '20um fibronectin discs'),
           (data['date'].apply(lambda x : x in dates))]
Filters += [(data['validatedFit_' + f] == True) for f in fits]

Parameters = ['EChadwick_' + f for f in fits]

fig, ax = D1PlotPaired(data, Parameters=Parameters, Filters=Filters, Boxplot=True, cellID='cellID', 
                   co_order=[], stats=True, statMethod='Wilcox_less', box_pairs=[],
                   figSizeFactor = 1.5, markersizeFactor=1, orientation = 'h', labels = fits)
ax.set_ylabel('E_Chadwick (Pa)')
fig.suptitle('Jan 2021 data ; B = 3 -> 40 mT ; no drug')
fig.show()

In [ ]:
data = GlobalTable_meca_Py2
dates = ['21-01-18', '21-01-21']
fits = ['s<500Pa', '250<s<750Pa', '500<s<1000Pa']

Filters = [(data['validatedFit'] == True),
           (data['validatedThickness'] == True),
           (data['drug'] == 'doxycyclin'),
           (data['substrate'] == '20um fibronectin discs'),
           (data['date'].apply(lambda x : x in dates))]
Filters += [(data['validatedFit_' + f] == True) for f in fits]

Parameters = ['EChadwick_' + f for f in fits]

fig, ax = D1PlotPaired(data, Parameters=Parameters, Filters=Filters, Boxplot=True, cellID='cellID', 
                   co_order=[], stats=True, statMethod='Wilcox_less', box_pairs=[],
                   figSizeFactor = 1.5, markersizeFactor=1, orientation = 'h', labels = fits)
ax.set_ylabel('E_Chadwick (Pa)')
fig.suptitle('Jan 2021 data ; B = 3 -> 40 mT ; doxy')
fig.show()

In [ ]:
data = GlobalTable_meca_Py2
dates = ['21-01-18', '21-01-21']
fits = ['s<400Pa', '300<s<700Pa', '600<s<1000Pa']

Filters = [(data['validatedFit'] == True),
           (data['validatedThickness'] == True),
           (data['drug'] == 'none'),
           (data['substrate'] == '20um fibronectin discs'),
           (data['date'].apply(lambda x : x in dates))]
Filters += [(data['validatedFit_' + f] == True) for f in fits]

Parameters = ['EChadwick_' + f for f in fits]

fig, ax = D1PlotPaired(data, Parameters=Parameters, Filters=Filters, Boxplot=True, cellID='cellID', 
                   co_order=[], stats=True, statMethod='Wilcox_less', box_pairs=[],
                   figSizeFactor = 1.5, markersizeFactor=1, orientation = 'h', labels = fits)
ax.set_ylabel('E_Chadwick (Pa)')
fig.suptitle('Jan 2021 data ; B = 3 -> 40 mT ; no drug')
fig.show()

In [ ]:
data = GlobalTable_meca_Py2
dates = ['21-12-08', '22-01-12']
fits = ['s<100Pa', '100<s<200Pa', '200<s<300Pa']

Filters = [(data['validatedFit'] == True),
           (data['validatedThickness'] == True),
           (data['drug'] == 'none'),
           (data['substrate'] == '20um fibronectin discs'),
           (data['date'].apply(lambda x : x in dates))]
Filters += [(data['validatedFit_' + f] == True) for f in fits]

Parameters = ['EChadwick_' + f for f in fits]

fig, ax = D1PlotPaired(data, Parameters=Parameters, Filters=Filters, Boxplot=True, cellID='cellID', 
                   co_order=[], stats=True, statMethod='Wilcox_less', box_pairs=[],
                   figSizeFactor = 1.5, markersizeFactor=1, orientation = 'h', labels = fits)
ax.set_ylabel('E_Chadwick (Pa)')
fig.suptitle('Jan 2022 data ; B = 1 -> 13 mT ; no drug')
fig.show()

In [ ]:
# ['21-12-08', '21-12-16', '22-01-12']
data = GlobalTable_meca_Py2

dates = ['22-02-09'] #['21-12-08', '22-01-12'] ['21-01-18', '21-01-21', '21-12-08']

filterList = [(data['validatedFit'] == True), 
           (data['validatedThickness'] == True),\
           (data['cell subtype'] == 'aSFL'), 
           (data['bead type'] == 'M450'),
           (data['substrate'] == '20um fibronectin discs'),
           (data['date'].apply(lambda x : x in dates))]


fig, ax = plt.subplots(1, 1, figsize = (9, 6), tight_layout=True)

globalFilter = filterList[0]
for k in range(1, len(filterList)):
    globalFilter = globalFilter & filterList[k]

X = data[globalFilter]['surroundingThickness'].values
Smin = data[globalFilter]['minStress'].values
Smax = data[globalFilter]['maxStress'].values

for i in range(len(X)):
    ax.plot([X[i], X[i]], [Smin[i], Smax[i]], 
            ls = '-', color = 'deepskyblue', alpha = 0.3,
            label = 'Stress range', zorder = 1)
    ax.plot([X[i]], [Smin[i]], 
            marker = 'o', markerfacecolor = 'skyblue', markeredgecolor = 'k', markersize = 4,
            ls = '')
    ax.plot([X[i]], [Smax[i]], 
            marker = 'o', markerfacecolor = 'royalblue', markeredgecolor = 'k', markersize = 4,
            ls = '')

ax.set_xlabel('Cortical thickness (nm)')
ax.set_xlim([0, 1200])
locator = matplotlib.ticker.MultipleLocator(100)
ax.xaxis.set_major_locator(locator)

ax.set_ylabel('Extremal stress values (Pa)')
ax.set_ylim([0, 2500])
locator = matplotlib.ticker.MultipleLocator(100)
ax.yaxis.set_major_locator(locator)
ax.yaxis.grid(True)

ax.set_title('Jan 21 & jan 22 - Extremal stress values vs. thickness, for each compression')
# archiveFig(fig, ax, name='3T3aSFL_Jan22_M450vsM270_CompressionsLowStart_StressRanges', figSubDir = figSubDir)
fig.show()

In [ ]:
data = GlobalTable_meca_Py2
dates = ['21-01-18', '21-01-21']
fit = 's<200Pa_included' # 's<400Pa', '300<s<700Pa', '600<s<1000Pa'

Filters = [(data['validatedFit'] == True), 
           (data['validatedThickness'] == True),
           (data['validatedFit_'+fit] == True),
           (data['substrate'] == '20um fibronectin discs'),
           (data['drug'] == 'none'),
           (data['date'].apply(lambda x : x in dates))]

fig, ax = D2Plot_wFit(data, XCol='surroundingThickness',YCol='EChadwick_' + fit,CondCol = ['bead type', 'date'],\
           Filters=Filters, cellID = 'cellID', AvgPerCell=False, xscale = 'log', yscale = 'log', modelFit=True, modelType='y=k*x^a')

ax.set_ylabel('EChadwick ['+ fit +']  (Pa)')
ax.set_xlabel('Thickness at low force (nm)')
fig.suptitle('3T3aSFL: E(h) // dates = ' + dates[0][:5] + ' // ' + fit + ' // All comp')
ax.legend(loc = 'upper right', fontsize = 8)
# archiveFig(fig, ax, name='3T3aSFL_Jan22_M450vsM270_CompressionsLowStart_E(h)', figDir = todayFigDir, figSubDir=figSubDir)
fig.show()

In [ ]:
data = GlobalTable_meca_Py2
dates = ['21-01-18', '21-01-21']
fit = '100<s<300Pa_200included' # 's<400Pa', '300<s<700Pa', '600<s<1000Pa'

Filters = [(data['validatedFit'] == True), 
           (data['validatedThickness'] == True),
           (data['bead type'] == 'M450'),
           (data['validatedFit_'+fit] == True),
           (data['Npts_'+fit] >= 15),
           (data['surroundingThickness'] <= 800),
           (data['substrate'] == '20um fibronectin discs'),
           (data['drug'] == 'none'),
           (data['date'].apply(lambda x : x in dates))]

fig, ax = D2Plot_wFit(data, XCol='surroundingThickness',YCol='EChadwick_' + fit,CondCol = ['bead type', 'date'],\
           Filters=Filters, cellID = 'cellID', AvgPerCell=False, xscale = 'log', yscale = 'log', modelFit=True, modelType='y=k*x^a')

ax.set_ylabel('EChadwick ['+ fit +']  (Pa)', fontsize = 12)
ax.set_xlabel('Thickness at low force (nm)', fontsize = 12)
fig.suptitle('3T3aSFL: E(h) // dates = ' + dates[0][:5] + ' // ' + fit + ' // All comp', fontsize = 14)
ax.legend(loc = 'upper right', fontsize = 8)
# archiveFig(fig, ax, name='3T3aSFL_Jan22_M450vsM270_CompressionsLowStart_E(h)', figDir = todayFigDir, figSubDir=figSubDir)
fig.show()

In [ ]:
data = GlobalTable_meca_Py2
dates = ['21-01-18', '21-01-21']
fit = '50<s<250Pa_150included' # 's<400Pa', '300<s<700Pa', '600<s<1000Pa'

Filters = [(data['validatedFit'] == True), 
           (data['validatedThickness'] == True),
           (data['bead type'] == 'M450'),
           (data['validatedFit_'+fit] == True),
           (data['Npts_'+fit] >= 15),
           (data['surroundingThickness'] <= 800),
           (data['substrate'] == '20um fibronectin discs'),
           (data['drug'] == 'none'),
           (data['date'].apply(lambda x : x in dates))]

fig, ax = D2Plot_wFit(data, XCol='surroundingThickness',YCol='EChadwick_' + fit,CondCol = ['bead type', 'date'],\
           Filters=Filters, cellID = 'cellID', AvgPerCell=False, xscale = 'log', yscale = 'log', modelFit=True, modelType='y=k*x^a')

ax.set_ylabel('EChadwick ['+ fit +']  (Pa)', fontsize = 12)
ax.set_xlabel('Thickness at low force (nm)', fontsize = 12)
fig.suptitle('3T3aSFL: E(h) // dates = ' + dates[0][:5] + ' // ' + fit + ' // All comp', fontsize = 14)
ax.legend(loc = 'upper right', fontsize = 8)
# archiveFig(fig, ax, name='3T3aSFL_Jan22_M450vsM270_CompressionsLowStart_E(h)', figDir = todayFigDir, figSubDir=figSubDir)
fig.show()

In [ ]:
data = GlobalTable_meca_Py2
dates = ['22-01-12']
fit = '50<s<250Pa_150included' # 's<400Pa', '300<s<700Pa', '600<s<1000Pa'

Filters = [(data['validatedFit'] == True), 
           (data['validatedThickness'] == True),
           (data['bead type'] == 'M450'),
           (data['validatedFit_'+fit] == True),
           (data['Npts_'+fit] >= 15),
           (data['surroundingThickness'] <= 800),
           (data['substrate'] == '20um fibronectin discs'),
           (data['drug'] == 'none'),
           (data['date'].apply(lambda x : x in dates))]

fig, ax = D2Plot_wFit(data, XCol='surroundingThickness',YCol='EChadwick_' + fit,CondCol = ['bead type'],\
           Filters=Filters, cellID = 'cellID', AvgPerCell=False, xscale = 'log', yscale = 'log', modelFit=True, modelType='y=k*x^a')

ax.set_ylabel('EChadwick ['+ fit +']  (Pa)', fontsize = 12)
ax.set_xlabel('Thickness at low force (nm)', fontsize = 12)
fig.suptitle('3T3aSFL: E(h) // dates = ' + dates[0][:5] + ' // ' + fit + ' // All comp', fontsize = 14)
ax.legend(loc = 'upper right', fontsize = 8)
# archiveFig(fig, ax, name='3T3aSFL_Jan22_M450vsM270_CompressionsLowStart_E(h)', figDir = todayFigDir, figSubDir=figSubDir)
fig.show()

In [ ]:
data = GlobalTable_meca_Py2
dates = ['21-01-18', '21-01-21', '22-01-12']
fit = '50<s<250Pa_150included' # 's<400Pa', '300<s<700Pa', '600<s<1000Pa'

Filters = [(data['validatedFit'] == True), 
           (data['validatedThickness'] == True),
           (data['bead type'] == 'M450'),
           (data['validatedFit_'+fit] == True),
           (data['Npts_'+fit] >= 15),
           (data['surroundingThickness'] <= 800),
           (data['substrate'] == '20um fibronectin discs'),
           (data['drug'] == 'none'),
           (data['date'].apply(lambda x : x in dates))]

fig, ax = D2Plot_wFit(data, XCol='surroundingThickness',YCol='EChadwick_' + fit,CondCol = ['bead type', 'date'],\
           Filters=Filters, cellID = 'cellID', AvgPerCell=False, xscale = 'log', yscale = 'log', modelFit=True, modelType='y=k*x^a')

ax.set_ylabel('EChadwick ['+ fit +']  (Pa)', fontsize = 12)
ax.set_xlabel('Thickness at low force (nm)', fontsize = 12)
fig.suptitle('3T3aSFL: E(h) // dates = ' + 'all' + ' // ' + fit + ' // All comp', fontsize = 14)
ax.legend(loc = 'upper right', fontsize = 8)
# archiveFig(fig, ax, name='3T3aSFL_Jan22_M450vsM270_CompressionsLowStart_E(h)', figDir = todayFigDir, figSubDir=figSubDir)
fig.show()

In [ ]:
data = GlobalTable_meca_Py2
dates = ['21-01-18', '21-01-21']
fit = 's<300Pa_included' # 's<400Pa', '300<s<700Pa', '600<s<1000Pa'

Filters = [(data['validatedFit'] == True), 
           (data['validatedThickness'] == True),
           (data['validatedFit_'+fit] == True),
           (data['substrate'] == '20um fibronectin discs'),
           (data['drug'] == 'none'),
           (data['date'].apply(lambda x : x in dates))]

fig, ax = D2Plot_wFit(data, XCol='surroundingThickness',YCol='EChadwick_' + fit,CondCol = ['bead type', 'date'],\
           Filters=Filters, cellID = 'cellID', AvgPerCell=False, xscale = 'log', yscale = 'log', modelFit=True, modelType='y=k*x^a')

ax.set_ylabel('EChadwick ['+ fit +']  (Pa)')
ax.set_xlabel('Thickness at low force (nm)')
fig.suptitle('3T3aSFL: E(h) // dates = ' + dates[0][:5] + ' // ' + fit + ' // All comp')
ax.legend(loc = 'upper right', fontsize = 8)
# archiveFig(fig, ax, name='3T3aSFL_Jan22_M450vsM270_CompressionsLowStart_E(h)', figDir = todayFigDir, figSubDir=figSubDir)
fig.show()

In [ ]:
data = GlobalTable_meca_Py2
dates = ['21-01-18', '21-01-21']
fit = '200<s<500Pa_350included' # 's<400Pa', '300<s<700Pa', '600<s<1000Pa'

Filters = [(data['validatedFit'] == True), 
           (data['validatedThickness'] == True),
           (data['validatedFit_'+fit] == True),
           (data['substrate'] == '20um fibronectin discs'),
           (data['drug'] == 'none'),
           (data['date'].apply(lambda x : x in dates))]

fig, ax = D2Plot_wFit(data, XCol='surroundingThickness',YCol='EChadwick_' + fit,CondCol = ['bead type', 'date'],\
           Filters=Filters, cellID = 'cellID', AvgPerCell=False, xscale = 'log', yscale = 'log', modelFit=True, modelType='y=k*x^a')

ax.set_ylabel('EChadwick ['+ fit +']  (Pa)')
ax.set_xlabel('Thickness at low force (nm)')
fig.suptitle('3T3aSFL: E(h) // dates = ' + dates[0][:5] + ' // ' + fit + ' // All comp')
ax.legend(loc = 'upper right', fontsize = 8)
# archiveFig(fig, ax, name='3T3aSFL_Jan22_M450vsM270_CompressionsLowStart_E(h)', figDir = todayFigDir, figSubDir=figSubDir)
fig.show()

In [ ]:
data = GlobalTable_meca_Py2
dates = ['21-01-18', '21-01-21']
fit = 's<200Pa' # 's<400Pa', '300<s<700Pa', '600<s<1000Pa'

Filters = [(data['validatedFit'] == True), 
           (data['validatedThickness'] == True),
           (data['validatedFit_'+fit] == True),
           (data['substrate'] == '20um fibronectin discs'),
           (data['drug'] == 'none'),
           (data['date'].apply(lambda x : x in dates))]

fig, ax = D2Plot_wFit(data, XCol='surroundingThickness',YCol='EChadwick_' + fit,CondCol = ['bead type', 'date'],\
           Filters=Filters, cellID = 'cellID', AvgPerCell=False, xscale = 'log', yscale = 'log', modelFit=True, modelType='y=k*x^a')

ax.set_ylabel('EChadwick ['+ fit +']  (Pa)')
ax.set_xlabel('Thickness at low force (nm)')
fig.suptitle('3T3aSFL: E(h) // dates = ' + dates[0][:5] + ' // ' + fit + ' // All comp')
ax.legend(loc = 'upper right', fontsize = 8)
# archiveFig(fig, ax, name='3T3aSFL_Jan22_M450vsM270_CompressionsLowStart_E(h)', figDir = todayFigDir, figSubDir=figSubDir)
fig.show()

In [ ]:
data = GlobalTable_meca_Py2
dates = ['21-01-18', '21-01-21']
fit = '250<s<750Pa' # 's<400Pa', '300<s<700Pa', '600<s<1000Pa'

Filters = [(data['validatedFit'] == True), 
           (data['validatedThickness'] == True),
           (data['validatedFit_'+fit] == True),
           (data['substrate'] == '20um fibronectin discs'),
           (data['drug'] == 'none'),
           (data['date'].apply(lambda x : x in dates))]

fig, ax = D2Plot_wFit(data, XCol='surroundingThickness',YCol='EChadwick_' + fit,CondCol = ['bead type', 'date'],\
           Filters=Filters, cellID = 'cellID', AvgPerCell=False, xscale = 'log', yscale = 'log', modelFit=True, modelType='y=k*x^a')

ax.set_ylabel('EChadwick ['+ fit +']  (Pa)')
ax.set_xlabel('Thickness at low force (nm)')
fig.suptitle('3T3aSFL: E(h) // dates = ' + dates[0][:5] + ' // ' + fit + ' // All comp', fontsize = 17)
ax.legend(loc = 'upper right', fontsize = 8)
# archiveFig(fig, ax, name='3T3aSFL_Jan22_M450vsM270_CompressionsLowStart_E(h)', figDir = todayFigDir, figSubDir=figSubDir)
fig.show()

In [ ]:
data = GlobalTable_meca_Py2
dates = ['21-01-18', '21-01-21']
fit = '500<s<1000Pa' # 's<400Pa', '300<s<700Pa', '600<s<1000Pa'

Filters = [(data['validatedFit'] == True), 
           (data['validatedThickness'] == True),
           (data['validatedFit_'+fit] == True),
           (data['substrate'] == '20um fibronectin discs'),
           (data['drug'] == 'none'),
           (data['date'].apply(lambda x : x in dates))]

fig, ax = D2Plot_wFit(data, XCol='surroundingThickness',YCol='EChadwick_' + fit,CondCol = ['bead type', 'date'],\
           Filters=Filters, cellID = 'cellID', AvgPerCell=False, xscale = 'log', yscale = 'log', modelFit=True, modelType='y=k*x^a')

ax.set_ylabel('EChadwick ['+ fit +']  (Pa)')
ax.set_xlabel('Thickness at low force (nm)')
fig.suptitle('3T3aSFL: E(h) // dates = ' + dates[0][:5] + ' // ' + fit + ' // All comp', fontsize = 17)
ax.legend(loc = 'upper right', fontsize = 8)
# archiveFig(fig, ax, name='3T3aSFL_Jan22_M450vsM270_CompressionsLowStart_E(h)', figDir = todayFigDir, figSubDir=figSubDir)
fig.show()

In [ ]:
data = GlobalTable_meca_Py2
dates = ['21-12-08', '21-12-16', '22-01-12']
fit = '' # ['s<100Pa', '100<s<200Pa', '200<s<300Pa']

Filters = [(data['validatedFit'] == True), 
           (data['validatedThickness'] == True),
           (data['validatedFit'+fit] == True),
           (data['substrate'] == '20um fibronectin discs'),
           (data['drug'] == 'none'),
           (data['date'].apply(lambda x : x in dates))]

fig, ax = D2Plot_wFit(data, XCol='surroundingThickness',YCol='EChadwick' + fit,CondCol = ['bead type', 'date'],\
           Filters=Filters, cellID = 'cellID', AvgPerCell=False, xscale = 'log', yscale = 'log', modelFit=True, modelType='y=k*x^a')

ax.set_ylabel('EChadwick ['+ fit +']  (Pa)')
ax.set_xlabel('Thickness at low force (nm)')
ax.legend(loc = 'upper right', fontsize = 8)
fig.suptitle('3T3aSFL: E(h) // dates = ' + dates[0][:5] + ' // ' + fit + ' // All comp')
# archiveFig(fig, ax, name='3T3aSFL_Jan22_M450vsM270_CompressionsLowStart_E(h)', figDir = todayFigDir, figSubDir=figSubDir)
fig.show()

In [ ]:
data = GlobalTable_meca_Py2
dates = ['22-01-12']
fit = '_s<400Pa' # ['s<100Pa', '100<s<200Pa', '200<s<300Pa']

Filters = [(GlobalTable_meca_Py2['validatedFit'] == True), 
           (GlobalTable_meca_Py2['validatedThickness'] == True), 
           (GlobalTable_meca_Py2['substrate'] == '20um fibronectin discs'),
           (GlobalTable_meca_Py2['date'].apply(lambda x : x in dates))]

fig, ax = D2Plot_wFit(data, XCol='surroundingThickness',YCol='EChadwick' + fit,CondCol = ['bead type'],\
           Filters=Filters, cellID = 'cellID', AvgPerCell=False, xscale = 'log', yscale = 'log', modelFit=True, modelType='y=k*x^a')

ax.set_ylabel('EChadwick ['+ fit[1:] +']  (Pa)')
ax.set_xlabel('Thickness at low force (nm)')
fig.suptitle('3T3aSFL: E(h)')
ax.legend(loc = 'upper right', fontsize = 8)
fig.suptitle('3T3aSFL: E(h) // dates = ' + dates[0][:5] + ' // ' + fit[1:] + ' // All comp')
# archiveFig(fig, ax, name='3T3aSFL_Jan22_M450vsM270_CompressionsLowStart_E(h)', figDir = todayFigDir, figSubDir=figSubDir)
fig.show()

In [ ]:
data = GlobalTable_meca_Py2
dates = ['22-01-12']
fit = '_s<300Pa' # ['s<100Pa', '100<s<200Pa', '200<s<300Pa']

Filters = [(GlobalTable_meca_Py2['validatedFit'] == True), 
           (GlobalTable_meca_Py2['validatedThickness'] == True), 
           (GlobalTable_meca_Py2['substrate'] == '20um fibronectin discs'),
           (GlobalTable_meca_Py2['date'].apply(lambda x : x in dates))]

fig, ax = D2Plot_wFit(data, XCol='surroundingThickness',YCol='EChadwick' + fit,CondCol = ['bead type'],\
           Filters=Filters, cellID = 'cellID', AvgPerCell=False, xscale = 'log', yscale = 'log', modelFit=True, modelType='y=k*x^a')

ax.set_ylabel('EChadwick ['+ fit[1:] +']  (Pa)')
ax.set_xlabel('Thickness at low force (nm)')
fig.suptitle('3T3aSFL: E(h)')
ax.legend(loc = 'upper right', fontsize = 8)
fig.suptitle('3T3aSFL: E(h) // dates = ' + dates[0][:5] + ' // ' + fit[1:] + ' // All comp')
# archiveFig(fig, ax, name='3T3aSFL_Jan22_M450vsM270_CompressionsLowStart_E(h)', figDir = todayFigDir, figSubDir=figSubDir)
fig.show()

In [ ]:
data = GlobalTable_meca_Py2
dates = ['21-12-08', '21-12-16', '22-01-12']
fit = '' # ['s<100Pa', '100<s<200Pa', '200<s<300Pa']

Filters = [(data['validatedFit'] == True), 
           (data['validatedThickness'] == True), 
           (data['substrate'] == '20um fibronectin discs'),
           (data['date'].apply(lambda x : x in dates))]

fig, ax = D2Plot_wFit(GlobalTable_meca_Py2, XCol='ctFieldThickness',YCol='EChadwick' + fit, CondCol = ['bead type', 'date'],\
           Filters=Filters, cellID = 'cellID', AvgPerCell=True, xscale = 'log', yscale = 'log', modelFit=True, modelType='y=k*x^a')

ax.set_ylabel('EChadwick ['+ fit[1:] +']  (Pa)')
ax.set_xlabel('Thickness at low force (nm)')
fig.suptitle('3T3aSFL: E(h)')
ax.legend(loc = 'upper right', fontsize = 8)
fig.suptitle('3T3aSFL: E(h) // dates = ' + dates[0][:5] + ' // ' + fit[1:] + ' // All comp')
# archiveFig(fig, ax, name='3T3aSFL_Jan22_M450vsM270_CompressionsLowStart_E(h)_all3exp', figDir = todayFigDir, figSubDir=figSubDir)
fig.show()

In [ ]:
data = GlobalTable_meca_Py2
dates = ['21-12-08', '22-01-12']
fit = '_s<400Pa' # ['s<100Pa', '100<s<200Pa', '200<s<300Pa']

# Filters = [(data['validatedFit'] == True), 
#            (data['validatedThickness'] == True),
#            (data['validatedFit_'+fit] == True),
#            (data['substrate'] == '20um fibronectin discs'),
#            (data['drug'] == 'none'),
#            (data['date'].apply(lambda x : x in dates))]

# fig, ax = D2Plot_wFit(data, XCol='surroundingThickness', YCol='EChadwick_' + fit, CondCol = ['bead type', 'date'],\
#            Filters=Filters, cellID = 'cellID', AvgPerCell=False, xscale = 'log', yscale = 'log', modelFit=True, modelType='y=k*x^a')


Filters = [(data['validatedFit'] == True), 
           (data['validatedThickness'] == True), 
           (data['substrate'] == '20um fibronectin discs'),
           (data['date'].apply(lambda x : x in dates))]

fig, ax = D2Plot_wFit(GlobalTable_meca_Py2, XCol='surroundingThickness',YCol='EChadwick' + fit, CondCol = ['bead type', 'date'],\
           Filters=Filters, cellID = 'cellID', AvgPerCell=False, xscale = 'log', yscale = 'log', modelFit=True, modelType='y=k*x^a')

ax.set_ylabel('EChadwick ['+ fit[1:] +']  (Pa)')
ax.set_xlabel('Thickness at low force (nm)')
fig.suptitle('3T3aSFL: E(h)')
ax.legend(loc = 'upper right', fontsize = 8)
fig.suptitle('3T3aSFL: E(h) // dates = ' + dates[0][:5] + ' // ' + fit[1:] + ' // All comp')
# archiveFig(fig, ax, name='3T3aSFL_Jan22_M450vsM270_CompressionsLowStart_E(h)_all3exp', figDir = todayFigDir, figSubDir=figSubDir)
fig.show()

In [ ]:
# plt.close('all')
data = GlobalTable_meca_Py2

Filters = [(data['validatedFit'] == True), (data['validatedThickness'] == True),
          (data['date'].apply(lambda x : x in ['21-12-08', '22-01-12']))] #, '21-12-16'

fig, ax = D1PlotDetailed(data, CondCol=['bead type'], Parameters=['surroundingThickness', 'EChadwick'], Filters=Filters, 
                Boxplot=True, cellID='cellID', co_order=[], stats=True, statMethod='Mann-Whitney', 
               box_pairs=[], figSizeFactor = 1.8, markersizeFactor=1, orientation = 'v', showManips = True)
# archiveFig(fig, ax, name='3T3aSFL_Jan22_M450vsM270_CompressionsLowStart_Detailed1DPlot', figSubDir = figSubDir)
fig.show()

In [ ]:
plt.close('all')
data = GlobalTable_meca_Py2

Filters = [(data['validatedFit'] == True), (data['validatedThickness'] == True),
          (data['date'].apply(lambda x : x in ['21-12-08', '21-12-16', '22-01-12']))] #, '21-12-16'

box_pairs=[('21-12-08 & M270', '21-12-08 & M450'),
             ('21-12-16 & M450', '21-12-16 & M270'),
             ('22-01-12 & M270', '22-01-12 & M450')]

fig, ax = D1PlotDetailed(data, CondCol=['date', 'bead type'], Parameters=['surroundingThickness', 'EChadwick'], Filters=Filters, 
                Boxplot=True, cellID='cellID', co_order=[], stats=True, statMethod='Mann-Whitney', 
               box_pairs=box_pairs, figSizeFactor = 0.9, markersizeFactor=1, orientation = 'v', showManips = True)
# archiveFig(fig, ax, name='3T3aSFL_Jan22_M450vsM270_CompressionsLowStart_Detailed1DPlot_all3exp', figSubDir = figSubDir)
fig.show()

##### On Feb 2022 data

In [21]:
# ['21-12-08', '21-12-16', '22-01-12']
data = GlobalTable_meca_Py2

dates = ['22-02-09'] #['21-12-08', '22-01-12'] ['21-01-18', '21-01-21', '21-12-08']

filterList = [(data['validatedThickness'] == True),
              (data['cell subtype'] == 'aSFL'), 
              (data['bead type'] == 'M450'),
              (data['substrate'] == '20um fibronectin discs'),
              (data['date'].apply(lambda x : x in dates))]  # (data['validatedFit'] == True), 


fig, ax = plt.subplots(1, 1, figsize = (9, 6), tight_layout=True)

globalFilter = filterList[0]
for k in range(1, len(filterList)):
    globalFilter = globalFilter & filterList[k]

Ncomp = len(X)
X = data[globalFilter]['H0Chadwick'].values
Smin = data[globalFilter]['minStress'].values
Smax = data[globalFilter]['maxStress'].values

for i in range(Ncomp):
    ax.plot([X[i], X[i]], [Smin[i], Smax[i]], 
            ls = '-', color = 'deepskyblue', alpha = 0.3,
            label = 'Stress range', zorder = 1)
    ax.plot([X[i]], [Smin[i]], 
            marker = 'o', markerfacecolor = 'skyblue', markeredgecolor = 'k', markersize = 4,
            ls = '')
    ax.plot([X[i]], [Smax[i]], 
            marker = 'o', markerfacecolor = 'royalblue', markeredgecolor = 'k', markersize = 4,
            ls = '')

ax.set_xlabel('Cortical thickness (nm)')
ax.set_xlim([0, 1200])
locator = matplotlib.ticker.MultipleLocator(100)
ax.xaxis.set_major_locator(locator)

ax.set_ylabel('Extremal stress values (Pa)')
ax.set_ylim([0, 2500])
locator = matplotlib.ticker.MultipleLocator(100)
ax.yaxis.set_major_locator(locator)
ax.yaxis.grid(True)

ax.set_title('Jan 21 & jan 22 - Extremal stress values vs. thickness, for each compression')
# archiveFig(fig, ax, name='3T3aSFL_Jan22_M450vsM270_CompressionsLowStart_StressRanges', figSubDir = figSubDir)
print(Ncomp)
fig.show()

IndexError: index 0 is out of bounds for axis 0 with size 0

### Clean Graphs

In [ ]:
figSubDir = 'CleanPlots'

print(pairedPalette)
pairedPalette

#### 3T3 aSFL (A11) on and off pattern, with or without doxy -- (january 2021)

In [ ]:
Filters = [(GlobalTable_meca['Validated'] == 1), (GlobalTable_meca['cell subtype'] == 'aSFL'), (GlobalTable_meca['substrate'] == 'BSA coated glass'), (GlobalTable_meca['SurroundingThickness'] > 0)]
co_order = makeOrder(['BSA coated glass'],['none','doxycyclin'])
fig, ax = D1Plot(GlobalTable_meca, CondCol=['substrate','drug'],\
                 Parameters=['SurroundingThickness','EChadwick'], Filters=Filters,\
                 AvgPerCell=False, cellID='CellName', co_order=co_order, figSizeFactor = 0.5)
renameAxes(ax,renameDict1)
fig.suptitle('3T3aSFL non-adherent: Compressions')
# archiveFig(fig, ax, name='3T3aSFLonBSA_drug_SurroundingThickness&EChadwick_allComp', figSubDir = figSubDir)
fig.show()

In [ ]:
Filters = [(GlobalTable_meca['Validated'] == 1), (GlobalTable_meca['cell subtype'] == 'aSFL'), (GlobalTable_meca['SurroundingThickness'] > 0)]
co_order = makeOrder(['BSA coated glass','20um fibronectin discs'],['none','doxycyclin'])
fig, ax = D1Plot(GlobalTable_meca, CondCol=['substrate','drug'],\
                 Parameters=['SurroundingThickness','EChadwick'],Filters=Filters,\
                 AvgPerCell=True, cellID='CellName', co_order=co_order, figSizeFactor = 0.8)
renameAxes(ax,renameDict1)
fig.suptitle('3T3aSFL on diverse substrates: Compressions')
# archiveFig(fig, ax, name='3T3aSFL_substrate&drug_SurroundingThickness&EChadwick', figSubDir = figSubDir)
fig.show()

In [ ]:
Filters = [(GlobalTable_meca_Py['validatedFit'] == True), (GlobalTable_meca_Py['validatedThickness'] == True), (GlobalTable_meca_Py['cell subtype'] == 'aSFL')]
co_order = makeOrder(['BSA coated glass','20um fibronectin discs'],['none','doxycyclin'])
fig, ax = D1Plot(GlobalTable_meca_Py, CondCol=['substrate','drug'],\
                 Parameters=['surroundingThickness','EChadwick'],Filters=Filters,\
                 AvgPerCell=True, cellID='cellID', co_order=co_order, figSizeFactor = 0.8,
                 useHue = False, orientation = 'v')
renameAxes(ax,renameDict1)
fig.suptitle('3T3aSFL on diverse substrates: Compressions')
# archiveFig(fig, ax, name='3T3aSFL_substrate&drug_SurroundingThickness&EChadwick_NEWTABLE', figSubDir = figSubDir)
fig.show()


In [ ]:
Filters = [(GlobalTable_meca_Py['validatedFit'] == True), 
           (GlobalTable_meca_Py['validatedThickness'] == True), 
           (GlobalTable_meca_Py['cell subtype'] == 'aSFL'),
          (GlobalTable_meca_Py['substrate'] == '20um fibronectin discs'),
           (GlobalTable_meca_Py['bead type'] == 'M450'),
          (GlobalTable_meca_Py['ctFieldThickness'] <= 600)]
co_order = makeOrder(['none','doxycyclin'])
fig, ax = D1Plot(GlobalTable_meca_Py, CondCol=['drug'],\
                 Parameters=['ctFieldThickness','EChadwick'],Filters=Filters,\
                 AvgPerCell=True, cellID='cellID', co_order=co_order, figSizeFactor = 1.0,
                 useHue = False, orientation = 'h', markerSizeFactor=1.2)
ax[0].set_ylim([0, ax[0].get_ylim()[1]])
renameAxes(ax,renameDict1)
fig.suptitle('3T3aSFL on fibronectin discs: Compressions')
# archiveFig(fig, ax, name='3T3aSFL_substrate&drug_SurroundingThickness&EChadwick_NEWTABLE', figSubDir = figSubDir)
fig.show()


In [ ]:
Filters = [(GlobalTable_ctField['validated'] == True), (GlobalTable_ctField['medianThickness'] <= 1000)]
co_order = makeOrder(['20um fibronectin discs'],['none','doxycyclin'])
fig, ax = D1Plot(GlobalTable_ctField, CondCol=['substrate','drug'],Parameters=['medianThickness','fluctuAmpli'],\
                 Filters=Filters,stats=True,co_order=co_order,figSizeFactor=0.5)
renameAxes(ax,renameDict1)
fig.suptitle('3T3aSFL on patterns: Constant Field')
# archiveFig(fig, ax, name='3T3aSFL_drug_medianThickness', figSubDir = figSubDir)

fig.show()

In [ ]:
Filters = [(GlobalTable_meca['Validated'] == True), (GlobalTable_meca['cell subtype'] == 'aSFL')]
fig, ax = D2Plot(GlobalTable_meca, XCol='meanFluoPeakAmplitude', YCol='EChadwick', CondCol = ['cell subtype', 'drug'], \
                 Filters=Filters, cellID = 'CellName', AvgPerCell=True, modelFit=True)

renameAxes(ax,renameDict1)
fig.suptitle('aSFL expressing linker: E(fluo)')
yt = ax.get_yticks()
ax.set_yticklabels((yt/1000).astype(int))
ax.set_ylabel('E Chadwick (kPa)')

# archiveFig(fig, ax, name='aSFL_iMC_E(fluo)', figDir = todayFigDir, figSubDir='FluoPlots')
fig.show()

In [ ]:
Filters = [(GlobalTable_meca['Validated'] == True), (GlobalTable_meca['cell subtype'] == 'aSFL')]
fig, ax = D2Plot(GlobalTable_meca, XCol='meanFluoPeakAmplitude', YCol='EChadwick', CondCol = ['cell subtype', 'drug'], \
                 Filters=Filters, cellID = 'CellName', AvgPerCell=True, modelFit=True)
renameAxes(ax,renameDict1)
fig.suptitle('aSFL expressing linker: E(fluo)')
# archiveFig(fig, ax, name='aSFL_iMC_E(fluo)', figDir = todayFigDir, figSubDir='FluoPlots')
fig.show()

In [ ]:
#Same as above without the lonely point
Filters = [(GlobalTable_meca['Validated'] == True), (GlobalTable_meca['cell subtype'] == 'aSFL'), (GlobalTable_meca['EChadwick'] <= 80000)]
fig, ax = D2Plot(GlobalTable_meca, XCol='meanFluoPeakAmplitude', YCol='EChadwick', CondCol = ['cell subtype', 'drug'], \
                 Filters=Filters, cellID = 'CellName', AvgPerCell=True, modelFit=True)
renameAxes(ax,renameDict1)
fig.suptitle('aSFL expressing linker: E(fluo)')
# archiveFig(fig, ax, name='aSFL_iMC_E(fluo)_woLastPoint', figDir = todayFigDir, figSubDir='FluoPlots')
fig.show()

In [ ]:
Filters = [(GlobalTable_meca['Validated'] == True), (GlobalTable_meca['cell subtype'] == 'aSFL')]
fig, ax = D2Plot(GlobalTable_meca, XCol='meanFluoPeakAmplitude',YCol='SurroundingThickness',CondCol = ['cell subtype', 'drug'], Filters=Filters, cellID = 'CellName',AvgPerCell=True)
renameAxes(ax,renameDict1)
fig.suptitle('aSFL expressing linker: H(fluo)')
# archiveFig(fig, ax, name='aSFL_iMC_H(fluo)', figDir = todayFigDir, figSubDir='FluoPlots')
fig.show()

In [ ]:
Filters = [(GlobalTable_ctField['validated'] == True)]
fig, ax = D2Plot(GlobalTable_ctField, XCol='meanFluoPeakAmplitude',YCol='medianThickness',CondCol = ['cell subtype', 'drug'],\
                 Filters=Filters, cellID = 'cellID', AvgPerCell=True)
renameAxes(ax,renameDict1)
fig.suptitle('aSFL expressing linker: medianH(fluo)')
# archiveFig(fig, ax, name='aSFL_iMC_medianH(fluo)', figDir = todayFigDir, figSubDir='FluoPlots')
fig.show()

In [17]:
data = GlobalTable_meca_Py2

Filters = [(data['validatedFit'] == True), (data['validatedThickness'] == True),
          (data['date'].apply(lambda x : x in ['21-01-18', '21-01-21']))] #, '21-12-16'

fig, ax = D1PlotDetailed(data, CondCol=['drug'], Parameters=['surroundingThickness', 'EChadwick'], Filters=Filters, 
                Boxplot=True, cellID='cellID', co_order=[], stats=True, statMethod='Mann-Whitney', 
               box_pairs=[], figSizeFactor = 1.8, markersizeFactor=1, orientation = 'v', showManips = True)
# archiveFig(fig, ax, name='3T3aSFL_Jan22_M450vsM270_CompressionsLowStart_Detailed1DPlot', figSubDir = figSubDir)
fig.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

##### Older graphs

In [ ]:
# Filters = [(GlobalTable_meca['Validated'] == 1), (GlobalTable_meca['cell subtype'] == 'aSFL')]
# co_order = makeOrder(['BSA coated glass','20um fibronectin discs'],['none','doxycyclin'])
# fig, ax = D1Plot(GlobalTable_meca, CondCol=['substrate','drug'],\
#                  Parameters=['EChadwick'],Filters=Filters,\
#                  AvgPerCell=True, cellID='CellName', co_order=co_order,statMethod = 'Mann-Whitney', statVerbose = 2)
# renameAxes(ax,renameDict1)
# fig.suptitle('3T3aSFL on diverse substrates: Compressions')
# fig.show()

# # test value should be one of the following: t-test_ind, t-test_welch, t-test_paired, Mann-Whitney, Mann-Whitney-gt, Mann-Whitney-ls, Levene, Wilcoxon, Kruskal.

In [ ]:
# Filters = [(GlobalTable_meca['Validated'] == 1), (GlobalTable_meca['cell subtype'] == 'aSFL')]
# co_order = makeOrder(['BSA coated glass','20um fibronectin discs'],['none','doxycyclin'])
# fig, ax = D1Plot(GlobalTable_meca, CondCol=['substrate','drug'],\
#                  Parameters=['SurroundingThickness','EChadwick'],Filters=Filters,\
#                  AvgPerCell=True, cellID='CellName', co_order=co_order, stats=False)
# renameAxes(ax,renameDict1)
# fig.suptitle('3T3aSFL on diverse substrates: Compressions')
# fig.show()
# # fig.savefig(todayFigDir + '//' + 'compressionsBSAvsFibro_woStats.png')

#### 3T3 aSFL : standard line (A11) versus different clones (A8 - high exp ; 6FP - long linker) -- First round (april - june 2021)

In [ ]:
Filters = [(GlobalTable_meca_Py['validatedThickness'] == True), (GlobalTable_meca_Py['ctFieldThickness'] <= 1000)]
co_order = makeOrder(['aSFL','aSFL-A8','aSFL-6FP'],['none','doxycyclin'])
box_pairs=[('aSFL & none', 'aSFL & doxycyclin'),
 ('aSFL-A8 & none', 'aSFL-A8 & doxycyclin'),
 ('aSFL-6FP & none', 'aSFL-6FP & doxycyclin'),
 ('aSFL & none', 'aSFL-A8 & none'),
 ('aSFL & none', 'aSFL-6FP & none')]
fig, ax = D1Plot(GlobalTable_meca_Py, CondCol=['cell subtype','drug'],Parameters=['ctFieldThickness','ctFieldFluctuAmpli'],\
                 Filters=Filters,AvgPerCell=True,stats=True,co_order=co_order,box_pairs=box_pairs,figSizeFactor=1)
renameAxes(ax,renameDict1)
fig.suptitle('3T3aSFL on patterns: H and DH from meca expe')
# archiveFig(fig, ax, name='3T3aSFL_drug_medianThickness_fromMeca_PYTHONTABLE', figSubDir = figSubDir)

fig.show()

In [ ]:
Filters = [(GlobalTable_meca['Validated'] == 1), (GlobalTable_meca['substrate'] == '20um fibronectin discs')]
co_order = makeOrder(['aSFL','aSFL-6FP'],['none','doxycyclin'])
fig, ax = D1Plot(GlobalTable_meca, CondCol=['cell subtype','drug'],Parameters=['SurroundingThickness','EChadwick'],\
                 Filters=Filters,AvgPerCell=True,cellID='CellName',co_order=co_order,stats=True)
renameAxes(ax,renameDict1)
fig.suptitle('3T3aSFL short vs long linker: Compressions')
# archiveFig(fig, ax, name='3T3aSFL_likerType&drug_SurroundingThickness&EChadwick', figSubDir = figSubDir)
fig.show()

In [ ]:
Filters = [(GlobalTable_meca['Validated'] == True), (GlobalTable_meca['substrate'] == '20um fibronectin discs')]
co_order = makeOrder(['aSFL','aSFL-A8','aSFL-6FP'],['none','doxycyclin'])
box_pairs=[('aSFL & none', 'aSFL & doxycyclin'),
 ('aSFL-A8 & none', 'aSFL-A8 & doxycyclin'),
 ('aSFL-6FP & none', 'aSFL-6FP & doxycyclin'),
 ('aSFL & none', 'aSFL-A8 & none'),
 ('aSFL & none', 'aSFL-6FP & none')]
fig, ax = D1Plot(GlobalTable_meca, CondCol=['cell subtype','drug'],Parameters=['SurroundingThickness','EChadwick'],\
                 Filters=Filters,AvgPerCell=True,cellID='CellName',co_order=co_order,box_pairs=box_pairs,stats=True)
renameAxes(ax,renameDict1)
fig.suptitle('3T3aSFL - All linker types: Compressions')
# archiveFig(fig, ax, name='3T3aSFL_likerType&drug_SurroundingThickness&EChadwick', figSubDir = figSubDir)
fig.show()

In [ ]:
Filters = [(GlobalTable_meca_Py['validatedFit'] == True), (GlobalTable_meca_Py['validatedThickness'] == True), (GlobalTable_meca_Py['substrate'] == '20um fibronectin discs')]
co_order = makeOrder(['aSFL','aSFL-A8','aSFL-6FP'],['none','doxycyclin'])
box_pairs=[('aSFL & none', 'aSFL & doxycyclin'),
 ('aSFL-A8 & none', 'aSFL-A8 & doxycyclin'),
 ('aSFL-6FP & none', 'aSFL-6FP & doxycyclin'),
 ('aSFL & none', 'aSFL-A8 & none'),
 ('aSFL & none', 'aSFL-6FP & none')]
fig, ax = D1Plot(GlobalTable_meca_Py, CondCol=['cell subtype','drug'],Parameters=['ctFieldThickness','EChadwick'],\
                 Filters=Filters,AvgPerCell=True,cellID='cellID',co_order=co_order,box_pairs=box_pairs,stats=True)
renameAxes(ax,renameDict1)
fig.suptitle('3T3aSFL - All linker types: Compressions')
# archiveFig(fig, ax, name='3T3aSFL_likerType&drug_ctFThickness&EChadwick_PYTHONTABLE', figSubDir = figSubDir)
fig.show()

In [ ]:
Filters = [(GlobalTable_meca_Py['validatedFit'] == True), (GlobalTable_meca_Py['validatedThickness'] == True),\
           (GlobalTable_meca_Py['substrate'] == '20um fibronectin discs'), (GlobalTable_meca_Py['drug'] == 'doxycyclin'),\
           (pd.isna(GlobalTable_meca_Py['meanFluoPeakAmplitude']) != True)]
co_order = makeOrder(['aSFL','aSFL-A8','aSFL-6FP'])
fig, ax = D1Plot(GlobalTable_meca_Py, CondCol=['cell subtype'],Parameters=['meanFluoPeakAmplitude'],\
                 Filters=Filters,AvgPerCell=True,cellID='cellID',co_order=co_order,stats=True,figSizeFactor=1.25)
renameAxes(ax,renameDict1)
fig.suptitle('3T3aSFL different cell lines\nLinker expression quantif by fluo')
archiveFig(fig, ax, name='3T3aSFL_likerType&_fluoExp_PYTHONTABLE', figSubDir = figSubDir)
fig.show()

In [ ]:
Filters = [(GlobalTable_meca_Py['validatedFit'] == True), ((GlobalTable_meca_Py['cell subtype'] == 'aSFL') | (GlobalTable_meca_Py['cell subtype'] == 'aSFL-A8')), (GlobalTable_meca_Py['drug'] == 'doxycyclin')]
fig, ax = D2Plot(GlobalTable_meca_Py, XCol='meanFluoPeakAmplitude', YCol='EChadwick', CondCol = ['cell subtype'], \
                 Filters=Filters, cellID = 'cellID', AvgPerCell=True, modelFit=True)
renameAxes(ax,renameDict1)
fig.suptitle('aSFL & aSFL-A8 expressing linker: E(fluo)')
# archiveFig(fig, ax, name='aSFL&A8_iMC_E(fluo)', figDir = todayFigDir, figSubDir='FluoPlots')
fig.show()

In [ ]:
#Same as above without the lonely point
Filters = [(GlobalTable_meca_Py['validatedFit'] == True), (GlobalTable_meca_Py['drug'] == 'doxycyclin'), \
           ((GlobalTable_meca_Py['cell subtype'] == 'aSFL') | (GlobalTable_meca_Py['cell subtype'] == 'aSFL-A8')), \
           (GlobalTable_meca_Py['meanFluoPeakAmplitude'] <= 1200), (GlobalTable_meca_Py['EChadwick'] >= 2000)]
fig, ax = D2Plot(GlobalTable_meca_Py, XCol='meanFluoPeakAmplitude', YCol='EChadwick', CondCol = ['cell subtype'], \
                 Filters=Filters, cellID = 'cellID', AvgPerCell=True, modelFit=True)
renameAxes(ax,renameDict1)
fig.suptitle('aSFL & aSFL-A8 expressing linker: E(fluo)')
# archiveFig(fig, ax, name='aSFL&A8_iMC_E(fluo)_fluo-1200_&_E+2000', figDir = todayFigDir, figSubDir='FluoPlots')
fig.show()

In [ ]:
Filters = [(GlobalTable_meca['Validated'] == True), (GlobalTable_meca['cell subtype'] == 'aSFL-6FP')]
fig, ax = D2Plot(GlobalTable_meca, XCol='meanFluoPeakAmplitude', YCol='EChadwick', CondCol = ['cell subtype', 'drug'], \
                 Filters=Filters, cellID = 'CellName', AvgPerCell=True, modelFit=True)
renameAxes(ax,renameDict1)
fig.suptitle('aSFL-6FP expressing long linker: E(fluo)')
# archiveFig(fig, ax, name='aSFL-6FP_iMC_E(fluo)', figDir = todayFigDir, figSubDir='FluoPlots')
fig.show()

In [ ]:
#Same as above without the lonely point
Filters = [(GlobalTable_meca['Validated'] == True), (GlobalTable_meca['cell subtype'] == 'aSFL-6FP'), (GlobalTable_meca['meanFluoPeakAmplitude'] <= 1000)]
fig, ax = D2Plot(GlobalTable_meca, XCol='meanFluoPeakAmplitude', YCol='EChadwick', CondCol = ['cell subtype', 'drug'], \
                 Filters=Filters, cellID = 'CellName', AvgPerCell=True, modelFit=True)
renameAxes(ax,renameDict1)
fig.suptitle('aSFL-6FP expressing long linker: E(fluo)')
# archiveFig(fig, ax, name='aSFL-6FP_iMC_E(fluo)_woLastPoint', figDir = todayFigDir, figSubDir='FluoPlots')
fig.show()

In [ ]:
Filters = [(GlobalTable_meca['Validated'] == True), (GlobalTable_meca['cell subtype'] == 'aSFL-6FP')]
fig, ax = D2Plot(GlobalTable_meca, XCol='meanFluoPeakAmplitude', YCol='SurroundingThickness', CondCol = ['cell subtype', 'drug'],\
                 Filters=Filters, cellID = 'CellName',AvgPerCell=True)
renameAxes(ax,renameDict1)
fig.suptitle('aSFL-6FP expressing long linker: H(fluo)')
# archiveFig(fig, ax, name='aSFL-6FP_iMC_H(fluo)', figDir = todayFigDir, figSubDir='FluoPlots')
fig.show()

#### Diverse 2D plots

In [ ]:
Filters = [(GlobalTable_ctField['validated'] == True), (GlobalTable_ctField['cell subtype'] == 'aSFL')]
fig, ax = D2Plot(GlobalTable_ctField, XCol='medianThickness',YCol='fluctuAmpli',CondCol = ['cell subtype', 'drug'],\
                 Filters=Filters, modelFit=True, figSizeFactor = 0.8)
fig.suptitle('3T3aSFL - Dh = f(H)')
# archiveFig(fig, ax, name='aSFL_Dh(h)_drug', figDir = todayFigDir, figSubDir='ThicknessPlots')
fig.show()

In [ ]:
# Same as above without the 2 thickest cells
Filters = [(GlobalTable_ctField['validated'] == True), (GlobalTable_ctField['cell subtype'] == 'aSFL'),\
           (GlobalTable_ctField['medianThickness'] <= 700)]
fig, ax = D2Plot(GlobalTable_ctField, XCol='medianThickness',YCol='fluctuAmpli',CondCol = ['cell subtype', 'drug'],\
                 Filters=Filters, modelFit=True, figSizeFactor = 0.8)
fig.suptitle('3T3aSFL - Dh = f(H)')
# archiveFig(fig, ax, name='aSFL_Dh(h)_drug_wo2LastPoints', figDir = todayFigDir, figSubDir='ThicknessPlots')
fig.show()

In [ ]:
Filters = [(GlobalTable_meca['Validated'] == True), (GlobalTable_meca['cell subtype'] == 'aSFL')]
fig, ax = D2Plot(GlobalTable_meca, XCol='SurroundingThickness',YCol='EChadwick',CondCol = ['substrate','drug'],\
           Filters=Filters, cellID = 'CellName', AvgPerCell=True, modelFit=False, modelType='y=A*exp(kx)')
fig.suptitle('3T3aSFL: E(h)')
# archiveFig(fig, ax, name='aSFL_E(h)_drug&substrate', figDir = todayFigDir, figSubDir='')
fig.show()

#### Asym bead pairs -- (july 2021)

In [ ]:
rawMecaTable = getGlobalTable_meca()
# rawMecaTable.head()

In [ ]:
#  GlobalTable_meca_Py['bead type'].apply(lambda x : x in ['M270_M270', 'M270_M450', 'M450_M270', 'M450_M450'])

In [ ]:
Outliers = ['21-07-08_M3_P1_C10', '21-07-08_M3_P1_C5', '21-07-08_M3_P1_C6']
Filters = [(GlobalTable_meca_Py['validatedFit'] == True), (GlobalTable_meca_Py['validatedThickness'] == True), (GlobalTable_meca_Py['drug'] == 'none'), (GlobalTable_meca_Py['substrate'] == '20um fibronectin discs'),\
           (GlobalTable_meca_Py['cell subtype'] == 'aSFL'), (GlobalTable_meca_Py['cellID'].apply(lambda x : x not in Outliers)), \
           (GlobalTable_meca_Py['bead type'].apply(lambda x : x in ['M270', 'M270_M450', 'M450_M270', 'M450']))]
co_order = makeOrder(['M270', 'M270_M450', 'M450_M270', 'M450'])
fig, ax = D1Plot(GlobalTable_meca_Py, CondCol=['bead type'],\
                 Parameters=['surroundingThickness','EChadwick'],Filters=Filters,\
                 AvgPerCell=False, cellID='cellID', co_order=co_order, figSizeFactor = 2, orientation = 'v', useHue = False)
renameAxes(ax,renameDict1)
fig.suptitle('3T3aSFL asym bead pairs - all comps')
# archiveFig(fig, ax, name='3T3aSFL_asymBeadsAllComp_SurroundingThickness&EChadwick', figSubDir = figSubDir)
fig.show()

In [ ]:
Outliers = ['21-07-08_M3_P1_C10','21-07-08_M3_P1_C5','21-07-08_M3_P1_C6']
Filters = [(GlobalTable_meca_Py['validatedFit'] == True), (GlobalTable_meca_Py['validatedThickness'] == True), (GlobalTable_meca_Py['drug'] == 'none'), (GlobalTable_meca_Py['substrate'] == '20um fibronectin discs'), \
           (GlobalTable_meca_Py['cell subtype'] == 'aSFL'), (GlobalTable_meca_Py['cellID'].apply(lambda x : x not in Outliers)), \
           (GlobalTable_meca_Py['bead type'].apply(lambda x : x in ['M270', 'M270_M450', 'M450_M270', 'M450']))]
co_order = makeOrder(['M270', 'M270_M450', 'M450_M270', 'M450'])
fig, ax = D1Plot(GlobalTable_meca_Py, CondCol=['bead type'],\
                 Parameters=['surroundingThickness','EChadwick'],Filters=Filters,\
                 AvgPerCell=True, cellID='cellID', co_order=co_order, figSizeFactor = 2, orientation = 'v', useHue = True)
renameAxes(ax,renameDict1)
fig.suptitle('3T3aSFL asym bead pairs - avg per cell')
# archiveFig(fig, ax, name='3T3aSFL_asymBeadsPerCell_SurroundingThickness&EChadwick', figSubDir = figSubDir)
fig.show()

In [ ]:
Filters = [(rawMecaTable['Validated'] == 1), ((rawMecaTable['ExpType'] == 'DictyDB_M270') | (rawMecaTable['ExpType'] == 'DictyDB_M450')), (rawMecaTable['TpsComp'] == '1s')]
# Filters = [(rawMecaTable['Validated'] == 1), ((rawMecaTable['ExpType'] == 'DictyDB_M450')), (rawMecaTable['TpsComp'] == '1s')]
fig, ax = D1Plot(rawMecaTable, CondCol=['ExpType'],Parameters=['SurroundingThickness','EChadwick'],\
                 Filters=Filters,AvgPerCell=True,cellID='CellName', useHue = False)
fig.suptitle('M450 vs M270 pour compressions de 1s')
renameAxes(ax,renameDict1)
archiveFig(fig, ax, name='Dictys_beadTypes_SurroundingThickness&EChadwick', figSubDir = figSubDir)
fig.show()
# rawMecaTable[Filters[0] & Filters[1] & Filters[2]]

#### Smifh2 vs Dmso -- (Sergio's visit september 2021)

In [ ]:
pd.set_option('max_columns', None)
# pd.reset_option('max_columns')
pd.set_option('max_rows', None)
# pd.reset_option('max_rows')

In [ ]:
Outliers = ['21-09-02_M4_P1_C3', '21-09-02_M3_P1_C2', '21-09-02_M3_P1_C9']
Filters = [(GlobalTable_meca_Py['drug'].apply(lambda x : x in ['dmso', 'smifh2', 'dmso. doxycyclin', 'smifh2. doxycyclin'])), \
           (GlobalTable_meca_Py['substrate'] == '20um fibronectin discs'), (GlobalTable_meca_Py['cell subtype'] == 'aSFL'), \
           (GlobalTable_meca_Py['cellID'].apply(lambda x : x not in Outliers))]
data_filtered = GlobalTable_meca_Py
for fltr in Filters:
    data_filtered = data_filtered.loc[fltr]
data_filtered

In [ ]:
Outliers = ['21-09-02_M4_P1_C3', '21-09-02_M3_P1_C2', '21-09-02_M3_P1_C9']
Filters = [(GlobalTable_meca_Py['validatedFit'] == True), (GlobalTable_meca_Py['validatedThickness'] == True), \
           (GlobalTable_meca_Py['drug'].apply(lambda x : x in ['dmso', 'smifh2', 'dmso. doxycyclin', 'smifh2. doxycyclin'])), \
           (GlobalTable_meca_Py['substrate'] == '20um fibronectin discs'), (GlobalTable_meca_Py['cell subtype'] == 'aSFL'), \
           (GlobalTable_meca_Py['cellID'].apply(lambda x : x not in Outliers))]
co_order = makeOrder(['dmso', 'smifh2', 'dmso. doxycyclin', 'smifh2. doxycyclin'])
fig, ax = D1Plot(GlobalTable_meca_Py, CondCol=['drug'],\
                 Parameters=['ctFieldThickness','EChadwick'], Filters=Filters,\
                 AvgPerCell=True, cellID='cellID', co_order=co_order, figSizeFactor = 1.8, orientation = 'v')
renameAxes(ax,renameDict1)
fig.suptitle('3T3aSFL smifh2')
archiveFig(fig, ax, name='3T3aSFL_smifh2&doxy_Thickness&EChadwick', figSubDir = figSubDir)
fig.show()
# 'ctFieldThickness', 'surroundingThickness', 'none', 'doxycyclin', 

#### Long linker second clone (3T3 aSFL-6FP-2) -- (Sergio's visit september 2021)

In [ ]:
pd.set_option('max_columns', None)
pd.reset_option('max_columns')
pd.set_option('max_rows', None)
pd.reset_option('max_rows')

In [ ]:
# Outliers = []
# Filters = [(GlobalTable_meca_Py['validatedFit'] == True), (GlobalTable_meca_Py['validatedThickness'] == True), \
#            (GlobalTable_meca_Py['drug'].apply(lambda x : x in ['none', 'doxycyclin'])), \
#            (GlobalTable_meca_Py['substrate'] == '20um fibronectin discs'), (GlobalTable_meca_Py['cell subtype'].apply(lambda x : x in ['aSFL', 'aSFL-6FP', 'aSFL-6FP-2'])), \
#            (GlobalTable_meca_Py['cellID'].apply(lambda x : x not in Outliers))]
# co_order = makeOrder(['aSFL', 'aSFL-6FP', 'aSFL-6FP-2'], ['none', 'doxycyclin'])
# box_pairs=[('aSFL & none', 'aSFL & doxycyclin'),
#  ('aSFL-6FP & none', 'aSFL-6FP & doxycyclin'),
#  ('aSFL-6FP-2 & none', 'aSFL-6FP-2 & doxycyclin'),
#  ('aSFL & none', 'aSFL-6FP & none'),
#  ('aSFL & none', 'aSFL-6FP-2 & none'),
#  ('aSFL-6FP & none', 'aSFL-6FP-2 & none')]
# fig, ax = D1Plot(GlobalTable_meca_Py, CondCol=['cell subtype', 'drug'],\
#                  Parameters=['ctFieldThickness','EChadwick'], Filters=Filters,\
#                  AvgPerCell=True, cellID='cellID', co_order=co_order, box_pairs = box_pairs, figSizeFactor = 1, orientation = 'v')
# ax[0].set_ylim([0, ax[0].get_ylim()[1]])
# renameAxes(ax,renameDict1)
# fig.suptitle('3T3aSFL 6FP-2')
# archiveFig(fig, ax, name='3T3aSFL-6FP-2_Thickness&EChadwick', figSubDir = figSubDir)
# fig.show()
# # 'ctFieldThickness', 'surroundingThickness', 'none', 'doxycyclin', 

In [ ]:
Outliers = []
Filters = [(GlobalTable_meca_Py['validatedFit'] == True), (GlobalTable_meca_Py['validatedThickness'] == True), \
           (GlobalTable_meca_Py['drug'].apply(lambda x : x in ['none', 'doxycyclin'])), \
           (GlobalTable_meca_Py['substrate'] == '20um fibronectin discs'), (GlobalTable_meca_Py['cell subtype'].apply(lambda x : x in ['aSFL', 'aSFL-6FP-2'])), \
           (GlobalTable_meca_Py['cellID'].apply(lambda x : x not in Outliers))]
co_order = makeOrder(['aSFL', 'aSFL-6FP-2'], ['none', 'doxycyclin'])
box_pairs=[('aSFL & none', 'aSFL & doxycyclin'),
 ('aSFL-6FP-2 & none', 'aSFL-6FP-2 & doxycyclin'),
 ('aSFL & none', 'aSFL-6FP-2 & none')]
fig, ax = D1Plot(GlobalTable_meca_Py, CondCol=['cell subtype', 'drug'],\
                 Parameters=['ctFieldThickness','EChadwick'], Filters=Filters,\
                 AvgPerCell=True, cellID='cellID', co_order=co_order, box_pairs = box_pairs, figSizeFactor = 1, orientation = 'v')
ax[0].set_ylim([0, ax[0].get_ylim()[1]])
renameAxes(ax,renameDict1)
fig.suptitle('3T3aSFL 6FP-2')
archiveFig(fig, ax, name='3T3aSFL-6FP-2_Thickness&EChadwick', figSubDir = figSubDir)
fig.show()
# 'ctFieldThickness', 'surroundingThickness', 'none', 'doxycyclin', 

In [ ]:
Outliers = []
Filters = [(GlobalTable_meca_Py['validatedFit'] == True), (GlobalTable_meca_Py['validatedThickness'] == True), \
           (GlobalTable_meca_Py['drug'].apply(lambda x : x in ['none', 'doxycyclin'])), \
           (GlobalTable_meca_Py['substrate'] == '20um fibronectin discs'), (GlobalTable_meca_Py['cell subtype'].apply(lambda x : x in ['aSFL-6FP-2'])), \
           (GlobalTable_meca_Py['cellID'].apply(lambda x : x not in Outliers))]
fig, ax = D2Plot(GlobalTable_meca_Py, XCol='meanFluoPeakAmplitude', YCol='EChadwick', CondCol = ['cell subtype', 'drug'], \
                 Filters=Filters, cellID = 'cellID', AvgPerCell=True, modelFit=True)

renameAxes(ax,renameDict1)
fig.suptitle('aSFL-6FP-2 expressing linker: E(fluo)')
yt = ax.get_yticks()
ax.set_yticklabels((yt/1000).astype(int))
ax.set_ylabel('E Chadwick (kPa)')

# archiveFig(fig, ax, name='aSFL_iMC_E(fluo)', figDir = todayFigDir, figSubDir='FluoPlots')
fig.show()

In [ ]:
Outliers = []
Filters = [(GlobalTable_meca_Py['validatedFit'] == True), (GlobalTable_meca_Py['validatedThickness'] == True), \
           (GlobalTable_meca_Py['drug'].apply(lambda x : x in ['none', 'doxycyclin'])), \
           (GlobalTable_meca_Py['substrate'] == '20um fibronectin discs'), 
           (GlobalTable_meca_Py['cell subtype'].apply(lambda x : x in ['aSFL-6FP'])), \
           (GlobalTable_meca_Py['cellID'].apply(lambda x : x not in Outliers))]
fig, ax = D2Plot(GlobalTable_meca_Py, XCol='meanFluoPeakAmplitude', YCol='EChadwick', CondCol = ['cell subtype', 'drug'], \
                 Filters=Filters, cellID = 'cellID', AvgPerCell=True, modelFit=True)

renameAxes(ax,renameDict1)
fig.suptitle('aSFL-6FP expressing linker: E(fluo)')
yt = ax.get_yticks()
ax.set_yticklabels((yt/1000).astype(int))
ax.set_ylabel('E Chadwick (kPa)')

# archiveFig(fig, ax, name='aSFL_iMC_E(fluo)', figDir = todayFigDir, figSubDir='FluoPlots')
fig.show()

In [ ]:
Outliers = []
Filters = [(GlobalTable_meca_Py['validatedFit'] == True), (GlobalTable_meca_Py['validatedThickness'] == True), \
           (GlobalTable_meca_Py['drug'].apply(lambda x : x in ['none', 'doxycyclin'])), \
           (GlobalTable_meca_Py['substrate'] == '20um fibronectin discs'), 
           (GlobalTable_meca_Py['cell subtype'].apply(lambda x : x in ['aSFL'])), \
           (GlobalTable_meca_Py['cellID'].apply(lambda x : x not in Outliers))]
fig, ax = D2Plot(GlobalTable_meca_Py, XCol='meanFluoPeakAmplitude', YCol='EChadwick', CondCol = ['cell subtype', 'drug'], \
                 Filters=Filters, cellID = 'cellID', AvgPerCell=True, modelFit=True)

renameAxes(ax,renameDict1)
fig.suptitle('aSFL expressing linker: E(fluo)')
yt = ax.get_yticks()
ax.set_yticklabels((yt/1000).astype(int))
ax.set_ylabel('E Chadwick (kPa)')

# archiveFig(fig, ax, name='aSFL_iMC_E(fluo)', figDir = todayFigDir, figSubDir='FluoPlots')
fig.show()

#### High expresser experiment, second version of the clone (aSFL A8-2) -- (Sergio's visit september 2021)

In [ ]:
# Outliers = []
# Filters = [(GlobalTable_meca_Py['validatedFit'] == True), (GlobalTable_meca_Py['validatedThickness'] == True),
#            (GlobalTable_meca_Py['drug'].apply(lambda x : x in ['none', 'doxycyclin'])),
#            (GlobalTable_meca_Py['substrate'] == '20um fibronectin discs'), 
#            (GlobalTable_meca_Py['cell subtype'].apply(lambda x : x in ['aSFL', 'aSFL-A8', 'aSFL-A8-2'])),
#            (GlobalTable_meca_Py['cellID'].apply(lambda x : x not in Outliers))]
# co_order = makeOrder(['aSFL', 'aSFL-A8', 'aSFL-A8-2'], ['none', 'doxycyclin'])
# box_pairs=[('aSFL & none', 'aSFL & doxycyclin'),
#  ('aSFL-A8 & none', 'aSFL-A8 & doxycyclin'),
#  ('aSFL-A8-2 & none', 'aSFL-A8-2 & doxycyclin'),
#  ('aSFL & none', 'aSFL-A8 & none'),
#  ('aSFL & none', 'aSFL-A8-2 & none'),
#  ('aSFL-A8 & none', 'aSFL-A8-2 & none')]
# fig, ax = D1Plot(GlobalTable_meca_Py, CondCol=['cell subtype', 'drug'],\
#                  Parameters=['surroundingThickness','EChadwick'], Filters=Filters,\
#                  AvgPerCell=True, cellID='cellID', co_order=co_order, box_pairs = box_pairs, 
#                  figSizeFactor = 1, orientation = 'v', useHue = True)
# ax[0].set_ylim([0, ax[0].get_ylim()[1]])
# renameAxes(ax,renameDict1)
# fig.suptitle('3T3aSFL A8-2')
# # archiveFig(fig, ax, name='3T3aSFL-A8-2_Thickness&EChadwick', figSubDir = figSubDir)
# fig.show()
# # 'ctFieldThickness', 'surroundingThickness', 'none', 'doxycyclin', 

In [ ]:
Outliers = []
Filters = [(GlobalTable_meca_Py['validatedFit'] == True), (GlobalTable_meca_Py['validatedThickness'] == True),
           (GlobalTable_meca_Py['drug'].apply(lambda x : x in ['none', 'doxycyclin'])),
           (GlobalTable_meca_Py['substrate'] == '20um fibronectin discs'), 
           (GlobalTable_meca_Py['cell subtype'].apply(lambda x : x in ['aSFL', 'aSFL-A8-2'])),
           (GlobalTable_meca_Py['cellID'].apply(lambda x : x not in Outliers))]
co_order = makeOrder(['aSFL', 'aSFL-A8-2'], ['none', 'doxycyclin'])
box_pairs=[('aSFL & none', 'aSFL & doxycyclin'),
 ('aSFL-A8-2 & none', 'aSFL-A8-2 & doxycyclin'),
 ('aSFL & none', 'aSFL-A8-2 & none')]
fig, ax = D1Plot(GlobalTable_meca_Py, CondCol=['cell subtype', 'drug'],\
                 Parameters=['surroundingThickness','EChadwick'], Filters=Filters,\
                 AvgPerCell=True, cellID='cellID', co_order=co_order, box_pairs = box_pairs, 
                 figSizeFactor = 1, orientation = 'v', useHue = False)
ax[0].set_ylim([0, ax[0].get_ylim()[1]])
renameAxes(ax,renameDict1)
fig.suptitle('3T3aSFL A8-2')
archiveFig(fig, ax, name='3T3aSFL-A8-2_Thickness&EChadwick', figSubDir = figSubDir)
fig.show()
# 'ctFieldThickness', 'surroundingThickness', 'none', 'doxycyclin', 

In [ ]:
Outliers = []
Filters = [(GlobalTable_meca_Py['validatedFit'] == True), (GlobalTable_meca_Py['validatedThickness'] == True), \
           (GlobalTable_meca_Py['drug'].apply(lambda x : x in ['none', 'doxycyclin'])), \
           (GlobalTable_meca_Py['substrate'] == '20um fibronectin discs'), 
           (GlobalTable_meca_Py['cell subtype'].apply(lambda x : x in ['aSFL-A8-2'])), \
           (GlobalTable_meca_Py['cellID'].apply(lambda x : x not in Outliers))]
fig, ax = D2Plot(GlobalTable_meca_Py, XCol='meanFluoPeakAmplitude', YCol='EChadwick', CondCol = ['cell subtype', 'drug'], \
                 Filters=Filters, cellID = 'cellID', AvgPerCell=True, modelFit=True)

renameAxes(ax,renameDict1)
fig.suptitle('aSFL-A8-2 expressing linker: E(fluo)')
yt = ax.get_yticks()
ax.set_yticklabels((yt/1000).astype(int))
ax.set_ylabel('E Chadwick (kPa)')

archiveFig(fig, ax, name='aSFL-A8-2_iMC_E(fluo)', figDir = todayFigDir, figSubDir='FluoPlots')
fig.show()

In [ ]:
Outliers = []
Filters = [(GlobalTable_meca_Py['validatedFit'] == True), (GlobalTable_meca_Py['validatedThickness'] == True), \
           (GlobalTable_meca_Py['drug'].apply(lambda x : x in ['none', 'doxycyclin'])), \
           (GlobalTable_meca_Py['substrate'] == '20um fibronectin discs'), 
           (GlobalTable_meca_Py['cell subtype'].apply(lambda x : x in ['aSFL'])), \
           (GlobalTable_meca_Py['cellID'].apply(lambda x : x not in Outliers))]
fig, ax = D2Plot(GlobalTable_meca_Py, XCol='meanFluoPeakAmplitude', YCol='EChadwick', CondCol = ['cell subtype', 'drug'], \
                 Filters=Filters, cellID = 'cellID', AvgPerCell=True, modelFit=True)

renameAxes(ax,renameDict1)
fig.suptitle('aSFL expressing linker: E(fluo)')
yt = ax.get_yticks()
ax.set_yticklabels((yt/1000).astype(int))
ax.set_ylabel('E Chadwick (kPa)')

# archiveFig(fig, ax, name='aSFL_iMC_E(fluo)', figDir = todayFigDir, figSubDir='FluoPlots')
fig.show()

In [ ]:
Outliers = []
Filters = [(GlobalTable_meca_Py['validatedFit'] == True), (GlobalTable_meca_Py['validatedThickness'] == True), \
           (GlobalTable_meca_Py['drug'].apply(lambda x : x in ['none', 'doxycyclin'])), \
           (GlobalTable_meca_Py['substrate'] == '20um fibronectin discs'), 
           (GlobalTable_meca_Py['cell subtype'].apply(lambda x : x in ['aSFL', 'aSFL-A8-2'])), \
           (GlobalTable_meca_Py['cellID'].apply(lambda x : x not in Outliers))]
fig, ax = D2Plot(GlobalTable_meca_Py, XCol='meanFluoPeakAmplitude', YCol='EChadwick', CondCol = ['drug'], \
                 Filters=Filters, cellID = 'cellID', AvgPerCell=True, modelFit=True)

renameAxes(ax,renameDict1)
fig.suptitle('{aSFL & aSFL-A8-2 mixed} expressing linker: E(fluo)')
yt = ax.get_yticks()
ax.set_yticklabels((yt/1000).astype(int))
ax.set_ylabel('E Chadwick (kPa)')

# archiveFig(fig, ax, name='aSFL_A11+A8-2_iMC_E(fluo)', figDir = todayFigDir, figSubDir='FluoPlots')
fig.show()

In [ ]:
Filters = [(GlobalTable_meca_Py['validatedFit'] == True), (GlobalTable_meca_Py['validatedThickness'] == True),\
           (GlobalTable_meca_Py['substrate'] == '20um fibronectin discs'), 
           (GlobalTable_meca_Py['drug'].apply(lambda x : x in ['doxycyclin'])), \
           (pd.isna(GlobalTable_meca_Py['meanFluoPeakAmplitude']) != True),
          (GlobalTable_meca_Py['cell subtype'].apply(lambda x : x in ['aSFL','aSFL-6FP','aSFL-A8','aSFL-6FP-2','aSFL-A8-2'])),]
co_order = makeOrder(['aSFL','aSFL-A8','aSFL-6FP','aSFL-A8-2','aSFL-6FP-2'])
box_pairs=[('aSFL', 'aSFL-A8'),
 ('aSFL', 'aSFL-6FP'),
 ('aSFL', 'aSFL-A8-2'),
 ('aSFL', 'aSFL-6FP-2')]
fig, ax = D1Plot(GlobalTable_meca_Py, CondCol=['cell subtype'],Parameters=['meanFluoPeakAmplitude'],\
                 Filters=Filters,AvgPerCell=True,cellID='cellID',co_order=co_order,box_pairs=box_pairs,stats=True,figSizeFactor=1.75)
renameAxes(ax,renameDict1)
fig.suptitle('3T3aSFL - different cell lines expressing the linker\nLinker expression quantification by fluo')
archiveFig(fig, ax, name='3T3aSFL_likerType&_fluoExp_NewVersion', figSubDir = figSubDir)
fig.show()

In [ ]:
Filters = [(GlobalTable_meca_Py['validatedFit'] == True), (GlobalTable_meca_Py['validatedThickness'] == True),\
           (GlobalTable_meca_Py['substrate'] == '20um fibronectin discs'), 
           (GlobalTable_meca_Py['drug'].apply(lambda x : x in ['none', 'doxycyclin'])), \
           (pd.isna(GlobalTable_meca_Py['meanFluoPeakAmplitude']) != True),
          (GlobalTable_meca_Py['cell subtype'].apply(lambda x : x in ['aSFL','aSFL-6FP','aSFL-A8','aSFL-6FP-2','aSFL-A8-2'])),]
co_order = makeOrder(['aSFL','aSFL-A8','aSFL-6FP','aSFL-A8-2','aSFL-6FP-2'], ['none', 'doxycyclin'])
fig, ax = D1Plot(GlobalTable_meca_Py, CondCol=['cell subtype', 'drug'],Parameters=['meanFluoPeakAmplitude'],\
                 Filters=Filters,AvgPerCell=True,cellID='cellID',co_order=co_order,stats=False,figSizeFactor=1.05)
renameAxes(ax,renameDict1)
fig.suptitle('3T3aSFL different cell lines\nLinker expression quantif by fluo')
# archiveFig(fig, ax, name='3T3aSFL_likerType&_fluoExp_NewVersion_2', figSubDir = figSubDir)
fig.show()

In [ ]:
Filters = [(GlobalTable_meca_Py['validatedFit'] == True), (GlobalTable_meca_Py['validatedThickness'] == True),\
           (GlobalTable_meca_Py['substrate'] == '20um fibronectin discs'), 
           (GlobalTable_meca_Py['drug'].apply(lambda x : x in ['none', 'doxycyclin'])), \
           (pd.isna(GlobalTable_meca_Py['meanFluoPeakAmplitude']) != True),
          (GlobalTable_meca_Py['cell subtype'].apply(lambda x : x in ['aSFL','aSFL-6FP-2','aSFL-A8-2'])),]
co_order = makeOrder(['aSFL','aSFL-A8-2','aSFL-6FP-2'], ['none', 'doxycyclin'])
box_pairs=[('aSFL & doxycyclin', 'aSFL-A8-2 & doxycyclin'),
 ('aSFL & doxycyclin', 'aSFL-6FP-2 & doxycyclin')]
fig, ax = D1Plot(GlobalTable_meca_Py, CondCol=['cell subtype', 'drug'],Parameters=['meanFluoPeakAmplitude'],\
                 Filters=Filters,AvgPerCell=True,cellID='cellID',co_order=co_order,box_pairs=box_pairs,stats=True,
                 figSizeFactor=0.95)
renameAxes(ax,renameDict1)
fig.suptitle('3T3aSFL - different cell lines\nLinker expression quantification by fluo')
archiveFig(fig, ax, name='3T3aSFL_likerType&_fluoExp_NewVersion_2', figSubDir = figSubDir)
fig.show()

## Utility scripts

### Create a test table to test different fits

In [ ]:
# Script !


testDir = os.path.join(dataDir, 'TestDataSet')
testFileName = 'testFitCompression.txt'
testFilePath = os.path.join(testDir, testFileName)

list_mecaFiles = [f for f in os.listdir(timeSeriesDataDir) \
                  if (os.path.isfile(os.path.join(timeSeriesDataDir, f)) and f.endswith(".csv") \
                  and ('R40' in f))] # Change to allow different formats in the future
expDf = getExperimentalConditions()
tableDictTest = {}

for f in list_mecaFiles:
    print(f)
    tS_DataFilePath = os.path.join(timeSeriesDataDir, f)
    tsDF = pd.read_csv(tS_DataFilePath, ';')
    
    split_f = f.split('_')
    tsDF.dx, tsDF.dy, tsDF.dz, tsDF.D2, tsDF.D3 = tsDF.dx*1000, tsDF.dy*1000, tsDF.dz*1000, tsDF.D2*1000, tsDF.D3*1000
    thisManipID = split_f[0] + '_' + split_f[1]
    expDf['manipID'] = expDf['date'] + '_' + expDf['manip']
    thisExpDf = expDf.loc[expDf['manipID'] == thisManipID]
    DIAMETER = thisExpDf.at[thisExpDf.index.values[0], 'bead diameter']
    thisCellID = split_f[0] + '_' + split_f[1] + '_' + split_f[2] + '_' + split_f[3]
    Ncomp = max(tsDF['idxCompression'])
    
    for i in range(1, Ncomp+1): #Ncomp+1):
        thisCompHiD = thisCellID + '__' + str(i) + '__h'
        thisCompFiD = thisCellID + '__' + str(i) + '__f'
        print(thisCompHiD)
        
        thisCompDf = tsDF.loc[tsDF['idxCompression'] == i, :]
        iStart = (findFirst(tsDF['idxCompression'], i))
        iStop = iStart + thisCompDf.shape[0]
        
        # Delimit the start of the increase of B (typically the moment when the field decrease from 5 to 3)
        # and the end of its decrease (typically when it goes back from 3 to 5)
        
        listB = thisCompDf.B.values
        offsetStart, offsetStop = 0, 0
        minB, maxB = min(listB), max(listB)
        thresholdB = (maxB-minB)/50
        k = 0
        
        while (listB[k] > minB+thresholdB) or (listB[-1-k] > minB+thresholdB):
            offsetStart += int(listB[k] > minB+thresholdB)
            k += 1
        
        jStart = offsetStart
        jMax = np.argmax(thisCompDf.B)
        
        hCompr = (thisCompDf.D3.values[jStart:jMax+1] - DIAMETER)
        fCompr = (thisCompDf.F.values[jStart:jMax+1])
        
        tableDictTest[thisCompHiD] = hCompr
        tableDictTest[thisCompFiD] = fCompr
        
saveFile = open(testFilePath, 'w')
for k in tableDictTest.keys():
    saveFile.write(k)
    for i in range(len(tableDictTest[k])):
        saveFile.write(';')
        saveFile.write(str(tableDictTest[k][i]))
    saveFile.write('\n')
saveFile.close()

### Create a test table to try statistical tests

In [ ]:
# Script !

# Create a test table to try statistical tests

testDir = os.path.join(dataDir, 'TestDataSet')
GlobalTable_meca = getGlobalTable_meca()
table_ExpConditions = getExperimentalConditions()
table_fluo = getFluoData()
GlobalTable_meca = pd.merge(GlobalTable_meca, table_ExpConditions, how="inner", on='manipID',
#     left_on=None,right_on=None,left_index=False,right_index=False,sort=True,
#     suffixes=("_x", "_y"),copy=True,indicator=False,validate=None,
)
GlobalTable_meca = pd.merge(GlobalTable_meca, table_fluo, how="left", left_on='CellName', right_on='cellID'
#     left_on=None,right_on=None,left_index=False,right_index=False,sort=True,
#     suffixes=("_x", "_y"),copy=True,indicator=False,validate=None,
)
print('Merged table has ' + str(GlobalTable_meca.shape[0]) + ' lines and ' + str(GlobalTable_meca.shape[1]) + ' columns.')

In [ ]:
# Table 1

testFileName = 'testStats01_allComp.csv'
testFilePath = os.path.join(testDir, testFileName)

Filters = [(GlobalTable_meca['Validated'] == 1), (GlobalTable_meca['cell subtype'] == 'aSFL')]
co_order = makeOrder([['BSA coated glass','20um fibronectin discs'],['none','doxycyclin']])
data = GlobalTable_meca
CondCol=['substrate','drug']
Parameters=['SurroundingThickness','EChadwick']
AvgPerCell=False
cellID='CellName'

data_filtered = data
for fltr in Filters:
    data_filtered = data_filtered.loc[fltr]

NCond = len(CondCol)    
if NCond == 1:
    CondCol = CondCol[0]
elif NCond > 1:
    newColName = ''
    for i in range(NCond):
        newColName += CondCol[i]
        newColName += ' & '
    newColName = newColName[:-3]
    data_filtered[newColName] = ''
    for i in range(NCond):
        data_filtered[newColName] += data_filtered[CondCol[i]].astype(str)
        data_filtered[newColName] = data_filtered[newColName].apply(lambda x : x + ' & ')
    data_filtered[newColName] = data_filtered[newColName].apply(lambda x : x[:-3])
    CondCol = newColName
    
if AvgPerCell:
    group = data_filtered.groupby(cellID)
    dictAggMean = getDictAggMean(data_filtered)
    data_filtered = group.agg(dictAggMean)

data_filtered.sort_values(CondCol, axis=0, ascending=True, inplace=True)

df_output = data_filtered[[cellID, 'CompNum', newColName] + Parameters]

# df_output.to_csv(testFilePath)

df_output

In [ ]:
# Table 2

testFileName = 'testStats02_avgPerCell.csv'
testFilePath = os.path.join(testDir, testFileName)

Filters = [(GlobalTable_meca['Validated'] == 1), (GlobalTable_meca['cell subtype'] == 'aSFL')]
co_order = makeOrder([['BSA coated glass','20um fibronectin discs'],['none','doxycyclin']])
data = GlobalTable_meca
CondCol=['substrate','drug']
Parameters=['SurroundingThickness','EChadwick']
AvgPerCell=True
cellID='CellName'

data_filtered = data
for fltr in Filters:
    data_filtered = data_filtered.loc[fltr]

NCond = len(CondCol)    
if NCond == 1:
    CondCol = CondCol[0]
elif NCond > 1:
    newColName = ''
    for i in range(NCond):
        newColName += CondCol[i]
        newColName += ' & '
    newColName = newColName[:-3]
    data_filtered[newColName] = ''
    for i in range(NCond):
        data_filtered[newColName] += data_filtered[CondCol[i]].astype(str)
        data_filtered[newColName] = data_filtered[newColName].apply(lambda x : x + ' & ')
    data_filtered[newColName] = data_filtered[newColName].apply(lambda x : x[:-3])
    CondCol = newColName
    
if AvgPerCell:
    group = data_filtered.groupby(cellID)
    dictAggMean = getDictAggMean(data_filtered)
    data_filtered = group.agg(dictAggMean)

data_filtered.sort_values(CondCol, axis=0, ascending=True, inplace=True)

df_output = data_filtered[[cellID, 'CompNum', newColName] + Parameters]

# df_output.to_csv(testFilePath)

df_output

In [ ]:
# Table 3
# Fake data

testDir = os.path.join(dataDir, 'TestDataSet')

testFileName = 'testStats03_FakeData.csv'
testFilePath = os.path.join(testDir, testFileName)

Npop = 10
Npoints = 30
minAvg = 100
maxAvg = 600
step = (maxAvg - minAvg)/(Npop-1)
std = 250
dictFakeData = {}
np.random.seed(11)

for i in [1, 2, 3, 10]:
    dictFakeData['Distribution_' + str(i)] = np.random.normal(loc=minAvg + step*(i-1), scale=std, size=Npoints)

dfFakeData = pd.DataFrame(dictFakeData)

# dfFakeData.to_csv(testFilePath)

dfFakeData

In [ ]:
# Table 4
# Fake data 2

testDir = os.path.join(dataDir, 'TestDataSet')

testFileName = 'testStats04_FakeDataLarge.csv'
testFilePath = os.path.join(testDir, testFileName)

Npop = 10
Npoints = 300
minAvg = 100
maxAvg = 600
step = (maxAvg - minAvg)/(Npop-1)
std = 250
dictFakeData = {}
np.random.seed(11)

for i in [1, 2, 3, 10]:
    dictFakeData['Distribution_' + str(i)] = np.random.normal(loc=minAvg + step*(i-1), scale=std, size=Npoints)

dfFakeData = pd.DataFrame(dictFakeData)

# dfFakeData.to_csv(testFilePath)

dfFakeData

### Comparison of stat tests

#### With the fake data

In [ ]:
testDir = os.path.join(dataDir, 'TestDataSet')

# Table 3
# Fake data
testFileName = 'testStats03_FakeData.csv'
testFilePath = os.path.join(testDir, testFileName)

dfFakeData = pd.read_csv(testFilePath)
dfFakeData = dfFakeData.drop(columns=['Unnamed: 0'])

Ncol = len(dfFakeData.columns)

refCol = dfFakeData[dfFakeData.columns[0]]
boxPlotMatrix = []

for i in range(0,Ncol):
    boxPlotMatrix.append(dfFakeData[dfFakeData.columns[i]].values)
    if i > 0:
        print('Comparison between distribution 1 and ' + str(i+1))
        tTest = st.ttest_ind(refCol.values, dfFakeData[dfFakeData.columns[i]].values)
        print('tTest : ' + str(tTest.pvalue))
        wilcox = st.wilcoxon(refCol.values, dfFakeData[dfFakeData.columns[i]].values)
        print('wilcox : ' + str(wilcox.pvalue))
        mannwhitneyu = st.mannwhitneyu(refCol.values, dfFakeData[dfFakeData.columns[i]].values)
        print('mannwhitneyu : ' + str(mannwhitneyu.pvalue))
        print('')

fig, ax = plt.subplots(1,1)
ax.boxplot(boxPlotMatrix,labels=dfFakeData.columns.values)
ax.tick_params(axis='x', labelrotation = 15, labelsize = 7)
fig.show()

In [ ]:
testDir = os.path.join(dataDir, 'TestDataSet')

# Table 4
# Fake data Large
testFileName = 'testStats04_FakeDataLarge.csv'
testFilePath = os.path.join(testDir, testFileName)

dfFakeData = pd.read_csv(testFilePath)
dfFakeData = dfFakeData.drop(columns=['Unnamed: 0'])

Ncol = len(dfFakeData.columns)

refCol = dfFakeData[dfFakeData.columns[0]]
boxPlotMatrix = []

for i in range(0,Ncol):
    boxPlotMatrix.append(dfFakeData[dfFakeData.columns[i]].values)
    if i > 0:
        print('Comparison between distribution 1 and ' + str(i+1))
        tTest = st.ttest_ind(refCol.values, dfFakeData[dfFakeData.columns[i]].values)
        print('tTest : ' + str(tTest.pvalue))
        wilcox = st.wilcoxon(refCol.values, dfFakeData[dfFakeData.columns[i]].values)
        print('wilcox : ' + str(wilcox.pvalue))
        mannwhitneyu = st.mannwhitneyu(refCol.values, dfFakeData[dfFakeData.columns[i]].values)
        print('mannwhitneyu : ' + str(mannwhitneyu.pvalue))
        print('')

fig, ax = plt.subplots(1,1)
ax.boxplot(boxPlotMatrix,labels=dfFakeData.columns.values)
ax.tick_params(axis='x', labelrotation = 15, labelsize = 7)
fig.show()

In [ ]:
# Summary of results: numpy.random.seed = 11
    
dictResultsFakeData = {}
dictResultsFakeData['language'] = ['python'   , 'python', 'python'      , 'R'     , 'R'          , 'Matlab' , 'Matlab' ]
dictResultsFakeData['test']     = ['ttest_ind', 'wilcox', 'mannwhitneyu', 't.test', 'wilcox.test', 'ttest2' , 'ranksum']
dictResultsFakeData['1 vs 2']   = [0.2700     , 0.2711  , 0.1353        , 0.2701  , 0.2729       , 0.2700   , 0.2707   ]
dictResultsFakeData['1 vs 3']   = [0.0714     , 0.0822  , 0.0452        , 0.0715  , 0.0906       , 0.0714   , 0.0905   ]
dictResultsFakeData['1 vs 10']  = [1.33e-11   , 4.28e-06, 2.98e-09      , 1.44e-11, 5.799e-11    , 1.33e-11 , 5.96e-09 ]
dfResultFakeData = pd.DataFrame(dictResultsFakeData)
dfResultFakeData

In [ ]:
# Summary of results: numpy.random.seed = 11

dictResultsFakeDataLarge = {}
dictResultsFakeDataLarge['language'] = ['python'   , 'python', 'python'      , 'R'     , 'R'          , 'Matlab' , 'Matlab' ]
dictResultsFakeDataLarge['test']     = ['ttest_ind', 'wilcox', 'mannwhitneyu', 't.test', 'wilcox.test', 'ttest2' , 'ranksum']
dictResultsFakeDataLarge['1 vs 2']   = [0.0082     , 0.0049  , 0.0038        , 0.0082  , 0.0077       , 0.0082   , 0.0077   ]
dictResultsFakeDataLarge['1 vs 3']   = [1.26e-06   , 9.29e-06, 1.37e-06      , 1.27e-06, 2.75e-06     , 1.26e-06 , 2.74e-06 ]
dictResultsFakeDataLarge['1 vs 10']  = [1.74e-98   , 5.64e-48, 6.39e-74      , 2.2e-16 , 2.2e-16      , 1.74e-98 , 1.27e-73 ]
dictResultsFakeDataLarge = pd.DataFrame(dictResultsFakeDataLarge)
dictResultsFakeDataLarge

#### With real data

In [ ]:
# testDir = os.path.join(dataDir, 'TestDataSet')

# Table 1
# Avg Per Cell
testFileName = 'testStats01_AvgPerCell.csv'
testFilePath = os.path.join(testDir, testFileName)

dfAvgPerCell = pd.read_csv(testFilePath)
# dfAvgPerCell = dfAvgPerCell.drop(columns=['Unnamed: 0'])
dfAvgPerCell
categories = list(dfAvgPerCell['substrate & drug'].unique())
Ncat = len(categories)
for i in range(Ncat):
    for j in range(i,Ncat):
        if j != i:
            x = dfAvgPerCell.loc[dfAvgPerCell['substrate & drug'] == categories[i], 'EChadwick'].values
            y = dfAvgPerCell.loc[dfAvgPerCell['substrate & drug'] == categories[j], 'EChadwick'].values
            print('Comparison between ' + categories[i] + ' and ' + categories[j])
            tTest = st.ttest_ind(x, y)
            print('tTest : ' + str(tTest.pvalue))
            mannwhitneyu = st.mannwhitneyu(x, y)
            print('mannwhitneyu : ' + str(mannwhitneyu.pvalue))
            print('')

# refCol = dfFakeData[dfFakeData.columns[0]]
# boxPlotMatrix = []

# for i in range(0,Ncol):
#     boxPlotMatrix.append(dfFakeData[dfFakeData.columns[i]].values)
#     if i > 0:
#         print('Comparison between distribution 1 and ' + str(i+1))
#         tTest = st.ttest_ind(refCol.values, dfFakeData[dfFakeData.columns[i]].values)
#         print('tTest : ' + str(tTest.pvalue))
#         wilcox = st.wilcoxon(refCol.values, dfFakeData[dfFakeData.columns[i]].values)
#         print('wilcox : ' + str(wilcox.pvalue))
#         mannwhitneyu = st.mannwhitneyu(refCol.values, dfFakeData[dfFakeData.columns[i]].values)
#         print('mannwhitneyu : ' + str(mannwhitneyu.pvalue))
#         print('')

# fig, ax = plt.subplots(1,1)
# ax.boxplot(boxPlotMatrix,labels=dfFakeData.columns.values)
# ax.tick_params(axis='x', labelrotation = 15, labelsize = 7)
# fig.show()

In [ ]:
dictResultsAvgPerCell = {}
order = ['language','test','c & NA vs iMC & A','c & NA vs c & A','iMC & NA vs iMC & A','c & NA vs iMC & NA','iMC & NA vs c & A','c & A vs iMC & A']
dictResultsAvgPerCell['language']            = ['python_statAnot'   , 'python_statAnot' ,'python'   , 'python', 'R'      , 'R'          , 'Matlab' , 'Matlab' ]
dictResultsAvgPerCell['test']                = ['ttest_ind', 'mannwhitneyu', 'ttest_ind', 'mannwhitneyu', 't.test' , 'wilcox.test', 'ttest2' , 'ranksum']
dictResultsAvgPerCell['c & NA vs iMC & NA']  = [1.000e+00  , 5.669e-01     ,0.19226082553146542  , 0.047244130659518     , 0.1503   , 0.09502       , 0.1923       , 0.0945]
dictResultsAvgPerCell['iMC & NA vs c & A']   = [9.673e-02  , 8.625e-03     ,0.016121864893694285  ,0.0007187494204219925     , 0.04082  , 0.0009726    , 0.0161     , 0.0014]
dictResultsAvgPerCell['c & A vs iMC & A']    = [2.331e-02  , 1.376e-01     ,0.00388458593467288  , 0.01146586677766893     , 0.007326 , 0.02214      , 0.0039      , 0.0229]
dictResultsAvgPerCell['c & NA vs c & A']     = [1.000e+00  , 1.000e+00     ,0.6977550928576132  , 0.2884535746840493     , 0.6948   , 0.5838       , 0.6978     , 0.5769]
dictResultsAvgPerCell['iMC & NA vs iMC & A'] = [1.000e+00  , 1.000e+00     ,0.5573451346686198  , 0.41831870120029446,  0.5031  , 0.8387       , 0.5573      , 0.8366 ]
dictResultsAvgPerCell['c & NA vs iMC & A']   = [9.726e-01  , 1.000e+00     ,0.16209530366557973  , 0.14893352365754048     , 0.04353  , 0.3043       , 0.1621       , 0.2979 ]
dfResultsAvgPerCell = pd.DataFrame(dictResultsAvgPerCell)
dfResultsAvgPerCell[order]

# dfResultsAvgPerCell = dfResultsAvgPerCell.sort_values(by='test',ascending=False)
# dfResultsAvgPerCell

### Plot small stuff

In [ ]:
SMALLER_SIZE = 6
SMALL_SIZE = 6
MEDIUM_SIZE = 10
BIGGER_SIZE = 10

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=MEDIUM_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALLER_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

F0, F1 = 500, 250
h0, h1 = 200, 50
om = 2*np.pi
delta = 1
T = np.arange(0,3,0.01)
F = F1 * np.sin(2*np.pi*T) + F0
h = h1 * np.sin(2*np.pi*T - delta) + h0

fig, ax = plt.subplots(figsize = (4,2))
color = 'firebrick'
ax.plot(T, F, color = color, lw = 1)
ax.plot([np.min(T), np.max(T)], [F0, F0], 'k--', lw = 1)
ax.set_xlabel('t (s)')
ax.set_ylabel('F (pN)', color=color)
ax.tick_params(axis='y', labelcolor=color)

axbis = ax.twinx()
color = 'blue'
axbis.set_ylabel('h (nm)', color=color)
axbis.plot(T, h, color=color, lw = 1)
axbis.tick_params(axis='y', labelcolor=color)
# axbis.set_yticks([0,500,1000,1500])
# minh = np.min(tsDF['D3'].values-DIAMETER)
# ratio = min(1/abs(minh/axM), 5)
# #             print(ratio)
#             (axmbis, axMbis) = ax1bis.get_ylim()
#             ax1bis.set_ylim([0, max(axMbis*ratio, 3*max(tsDF['F'].values))])

## Old Code

### Tweaked version of analyseTimeSeries_meca to plot specific stuff

In [ ]:
def analyseTimeSeries_meca_tweaked(f, tsDF, expDf, listColumnsMeca, PLOT, PLOT_SHOW):
    
    split_f = f.split('_')
    tsDF.dx, tsDF.dy, tsDF.dz, tsDF.D2, tsDF.D3 = tsDF.dx*1000, tsDF.dy*1000, tsDF.dz*1000, tsDF.D2*1000, tsDF.D3*1000
    thisManipID = split_f[0] + '_' + split_f[1]
    expDf['manipID'] = expDf['date'] + '_' + expDf['manip']
    thisExpDf = expDf.loc[expDf['manipID'] == thisManipID]
    DIAMETER = thisExpDf.at[thisExpDf.index.values[0], 'bead diameter']
    
    results = {}
    for c in listColumnsMeca:
        results[c] = []
        
    Ncomp = max(tsDF['idxCompression'])
    NimgComp = np.sum((tsDF['idxCompression'] != 0))/Ncomp
    NimgCompTh = round(0.49999999999 + np.sum((tsDF['idxCompression'] != 0))/Ncomp)
    NimgBtwComp = np.sum((tsDF['idxCompression'] == 0))/Ncomp
    NimgBtwCompTh = round(0.4999999999 + np.sum((tsDF['idxCompression'] == 0))/Ncomp)
#     print('Ncomp : ' + str(Ncomp) + ' ; ' + 'NimgComp : ' + str(NimgComp) + '/' + str(NimgCompTh) + ' ; ' + 'NimgBtwComp : ' + str(NimgBtwComp) + '/' + str(NimgBtwCompTh))
#     if not NimgBtwComp%2 == 0:
#         print('Bug with the compressions sequence delimitation')

    # These values are computed once for the whole cell D3 time series, but since the table has 1 line per compression, 
    # that same value will be put in the table for each line corresponding to that cell
    ctFieldH = (tsDF.loc[tsDF['idxCompression'] == 0, 'D3'].values - DIAMETER)
    ctFieldThickness   = np.median(ctFieldH)
    ctFieldFluctuAmpli = np.percentile(ctFieldH,90) - np.percentile(ctFieldH,10)
    
    # First part of the plot [mainly ax1 and ax1bis]
    if PLOT:
        nColsSubplot = 5
        nRowsSubplot = ((Ncomp-1) // nColsSubplot) + 1
        fig1, ax1 = plt.subplots(1,1,figsize=(tsDF.shape[0]*(1/100)*0.25,4))
#         fig2, ax2 = plt.subplots(1,Ncomp,figsize=(3*Ncomp,3))
        fig2, ax2 = plt.subplots(nRowsSubplot,nColsSubplot,figsize=(3*nColsSubplot,3*nRowsSubplot))
        
        color = 'blue'
#         ax1.plot(tsDF['T'].values, tsDF['D3'].values-DIAMETER, color = color, ls = '-', linewidth = 1)
        ax1.set_xlabel('t (s)')
#         ax1.set_ylabel('h (nm)', color=color)
#         ax1.tick_params(axis='y', labelcolor=color)
#         (axm, axM) = ax1.get_ylim()
#         ax1.set_ylim([min(0,axm), axM-50])
        ax1.set_xlim([0, 45])
#         if (max(tsDF['D3'].values-DIAMETER) > 200):
#             ax1.set_yticks(np.arange(0, max(tsDF['D3'].values-DIAMETER), 100))
        ax1.set_yticks([])
        
        twinAxis = True
        if twinAxis:
#             ax1.tick_params(axis='y', labelcolor='b')
            ax1bis = ax1.twinx()
            color = 'purple'
            ax1bis.set_ylabel('B (mT)', color=color)
            ax1bis.plot(tsDF['T'].values, tsDF['B'].values, color=color)
            ax1bis.tick_params(axis='y', labelcolor=color)
            ax1bis.set_yticks([0,3,6,10,20,30,40])
#             ax1bis.set_yticks([0,500,1000,1500])
#             minh = np.min(tsDF['D3'].values-DIAMETER)
#             ratio = min(1/abs(minh/axM), 5)
#             print(ratio)
#             (axmbis, axMbis) = ax1bis.get_ylim()
#             ax1bis.set_ylim([0, max(axMbis*ratio, 3*max(tsDF['F'].values))])
            yt = ax1bis.get_yticklabels()
            for item in yt:
                item.set_fontsize(11)
            
    
    for i in range(1, Ncomp+1):#Ncomp+1):

        # (1) Identifiers
        results['date'].append(split_f[0])
        results['cellName'].append(split_f[1] + '_' + split_f[2] + '_' + split_f[3])
        results['cellID'].append(split_f[0] + '_' + split_f[1] + '_' + split_f[2] + '_' + split_f[3])
        results['manipID'].append(split_f[0] + '_' + split_f[1])
        
        # (2) Segment the compression n°i
        thisCompDf = tsDF.loc[tsDF['idxCompression'] == i,:]
        iStart = (findFirst(tsDF['idxCompression'], i))
        iStop = iStart+thisCompDf.shape[0]
        
        # Easy-to-get parameters
        results['compNum'].append(i)
        results['compDuration'].append(thisExpDf.at[thisExpDf.index.values[0], 'compression duration'])
        results['compStartTime'].append(thisCompDf['T'].values[0])
        results['compAbsStartTime'].append(thisCompDf['Tabs'].values[0])
        results['compStartTimeThisDay'].append(thisCompDf['Tabs'].values[0]) # Will be modifyed later !
        
        # (3) Inside the compression n°i, delimit the compression and relaxation phases
        
        # Delimit the start of the increase of B (typically the moment when the field decrease from 5 to 3)
        # and the end of its decrease (typically when it goes back from 3 to 5)
        listB = thisCompDf.B.values
        offsetStart, offsetStop = 0, 0
        minB, maxB = min(listB), max(listB)
        thresholdB = (maxB-minB)/50
        
        k = 0
        while (listB[k] > minB+thresholdB) or (listB[-1-k] > minB+thresholdB):
            offsetStart += int(listB[k] > minB+thresholdB)
            offsetStop += int(listB[-1-k] > minB+thresholdB)
            k += 1
        jStart = offsetStart # Beginning of compression
        jMax = np.argmax(thisCompDf.B) # End of compression, beginning of relaxation
        jStop = thisCompDf.shape[0] - offsetStop # End of relaxation
        
        # Four arrays
        hCompr = (thisCompDf.D3.values[jStart:jMax+1] - DIAMETER)
        hRelax = (thisCompDf.D3.values[jMax+1:jStop] - DIAMETER)
        fCompr = (thisCompDf.F.values[jStart:jMax+1])
        fRelax = (thisCompDf.F.values[jMax+1:jStop])
        
        # Refinement of the compression delimitation.
        # Remove the 1-2 points at the begining where there is just the viscous relaxation of the cortex
        # because of the initial decrease of B and the cortex thickness increases.
        offsetStart2 = 0
        k = 0
        while (hCompr[k] < np.max(hCompr[k+1:min(k+10, len(hCompr))])) and k<len(hCompr)-10:
            offsetStart2 += 1
            k += 1
        # Better compressions arrays
        hCompr = hCompr[offsetStart2:]
        fCompr = fCompr[offsetStart2:]
        
        # Get the points of constant field preceding and surrounding the current compression
        # Ex : if the labview code was set so that there is 6 points of ct field before and after each compression,
        # previousPoints will contains D3[iStart-12:iStart]
        # surroundingPoints will contains D3[iStart-6:iStart] and D3[iStop:iStop+6]
        previousPoints = (tsDF.D3.values[max(0,iStart-(NimgBtwCompTh)):iStart]) - DIAMETER
        surroundingPoints = np.concatenate([tsDF.D3.values[max(0,iStart-(NimgBtwCompTh//2)):iStart],tsDF.D3.values[iStop:iStop+(NimgBtwCompTh//2)]]) - DIAMETER
        
        # Parameters relative to the thickness ( = D3-DIAMETER)
        results['initialThickness'].append(np.mean(hCompr[0:3]))
        results['minThickness'].append(np.min(hCompr))
        results['maxIndent'].append(results['initialThickness'][-1] - results['minThickness'][-1])
        results['previousThickness'].append(np.median(previousPoints))
        results['surroundingThickness'].append(np.median(surroundingPoints))
        results['ctFieldThickness'].append(ctFieldThickness)
        results['ctFieldFluctuAmpli'].append(ctFieldFluctuAmpli)
        
        validatedThickness = np.min([results['initialThickness'],results['minThickness'],results['previousThickness'],\
                                    results['surroundingThickness'],results['ctFieldThickness']]) > 0
        results['validatedThickness'].append(validatedThickness)

        # (4) Fit with Chadwick model of the force-thickness curve
        
        E, H0, hPredict, R2, confIntE, confIntH0, fitError = compressionFitChadwick(hCompr, fCompr, DIAMETER) # IMPORTANT SUBFUNCTION
        
        R2CRITERION = 0.9
        critFit = 'R2 > ' + str(R2CRITERION)
        results['critFit'].append(critFit)
        validatedFit = (R2 > R2CRITERION)
        
        if PLOT:

            # fig1
#             if not fitError:
#                 if validatedFit:
#                     ax1.plot(thisCompDf['T'].values, thisCompDf['D3'].values-DIAMETER, color = 'chartreuse', linestyle = '-', linewidth = 1.25)
#                 else:
#                     ax1.plot(thisCompDf['T'].values, thisCompDf['D3'].values-DIAMETER, color = 'gold', linestyle = '-', linewidth = 1.25)
#             else:
#                 ax1.plot(thisCompDf['T'].values, thisCompDf['D3'].values-DIAMETER, color = 'crimson', linestyle = '-', linewidth = 1.25)

            
            fig1.suptitle(results['cellID'][-1])
            
            # fig2
            colSp = (i-1) % nColsSubplot
            rowSp = (i-1) // nColsSubplot
            # ax2[i-1] with the 1 line plot
            if nRowsSubplot == 1:
                thisAx2 = ax2[colSp]
            elif nRowsSubplot >= 1:
                thisAx2 = ax2[rowSp,colSp]
            
            thisAx2.plot(hCompr,fCompr,'b-', linewidth = 0.8)
            thisAx2.plot(hRelax,fRelax,'r-', linewidth = 0.8)
            titleText = results['cellID'][-1] + '__c' + str(i)
            legendText = ''
            thisAx2.set_xlabel('h (nm)')
            thisAx2.set_ylabel('f (pN)')
            
            if not fitError:
                legendText += 'H0 = {:.1f}nm\nE = {:.2e}Pa\nR2 = {:.3f}'.format(H0, E, R2)
                thisAx2.plot(hPredict,fCompr,'k--', linewidth = 0.8, label = legendText)
                thisAx2.legend(loc = 'upper right', prop={'size': 6})
                if not validatedFit:
                    titleText += '\nNON VALIDATED'
            else:
                titleText += '\nFIT ERROR'
            
            thisAx2.title.set_text(titleText)

            for item in ([thisAx2.title, thisAx2.xaxis.label, \
                          thisAx2.yaxis.label] + thisAx2.get_xticklabels() + thisAx2.get_yticklabels()):
                item.set_fontsize(9)
                

        if not fitError:
            confIntEWidth = abs(confIntE[0] - confIntE[1])

            results['H0Chadwick'].append(H0)
            results['EChadwick'].append(E)
            results['R2Chadwick'].append(R2)
            results['EChadwick_CIWidth'].append(confIntEWidth)
            

            results['validatedFit'].append(validatedFit)
            if validatedFit:
                results['comments'].append('ok')
            else:
                results['comments'].append('R2 < ' + str(R2CRITERION))
                
        if fitError:
            validatedFit = False
            results['H0Chadwick'].append(np.nan)
            results['EChadwick'].append(np.nan)
            results['R2Chadwick'].append(np.nan)
            results['EChadwick_CIWidth'].append(np.nan)
            results['validatedFit'].append(validatedFit)
            results['comments'].append('fitFailure')
            
        
        # (5) hysteresis (its definition may change)
        results['hysteresis'].append(hCompr[0] - hRelax[-1])
    
    if PLOT:
        archiveFig(fig1, ax1, name=results['cellID'][-1] + '_h(t)', figSubDir = 'MecaAnalysis_allCells')
        archiveFig(fig2, ax2, name=results['cellID'][-1] + '_F(h)', figSubDir = 'MecaAnalysis_allCells')
        if PLOT_SHOW:
            fig1.show()
            fig2.tight_layout()
            fig2.show()
        else:
            plt.close('all')
    
    return(results)

### Data formatting and filtering

TBC

In [ ]:
mecaDataFile = 'Global_MecaData.csv'
mecaDataFilePath = os.path.join(dataDir, mecaDataFile)
mecaDF = pd.read_csv(mecaDataFilePath, sep=';')
print('Extracted a table with ' + str(mecaDF.shape[0]) + ' lines and ' + str(mecaDF.shape[1]) + ' columns.')

mecaDF = mecaDF.rename(columns={"CellID": "CellName", "CellName": "CellID"})

In [ ]:
# mecaDF

In [ ]:
experimentalDataFile = 'ExperimentalConditions.csv'
experimentalDataFilePath = os.path.join(experimentalDataDir, experimentalDataFile)
expConditionsDF = pd.read_csv(experimentalDataFilePath, sep=';',header=0)
print('Extracted a table with ' + str(expConditionsDF.shape[0]) + ' lines and ' + str(expConditionsDF.shape[1]) + ' columns.')

# Cleaning the table
try:
    expConditionsDF = expConditionsDF.convert_dtypes()

    listTextColumns = []
    for col in expConditionsDF.columns:
        if expConditionsDF[col].dtype == 'string':
            listTextColumns.append(col)

    expConditionsDF[listTextColumns] = expConditionsDF[listTextColumns].apply(lambda x: x.str.replace(',','.'))

    expConditionsDF['scale pixel per um'] = expConditionsDF['scale pixel per um'].astype(float)
    expConditionsDF['optical index correction'] = \
              expConditionsDF['optical index correction'].apply(lambda x: x.split('/')[0]).astype(float) \
            / expConditionsDF['optical index correction'].apply(lambda x: x.split('/')[1]).astype(float)
    expConditionsDF['magnetic field correction'] = expConditionsDF['magnetic field correction'].astype(float)
    expConditionsDF['with fluo images'] = expConditionsDF['with fluo images'].astype(bool)

    expConditionsDF['ramp field'] = \
    expConditionsDF['ramp field'].apply(lambda x: [x.split(';')[0], x.split(';')[1]] if not pd.isnull(x) else [])

except:
    print('Unexpected bug with the cleaning step')

In [ ]:
expConditionsDF

In [ ]:
# Unused for now
cellDescriptionDataFile = 'CellDescription.csv'
cellDescriptionDataFilePath = os.path.join(experimentalDataDir, cellDescriptionDataFile)
cellDescriptionDF = pd.read_csv(cellDescriptionDataFilePath, ',')
print('Extracted a table with ' + str(cellDescriptionDF.shape[0]) + ' lines and ' + str(cellDescriptionDF.shape[1]) + ' columns.')

In [ ]:
mecaDF['ManipID'] = mecaDF['ExpDay'] + '_' + mecaDF['CellName'].apply(lambda x: x.split('_')[0])
expConditionsDF['ManipID'] = expConditionsDF['date'] + '_' + expConditionsDF['manip']

mainMecaDF = pd.merge(
    expConditionsDF,
    mecaDF,
    how="inner",
    on='ManipID',
#     left_on=None,
#     right_on=None,
#     left_index=False,
#     right_index=False,
#     sort=True,
#     suffixes=("_x", "_y"),
#     copy=True,
#     indicator=False,
#     validate=None,
)

In [ ]:
# pd.set_option('display.max_columns', None)
# mainMecaDF.head()

In [ ]:
pd.reset_option('max_columns')

#### * Data filtering

In [ ]:
# pd.set_option('display.max_columns', None)
# mainMecaDF.head()

In [ ]:
# pd.reset_option('max_columns')
# mainMecaDF.columns

In [ ]:
mainMecaDF_f = mainMecaDF.loc[(mainMecaDF["Validated"] == 1)]
# mainMecaDF_f

In [ ]:
%matplotlib inline
listCells = mainMecaDF_f['CellID'].drop_duplicates().astype('string').values
timeSeriesDict = {}
for cell in listCells:
    currentCell_TimeSeriesData = getCellTimeSeriesData(cell)
    timeSeriesDict[cell] = currentCell_TimeSeriesData
start, stop = 80, 100
fig, axes = plt.subplots((stop-start),1, figsize = (7,4*(stop-start)))
fig.tight_layout()
for k in range(start, stop):
    if k < len(listCells):
        currentCell_TimeSeriesData = timeSeriesDict[listCells[k]]
        T = currentCell_TimeSeriesData['T'].values
        idxCompression = currentCell_TimeSeriesData['idxCompression'].values
        D3 = currentCell_TimeSeriesData['D3'].values
        maskConstant = (idxCompression == 0)
        maskCompression = (idxCompression > 0)
        axes[k - start].plot(T, D3*1000-4503, 'k-', linewidth = 0.5)
        axes[k - start].plot(T[maskCompression], D3[maskCompression]*1000-4503, 'ro', markersize=2)
        axes[k - start].plot(T[maskConstant], D3[maskConstant]*1000-4503, 'co', markersize=2)
        axes[k - start].set_title(listCells[k])
        axes[k - start].set_xlabel('T (s)')
        axes[k - start].set_ylabel('D3 (µm)')

In [ ]:
addExcludedCell('21-01-18_M1_P1_C2', 'passive')
addExcludedCell('21-01-18_M1_P1_C3', 'passive')
addExcludedCell('21-01-18_M1_P1_C4', 'passive')
addExcludedCell('21-01-21_M3_P1_C4', 'passive')
addExcludedCell('21-01-21_M3_P1_C5', 'passive')
addExcludedCell('20-08-07_M1_P1_C6', 'too thick')
addExcludedCell('20-08-07_M1_P1_C62', 'too thick')

excludedCellsDict = getExcludedCells()
# # excludedMask = (mainMecaDF_f["CellID"].values not in excludedCellsDict.keys())
# # mainMecaDF_f = mainMecaDF_f.loc[(mainMecaDF_f["CellID"].values not in excludedCellsDict.keys())]
# for i in range(len(excludedCellsDict)):
#     print('a')
# mainMecaDF_f["CellID"].drop_duplicates().astype('string').values
excludedCellsDict

In [ ]:
# currentCell_TimeSeriesData

In [ ]:
mainMecaDF_GroupedPerCell = mainMecaDF_f.groupby('CellID')
mainMecaDF_DataPerCell = mainMecaDF_GroupedPerCell.agg({"EChadwick": np.median, "SurroundingThickness": np.median, "H0Chadwick" : np.median})
# mainMecaDF_GroupedPerCell.agg({"EChadwick": np.median, "D": lambda x: np.std(x, ddof=1)})
cols = ['date', 'manip', 'experimentType', 'drug', 'substrate',
       'objective magnification', 'scale pixel per um', 'objective immersion',
       'optical index correction', 'magnetic field correction', 'cell type',
       'cell subtype', 'bead type', 'bead diameter', 'normal field',
       'ramp field', 'compression duration', 'with fluo images', 'comments',
       'ManipID', 'ExpType', 'CellName', 'CellID']
mainMecaDF_DataPerCell.dropna(inplace = True)
mainMecaDF_DataPerCell = pd.merge(mainMecaDF_DataPerCell,
                                  mainMecaDF_f[cols].drop_duplicates(subset=['CellID']),
                                  how="inner",
                                  on='CellID',
                                  #     left_on='CellID',
                                  #     right_on='CellID',
                                  #     left_index=False,
                                  #     right_index=False,
                                  #     sort=True,
                                  #     suffixes=("_x", "_y"),
                                  #     copy=True,
                                  #     indicator=False,
                                  #     validate=None,
                                  )
# mainMecaDF_DataPerCell

In [ ]:
GlobalTable_meca_Count = GlobalTable_meca.groupby(['cell type', 'cell subtype', 'bead type', 'drug', 'substrate']).count()
GlobalTable_meca_Count.loc[:, ['CellID']].rename(columns={'CellID' : 'Count'})

### Other old code

In [ ]:
# fig, axes = plt.subplots(1,2, figsize = (8, 5))
# axes[0].plot(np.ones(len(resDict['nodrug']['median'])), resDict['nodrug']['median'], 'co')
# axes[0].plot(2*np.ones(len(resDict['doxy']['median'])), resDict['doxy']['median'], 'ro')
# axes[0].set_xlim(0.5, 2.5)
# axes[0].set_ylabel('Median Thickness (nm)')
# axes[0].set_xticks([1,2])
# axes[0].set_xticklabels(['Control','Doxycylin'])
# axes[1].plot(np.ones(len(resDict['nodrug']['fluctu'])), resDict['nodrug']['fluctu'], 'co')
# axes[1].plot(2*np.ones(len(resDict['doxy']['fluctu'])), resDict['doxy']['fluctu'], 'ro')
# axes[1].set_xlim(0.5, 2.5)
# axes[1].set_ylabel('Thickness Fluctuations (nm)')
# axes[1].set_xticks([1,2])
# axes[1].set_xticklabels(['Control','Doxycylin'])
# fig.savefig("C://Users//JosephVermeil//Desktop//ActinCortexAnalysis//DataAnalysis//constantField.png")
# fig.show()

In [ ]:
# Old code
conditions = ['nodrug', 'doxy']
correspondance = {conditions[0] : 'M1', conditions[1] : 'M2'}
allTimeSeriesDataFiles = [f for f in os.listdir(timeSeriesDataDir) if (os.path.isfile(os.path.join(timeSeriesDataDir, f)) and f.endswith(".txt"))]
dates = ['21-02-10']
resDict = {conditions[0] : {}, conditions[1] : {}}
for C in conditions:
    resDict[C]['accepted'] = []
    resDict[C]['rejected'] = []
    resDict[C]['median'] = []
    resDict[C]['fluctu'] = []
    for D in dates:
        for f in allTimeSeriesDataFiles:
            if correspondance[C] in f and D in f:
                split_f = f.split('_')
                cellID = split_f[0] + '_' + split_f[1] + '_' + split_f[2] + '_' + split_f[3]
                currentCellTS = getCellTimeSeriesData(cellID)
                D3 = currentCellTS.D3.values
                decile_1 = np.percentile(D3, 10)
                median = np.median(D3)
                decile_9 = np.percentile(D3, 90)
                if decile_1 < 0:
                    resDict[C]['rejected'].append(cellID)
                else:
                    resDict[C]['accepted'].append(cellID)
                    resDict[C]['median'].append(median)
                    resDict[C]['fluctu'].append(decile_9-decile_1)
#resDict

In [ ]:
GlobalTable_ctField = getGlobalTable_ctField()
table_ExpConditions = getExperimentalConditions()
GlobalTable_ctField = pd.merge(table_ExpConditions, GlobalTable_ctField, how="inner", on='manipID',
#     left_on=None,right_on=None,left_index=False,right_index=False,sort=True,
#     suffixes=("_x", "_y"),copy=True,indicator=False,validate=None,
)

# data=pd.DataFrame(dict(
#             x=[1, 2, 3, 4, 5],
#             y=[2, 5, 8, 2, 7],
#             desc=['A', 'A', 'C', 'd', 'E'],
#         ))

data = GlobalTable_ctField[['medianThickness','fluctuAmpli','cellID']]

source = ColumnDataSource(
        data=data
    )

# hover = HoverTool(
#         tooltips=[
#             ("index", "$index"),
#             ("(x,y)", "($x, $y)"),
#             ("desc", "@desc"),
#         ]
#     )

hover = HoverTool(
        tooltips=[
            ("medianThickness", "@medianThickness"),
            ("fluctuAmpli", "@fluctuAmpli"),
        ]
    )

p = figure(plot_width=300, plot_height=300, tools=[hover], title="Mouse over the dots")

p.circle('medianThickness', 'fluctuAmpli', size=20, source=data)

show(p)

data

In [ ]:
data=pd.DataFrame(dict(
            x=[1, 2, 3, 4, 5],
            y=[2, 5, 8, 2, 7],
            desc=['A', 'A', 'C', 'd', 'E'],
        ))

Conditions = list(data['desc'].unique())
NCond = len(Conditions)
data['X'] = 0
for i in range(NCond):
    mask = data['desc'] == Conditions[i]
    data.loc[mask, ['X']] = i+1
data.index = data.x
data = data.drop(['x'], axis = 1)

data.reset_index(level=0, inplace=True)
data